<a id='home'></a>
### Contents

[Imports](#Imports)

[Dataloading](#Dataloading)

<a id='Imports'></a>
### Imports
[back to contents](#home)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os.path import join as p_join
import sys
from typing import List, Tuple, Dict, Set, Any, Optional, Callable
from pathlib import Path
sys.path.insert(0, str(Path(os.getcwd()).parent))
import requests
import json
import pickle
from tqdm import tqdm
try:
    from fabulous import color as fb_color
    color_print = lambda x, color='green': print(getattr(fb_color, color)(x)) if 'fb_color' in globals() else print(x)
except Exception as e:
    color_print = lambda x, color='green': print(x)

from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
pd.set_option('display.max_columns', 200)

In [4]:
from src.common_utils import get_container_ipaddr
from src.minio_utils import initialize_minio_client, load_json_from_minio
from src.db_utils import get_pg_engine
from src.processing import bfs_dict, minio_data_to_pandas

<a id='Dataloading'></a>
### Dataloading
[back to contents](#home)

In [5]:
MINIO_ACCESS_KEY = os.environ['MINIO_ACCESS_KEY']
MINIO_SECRET_KEY = os.environ['MINIO_SECRET_KEY']
POSTGRES_USER = os.environ['POSTGRES_USER']
POSTGRES_PASSWORD = os.environ['POSTGRES_PASSWORD']

In [6]:
minio_client = initialize_minio_client(
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY
)

all_fights_list = load_json_from_minio(
    minio_client=minio_client,
    bucket_name='ufc-raw-data',
    object_name='ufc_stats.json'
)
print(f"len(all_fights_list): {len(all_fights_list):,}")

len(all_fights_list): 7,230


In [7]:
k = 15
res_dict = bfs_dict(one_fight_dict=all_fights_list[k])
res_dict

{'names__1': 'Nazim Sadykhov',
 'names__2': 'Evan Elder',
 'fight_name': 'lightweight bout',
 'method': "TKO - Doctor's Stoppage",
 'round': '3',
 'time': '0:38',
 'timeformat': '3 Rnd (5-5-5)',
 'referee': 'Mark Smith',
 'date': 'February 18, 2023',
 'location': 'Las Vegas, Nevada, USA',
 'event_name': 'UFC Fight Night: Andrade vs. Blanchfield',
 'event_uri': 'http://www.ufcstats.com/event-details/f21a3d68fb9df387',
 'fight_uri': 'http://www.ufcstats.com/fight-details/d45d9c7d60f332da',
 'winloose_fighter1': 'W',
 'winloose_fighter2': 'L',
 'per_round_stats__round1__fighter__1': 'Nazim Sadykhov',
 'per_round_stats__round1__fighter__2': 'Evan Elder',
 'per_round_stats__round1__kd__1': '0',
 'per_round_stats__round1__kd__2': '1',
 'per_round_stats__round1__sig_str__1': '28 of 74',
 'per_round_stats__round1__sig_str__2': '35 of 78',
 'per_round_stats__round1__sig_str_perc__1': '37%',
 'per_round_stats__round1__sig_str_perc__2': '44%',
 'per_round_stats__round1__totalstr__1': '28 of 74',


In [8]:
res_df = minio_data_to_pandas(all_fights_list=all_fights_list)
print(f"res_df: {res_df.shape[0]:,} x {res_df.shape[1]:,}")
res_df.head(3)

7230it [00:01, 3873.34it/s]


res_df: 7,230 x 165


,names__1,names__2,fight_name,method,round,time,timeformat,referee,date,location,event_name,event_uri,fight_uri,winloose_fighter1,winloose_fighter2,per_round_stats__round1__fighter__1,per_round_stats__round1__fighter__2,per_round_stats__round1__kd__1,per_round_stats__round1__kd__2,per_round_stats__round1__sig_str__1,per_round_stats__round1__sig_str__2,per_round_stats__round1__sig_str_perc__1,per_round_stats__round1__sig_str_perc__2,per_round_stats__round1__totalstr__1,per_round_stats__round1__totalstr__2,per_round_stats__round1__td_perc__1,per_round_stats__round1__td_perc__2,per_round_stats__round1__sub_att__1,per_round_stats__round1__sub_att__2,per_round_stats__round1__rev__1,per_round_stats__round1__rev__2,per_round_stats__round1__ctrl__1,per_round_stats__round1__ctrl__2,per_round_stats__round1__head__1,per_round_stats__round1__head__2,per_round_stats__round1__body__1,per_round_stats__round1__body__2,per_round_stats__round1__leg__1,per_round_stats__round1__leg__2,per_round_stats__round1__distance__1,per_round_stats__round1__distance__2,per_round_stats__round1__clinch__1,per_round_stats__round1__clinch__2,per_round_stats__round1__ground__1,per_round_stats__round1__ground__2,per_round_stats__round2__fighter__1,per_round_stats__round2__fighter__2,per_round_stats__round2__kd__1,per_round_stats__round2__kd__2,per_round_stats__round2__sig_str__1,per_round_stats__round2__sig_str__2,per_round_stats__round2__sig_str_perc__1,per_round_stats__round2__sig_str_perc__2,per_round_stats__round2__totalstr__1,per_round_stats__round2__totalstr__2,per_round_stats__round2__td_perc__1,per_round_stats__round2__td_perc__2,per_round_stats__round2__sub_att__1,per_round_stats__round2__sub_att__2,per_round_stats__round2__rev__1,per_round_stats__round2__rev__2,per_round_stats__round2__ctrl__1,per_round_stats__round2__ctrl__2,per_round_stats__round2__head__1,per_round_stats__round2__head__2,per_round_stats__round2__body__1,per_round_stats__round2__body__2,per_round_stats__round2__leg__1,per_round_stats__round2__leg__2,per_round_stats__round2__distance__1,per_round_stats__round2__distance__2,per_round_stats__round2__clinch__1,per_round_stats__round2__clinch__2,per_round_stats__round2__ground__1,per_round_stats__round2__ground__2,per_round_stats__round3__fighter__1,per_round_stats__round3__fighter__2,per_round_stats__round3__kd__1,per_round_stats__round3__kd__2,per_round_stats__round3__sig_str__1,per_round_stats__round3__sig_str__2,per_round_stats__round3__sig_str_perc__1,per_round_stats__round3__sig_str_perc__2,per_round_stats__round3__totalstr__1,per_round_stats__round3__totalstr__2,per_round_stats__round3__td_perc__1,per_round_stats__round3__td_perc__2,per_round_stats__round3__sub_att__1,per_round_stats__round3__sub_att__2,per_round_stats__round3__rev__1,per_round_stats__round3__rev__2,per_round_stats__round3__ctrl__1,per_round_stats__round3__ctrl__2,per_round_stats__round3__head__1,per_round_stats__round3__head__2,per_round_stats__round3__body__1,per_round_stats__round3__body__2,per_round_stats__round3__leg__1,per_round_stats__round3__leg__2,per_round_stats__round3__distance__1,per_round_stats__round3__distance__2,per_round_stats__round3__clinch__1,per_round_stats__round3__clinch__2,per_round_stats__round3__ground__1,per_round_stats__round3__ground__2,per_round_stats__round4__fighter__1,per_round_stats__round4__fighter__2,per_round_stats__round4__kd__1,per_round_stats__round4__kd__2,per_round_stats__round4__sig_str__1,per_round_stats__round4__sig_str__2,per_round_stats__round4__sig_str_perc__1,per_round_stats__round4__sig_str_perc__2,per_round_stats__round4__totalstr__1,per_round_stats__round4__totalstr__2,per_round_stats__round4__td_perc__1,per_round_stats__round4__td_perc__2,per_round_stats__round4__sub_att__1,per_round_stats__round4__sub_att__2,per_round_stats__round4__rev__1,per_round_stats__round4__rev__2,per_round_stats__round4__ctrl__1,per_round_stats__round4__ctrl__2,per_round_stats__round4__head__1,per_round_stats__round4__head__2,per_round_st

In [9]:
eng = get_pg_engine()

with eng.connect() as conn:
    conn.execute('create schema if not exists raw_data')

res_df = minio_data_to_pandas(all_fights_list=all_fights_list)
res_df.to_sql(
    name='all_fights_info',
    con=eng,
    schema='raw_data',
    if_exists='replace',
    index=False
)
pd.read_sql('select * from raw_data.all_fights_info limit 3', eng)

7230it [00:01, 3985.86it/s]


,names__1,names__2,fight_name,method,round,time,timeformat,referee,date,location,event_name,event_uri,fight_uri,winloose_fighter1,winloose_fighter2,per_round_stats__round1__fighter__1,per_round_stats__round1__fighter__2,per_round_stats__round1__kd__1,per_round_stats__round1__kd__2,per_round_stats__round1__sig_str__1,per_round_stats__round1__sig_str__2,per_round_stats__round1__sig_str_perc__1,per_round_stats__round1__sig_str_perc__2,per_round_stats__round1__totalstr__1,per_round_stats__round1__totalstr__2,per_round_stats__round1__td_perc__1,per_round_stats__round1__td_perc__2,per_round_stats__round1__sub_att__1,per_round_stats__round1__sub_att__2,per_round_stats__round1__rev__1,per_round_stats__round1__rev__2,per_round_stats__round1__ctrl__1,per_round_stats__round1__ctrl__2,per_round_stats__round1__head__1,per_round_stats__round1__head__2,per_round_stats__round1__body__1,per_round_stats__round1__body__2,per_round_stats__round1__leg__1,per_round_stats__round1__leg__2,per_round_stats__round1__distance__1,per_round_stats__round1__distance__2,per_round_stats__round1__clinch__1,per_round_stats__round1__clinch__2,per_round_stats__round1__ground__1,per_round_stats__round1__ground__2,per_round_stats__round2__fighter__1,per_round_stats__round2__fighter__2,per_round_stats__round2__kd__1,per_round_stats__round2__kd__2,per_round_stats__round2__sig_str__1,per_round_stats__round2__sig_str__2,per_round_stats__round2__sig_str_perc__1,per_round_stats__round2__sig_str_perc__2,per_round_stats__round2__totalstr__1,per_round_stats__round2__totalstr__2,per_round_stats__round2__td_perc__1,per_round_stats__round2__td_perc__2,per_round_stats__round2__sub_att__1,per_round_stats__round2__sub_att__2,per_round_stats__round2__rev__1,per_round_stats__round2__rev__2,per_round_stats__round2__ctrl__1,per_round_stats__round2__ctrl__2,per_round_stats__round2__head__1,per_round_stats__round2__head__2,per_round_stats__round2__body__1,per_round_stats__round2__body__2,per_round_stats__round2__leg__1,per_round_stats__round2__leg__2,per_round_stats__round2__distance__1,per_round_stats__round2__distance__2,per_round_stats__round2__clinch__1,per_round_stats__round2__clinch__2,per_round_stats__round2__ground__1,per_round_stats__round2__ground__2,per_round_stats__round3__fighter__1,per_round_stats__round3__fighter__2,per_round_stats__round3__kd__1,per_round_stats__round3__kd__2,per_round_stats__round3__sig_str__1,per_round_stats__round3__sig_str__2,per_round_stats__round3__sig_str_perc__1,per_round_stats__round3__sig_str_perc__2,per_round_stats__round3__totalstr__1,per_round_stats__round3__totalstr__2,per_round_stats__round3__td_perc__1,per_round_stats__round3__td_perc__2,per_round_stats__round3__sub_att__1,per_round_stats__round3__sub_att__2,per_round_stats__round3__rev__1,per_round_stats__round3__rev__2,per_round_stats__round3__ctrl__1,per_round_stats__round3__ctrl__2,per_round_stats__round3__head__1,per_round_stats__round3__head__2,per_round_stats__round3__body__1,per_round_stats__round3__body__2,per_round_stats__round3__leg__1,per_round_stats__round3__leg__2,per_round_stats__round3__distance__1,per_round_stats__round3__distance__2,per_round_stats__round3__clinch__1,per_round_stats__round3__clinch__2,per_round_stats__round3__ground__1,per_round_stats__round3__ground__2,per_round_stats__round4__fighter__1,per_round_stats__round4__fighter__2,per_round_stats__round4__kd__1,per_round_stats__round4__kd__2,per_round_stats__round4__sig_str__1,per_round_stats__round4__sig_str__2,per_round_stats__round4__sig_str_perc__1,per_round_stats__round4__sig_str_perc__2,per_round_stats__round4__totalstr__1,per_round_stats__round4__totalstr__2,per_round_stats__round4__td_perc__1,per_round_stats__round4__td_perc__2,per_round_stats__round4__sub_att__1,per_round_stats__round4__sub_att__2,per_round_stats__round4__rev__1,per_round_stats__round4__rev__2,per_round_stats__round4__ctrl__1,per_round_stats__round4__ctrl__2,per_round_stats__round4__head__1,per_round_stats__round4__head__2,per_round_st

In [3]:
import json
tmp_dict = json.dumps("""{
   "nodes":[
      {
         "data":{
            "id":"HIRA",
            "altered":0,
            "rank":148,
            "cited":53,
            "uniprotdesc":"Cooperates with ASF1A to promote replication-independentchromatin assembly. Required for the periodic repression ofhistone gene transcription during the cell cycle. Required for theformation of senescence-associated heterochromatin foci (SAHF) andefficient senescence-associated cell cycle exit.",
            "isseed":false,
            "uniprot":"P54198",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":539.5,
            "y":641.7372431003224
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SERPINE1",
            "altered":0,
            "rank":5,
            "cited":7063,
            "uniprotdesc":"Serine protease inhibitor. This inhibitor acts as 'bait'for tissue plasminogen activator, urokinase, protein C andmatriptase-3/TMPRSS7. Its rapid interaction with PLAT may functionas a major control point in the regulation of fibrinolysis.",
            "isseed":false,
            "uniprot":"P05121",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":670.4301869950953,
            "y":229.70989985711336
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"FAS",
            "altered":0,
            "rank":38,
            "cited":1011,
            "uniprotdesc":"Receptor for TNFSF6/FASLG. The adapter molecule FADDrecruits caspase-8 to the activated receptor. The resulting death-inducing signaling complex (DISC) performs caspase-8 proteolyticactivation which initiates the subsequent cascade of caspases(aspartate-specific cysteine proteases) mediating apoptosis. FAS-mediated apoptosis may have a role in the induction of peripheraltolerance, in the antigen-stimulated suicide of mature T-cells, orboth. The secreted isoforms 2 to 6 block apoptosis (in vitro).",
            "isseed":false,
            "uniprot":"P25445",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":771.3915110914829,
            "y":613.8066148172879
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"H1F0",
            "altered":0,
            "rank":179,
            "cited":8,
            "uniprotdesc":"Histones H1 are necessary for the condensation ofnucleosome chains into higher-order structures. The H1F0 histonesare found in cells that are in terminal stages of differentiationor that have low rates of cell division.",
            "isseed":false,
            "uniprot":"P07305",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":396.564243092928,
            "y":545.1295331255458
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CHEK2",
            "altered":0,
            "rank":42,
            "cited":859,
            "uniprotdesc":"Serine/threonine-protein kinase which is required forcheckpoint-mediated cell cycle arrest, activation of DNA repairand apoptosis in response to the presence of DNA double-strandbreaks. May also negatively regulate cell cycle progression duringunperturbed cell cycles. Following activation, phosphorylatesnumerous effectors preferentially at the consensus sequence [L-X-R-X-X-S/T]. Regulates cell cycle checkpoint arrest throughphosphorylation of CDC25A, CDC25B and CDC25C, inhibiting theiractivity. Inhibition of CDC25 phosphatase activity leads toincreased inhibitory tyrosine phosphorylation of CDK-cyclincomplexes and blocks cell cycle progression. May alsophosphorylate NEK6 which is involved in G2/M cell cycle arrest.Regulates DNA repair through phosphorylation of BRCA2, enhancingthe association of RAD51 with chromatin which promotes DNA repairby homologous recombination. Also stimulates the transcription ofgenes involved in DNA repair (including BRCA2) through thephosphorylation and activation of the transcription factor FOXM1.Regulates apoptosis through the phosphorylation of p53/TP53, MDM4and PML. Phosphorylation of p53/TP53 at 'Ser-20' by CHEK2 mayalleviate inhibition by MDM2, leading to accumulation of activep53/TP53. Phosphorylation of MDM4 may also reduce degradation ofp53/TP53. Also controls the transcription of pro-apoptotic genesthrough phosphorylation of the transcription factor E2F1. Tumorsuppressor, it may also have a DNA damage-independent function inmitotic spindle assembly by phosphorylating BRCA1. Its absence maybe a cause of the chromosomal instability observed in some cancercells. ",
            "isseed":false,
            "uniprot":"O96017",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":665.61605725324,
            "y":327.88536532731405
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"COL18A1",
            "altered":0,
            "rank":157,
            "cited":30,
            "uniprotdesc":"COLA18A probably plays a major role in determining theretinal structure as well as in the closure of the neural tube.Endostatin potently inhibits endothelial cellproliferation and angiogenesis. May inhibit angiogenesis bybinding to the heparan sulfate proteoglycans involved in growthfactor signaling.",
            "isseed":false,
            "uniprot":"P39060",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":315.495548442639,
            "y":356.3503242429216
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CREBBP",
            "altered":0,
            "rank":24,
            "cited":2442,
            "uniprotdesc":"Acetylates histones, giving a specific tag fortranscriptional activation. Also acetylates non-histone proteins,like NCOA3 and FOXO1. Binds specifically to phosphorylated CREBand enhances its transcriptional activity toward cAMP-responsivegenes. Acts as a coactivator of ALX1 in the presence of EP300.Acts as a circadian transcriptional coactivator which enhances theactivity of the circadian transcriptional activators: NPAS2-ARNTL/BMAL1 and CLOCK-ARNTL/BMAL1 heterodimers.",
            "isseed":false,
            "uniprot":"Q92793",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":468.2392322240752,
            "y":311.08612174318614
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"FDXR",
            "altered":0,
            "rank":147,
            "cited":54,
            "uniprotdesc":"Serves as the first electron transfer protein in all themitochondrial P450 systems. Including cholesterol side chaincleavage in all steroidogenic tissues, steroid 11-betahydroxylation in the adrenal cortex, 25-OH-vitamin D3-24hydroxylation in the kidney, and sterol C-27 hydroxylation in theliver.",
            "isseed":false,
            "uniprot":"P22570",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":614.0116881932149,
            "y":209.57981996234878
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SMYD2",
            "altered":0,
            "rank":175,
            "cited":12,
            "uniprotdesc":"Protein-lysine N-methyltransferase that methylates bothhistones and non-histone proteins, including p53/TP53 and RB1.Specifically methylates histone H3 'Lys-4' (H3K4me) anddimethylates histone H3 'Lys-36' (H3K36me2). Shows even highermethyltransferase activity on p53/TP53. Monomethylates 'Lys-370'of p53/TP53, leading to decreased DNA-binding activity andsubsequent transcriptional regulation activity of p53/TP53.Monomethylates RB1 at 'Lys-860'. ",
            "isseed":false,
            "uniprot":"Q9NRG4",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":799.3458853307084,
            "y":560.8271840491047
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"ATR",
            "altered":0,
            "rank":52,
            "cited":628,
            "uniprotdesc":"Serine/threonine protein kinase which activatescheckpoint signaling upon genotoxic stresses such as ionizingradiation (IR), ultraviolet light (UV), or DNA replicationstalling, thereby acting as a DNA damage sensor. Recognizes thesubstrate consensus sequence [ST]-Q. Phosphorylates BRCA1, CHEK1,MCM2, RAD17, RPA2, SMC1 and p53/TP53, which collectively inhibitDNA replication and mitosis and promote DNA repair, recombinationand apoptosis. Phosphorylates 'Ser-139' of histone variantH2AX/H2AFX at sites of DNA damage, thereby regulating DNA damageresponse mechanism. Required for FANCD2 ubiquitination. Criticalfor maintenance of fragile site stability and efficient regulationof centrosome duplication. ",
            "isseed":false,
            "uniprot":"Q13535",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":696.7065384003552,
            "y":570.6754072575451
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HGF",
            "altered":0,
            "rank":4,
            "cited":7554,
            "uniprotdesc":"Potent mitogen for mature parenchymal hepatocyte cells,seems to be a hepatotrophic factor, and acts as a growth factorfor a broad spectrum of tissues and cell types. Activating ligandfor the receptor tyrosine kinase MET by binding to it andpromoting its dimerization. ",
            "isseed":false,
            "uniprot":"P14210",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":732.163947642761,
            "y":659.0776363377252
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"ATM",
            "altered":0,
            "rank":25,
            "cited":2298,
            "uniprotdesc":"Serine/threonine protein kinase which activatescheckpoint signaling upon double strand breaks (DSBs), apoptosisand genotoxic stresses such as ionizing ultraviolet A light (UVA),thereby acting as a DNA damage sensor. Recognizes the substrateconsensus sequence [ST]-Q. Phosphorylates 'Ser-139' of histonevariant H2AX/H2AFX at double strand breaks (DSBs), therebyregulating DNA damage response mechanism. Also plays a role inpre-B cell allelic exclusion, a process leading to expression of asingle immunoglobulin heavy chain allele to enforce clonality andmonospecific recognition by the B-cell antigen receptor (BCR)expressed on individual B-lymphocytes. After the introduction ofDNA breaks by the RAG complex on one immunoglobulin allele, actsby mediating a repositioning of the second allele topericentromeric heterochromatin, preventing accessibility to theRAG complex and recombination of the second allele. Also involvedin signal transduction and cell cycle control. May function as atumor suppressor. Necessary for activation of ABL1 and SAPK.Phosphorylates DYRK2, CHEK2, p53/TP53, FANCD2, NFKBIA, BRCA1,CTIP, nibrin (NBN), TERF1, RAD9 and DCLRE1C. May play a role invesicle and/or protein transport. Could play a role in T-celldevelopment, gonad and neurological function. Plays a role inreplication-dependent histone mRNA degradation. Binds DNA ends.Phosphorylation of DYRK2 in nucleus in response to genotoxicstress prevents its MDM2-mediated ubiquitination and subsequentproteasome degradation. Phosphorylates ATF2 which stimulates itsfunction in DNA damage response. ",
            "isseed":false,
            "uniprot":"Q13315",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":723.6214512404813,
            "y":517.2928623066505
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP63",
            "altered":0,
            "rank":37,
            "cited":1171,
            "uniprotdesc":"Acts as a sequence specific DNA binding transcriptionalactivator or repressor. The isoforms contain a varying set oftransactivation and auto-regulating transactivation inhibitingdomains thus showing an isoform specific activity. Isoform 2activates RIPK4 transcription. May be required in conjunction withTP73/p73 for initiation of p53/TP53 dependent apoptosis inresponse to genotoxic insults and the presence of activatedoncogenes. Involved in Notch signaling by probably inducing JAG1and JAG2. Plays a role in the regulation of epithelialmorphogenesis. The ratio of DeltaN-type and TA*-type isoforms maygovern the maintenance of epithelial stem cell compartments andregulate the initiation of epithelial stratification from theundifferentiated embryonal ectoderm. Required for limb formationfrom the apical ectodermal ridge. Activates transcription of thep21 promoter. ",
            "isseed":false,
            "uniprot":"Q9H3D4",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":554.5,
            "y":495.5
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"GPX1",
            "altered":0,
            "rank":13,
            "cited":3644,
            "uniprotdesc":"Protects the hemoglobin in erythrocytes from oxidativebreakdown.",
            "isseed":false,
            "uniprot":"P07203",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":412.4474306913114,
            "y":244.4618875964815
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TRIAP1",
            "altered":0,
            "rank":183,
            "cited":7,
            "uniprotdesc":"Involved in the modulation of the mitochondrialapoptotic pathway by ensuring the accumulation of cardiolipin (CL)in mitochondrial membranes. In vitro, the TRIAP1:PRELID1 complexmediates the transfer of phosphatidic acid (PA) between liposomesand probably functions as a PA transporter across themitochondrion intermembrane space to provide PA for CL synthesisin the inner membrane. Mediates cell survival by inhibitingactivation of caspase-9 which prevents induction of apoptosis.",
            "isseed":false,
            "uniprot":"O43715",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":554.5,
            "y":202.751489857487
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HIST1H1E",
            "altered":0,
            "rank":167,
            "cited":19,
            "uniprotdesc":"Histone H1 protein binds to linker DNA betweennucleosomes forming the macromolecular structure known as thechromatin fiber. Histones H1 are necessary for the condensation ofnucleosome chains into higher-order structured fibers. Acts alsoas a regulator of individual gene transcription through chromatinremodeling, nucleosome spacing and DNA methylation (Bysimilarity). ",
            "isseed":false,
            "uniprot":"P10412",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":509.931522689569,
            "y":636.667218490667
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HIST1H1D",
            "altered":0,
            "rank":184,
            "cited":6,
            "uniprotdesc":"Histone H1 protein binds to linker DNA betweennucleosomes forming the macromolecular structure known as thechromatin fiber. Histones H1 are necessary for the condensation ofnucleosome chains into higher-order structured fibers. Acts alsoas a regulator of individual gene transcription through chromatinremodeling, nucleosome spacing and DNA methylation (Bysimilarity). ",
            "isseed":false,
            "uniprot":"P16402",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":385.3785487595187,
            "y":517.2928623066505
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HIST1H1C",
            "altered":0,
            "rank":174,
            "cited":13,
            "uniprotdesc":"Histone H1 protein binds to linker DNA betweennucleosomes forming the macromolecular structure known as thechromatin fiber. Histones H1 are necessary for the condensation ofnucleosome chains into higher-order structured fibers. Acts alsoas a regulator of individual gene transcription through chromatinremodeling, nucleosome spacing and DNA methylation (Bysimilarity). ",
            "isseed":false,
            "uniprot":"P16403",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":379.058170659647,
            "y":487.9662051382201
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HIST1H1B",
            "altered":0,
            "rank":96,
            "cited":209,
            "uniprotdesc":"Histone H1 protein binds to linker DNA betweennucleosomes forming the macromolecular structure known as thechromatin fiber. Histones H1 are necessary for the condensation ofnucleosome chains into higher-order structured fibers. Acts alsoas a regulator of individual gene transcription through chromatinremodeling, nucleosome spacing and DNA methylation (Bysimilarity). ",
            "isseed":false,
            "uniprot":"P16401",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":381.59546915616875,
            "y":428.2362230943135
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HIST1H1A",
            "altered":0,
            "rank":173,
            "cited":13,
            "uniprotdesc":"Histone H1 protein binds to linker DNA betweennucleosomes forming the macromolecular structure known as thechromatin fiber. Histones H1 are necessary for the condensation ofnucleosome chains into higher-order structured fibers. Acts alsoas a regulator of individual gene transcription through chromatinremodeling, nucleosome spacing and DNA methylation (Bysimilarity). ",
            "isseed":false,
            "uniprot":"Q02539",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":443.3839427467599,
            "y":327.88536532731405
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53",
            "altered":0,
            "rank":0,
            "cited":24824,
            "uniprotdesc":"Acts as a tumor suppressor in many tumor types; inducesgrowth arrest or apoptosis depending on the physiologicalcircumstances and cell type. Involved in cell cycle regulation asa trans-activator that acts to negatively regulate cell divisionby controlling a set of genes required for this process. One ofthe activated genes is an inhibitor of cyclin-dependent kinases.Apoptosis induction seems to be mediated either by stimulation ofBAX and FAS antigen expression, or by repression of Bcl-2expression. In cooperation with mitochondrial PPIF is involved inactivating oxidative stress-induced necrosis; the function islargely independent of transcription. Induces the transcription oflong intergenic non-coding RNA p21 (lincRNA-p21) and lincRNA-Mkln1. LincRNA-p21 participates in TP53-dependent transcriptionalrepression leading to apoptosis and seem to have to effect oncell-cycle regulation. Implicated in Notch signaling cross-over.Prevents CDK7 kinase activity when associated to CAK complex inresponse to DNA damage, thus stopping cell cycle progression.Isoform 2 enhances the transactivation activity of isoform 1 fromsome but not all TP53-inducible promoters. Isoform 4 suppressestransactivation activity and impairs growth suppression mediatedby isoform 1. Isoform 7 inhibits isoform 1-mediated apoptosis.",
            "isseed":true,
            "uniprot":"P04637",
            "isvalid":true,
            "importance":4
         },
         "position":{
            "x":554.5,
            "y":465.5
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"GADD45A",
            "altered":0,
            "rank":98,
            "cited":198,
            "uniprotdesc":"In T-cells, functions as a regulator of p38 MAPKs byinhibiting p88 phosphorylation and activity (By similarity). Mightaffect PCNA interaction with some CDK (cell division proteinkinase) complexes; stimulates DNA excision repair in vitro andinhibits entry of cells into S phase. ",
            "isseed":false,
            "uniprot":"P24522",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":554.5,
            "y":375.5
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PML",
            "altered":0,
            "rank":35,
            "cited":1300,
            "uniprotdesc":"Functions via its association with PML-nuclear bodies(PML-NBs) in a wide range of important cellular processes,including tumor suppression, transcriptional regulation,apoptosis, senescence, DNA damage response, and viral defensemechanisms. Acts as the scaffold of PML-NBs allowing otherproteins to shuttle in and out, a process which is regulated bySUMO-mediated modifications and interactions. Isoform PML-4 has amultifaceted role in the regulation of apoptosis and growthsuppression: activates RB1 and inhibits AKT1 via interactions withPP1 and PP2A phosphatases respectively, negatively affects thePI3K pathway by inhibiting MTOR and activating PTEN, andpositively regulates p53/TP53 by acting at different levels (bypromoting its acetylation and phosphorylation and by inhibitingits MDM2-dependent degradation). Isoform PML-4 also: acts as atranscriptional repressor of TBX2 during cellular senescence andthe repression is dependent on a functional RBL2/E2F4 repressorcomplex, regulates double-strand break repair in gamma-irradiation-induced DNA damage responses via its interaction withWRN, acts as a negative regulator of telomerase by interactingwith TERT, and regulates PER2 nuclear localization and circadianfunction. Isoform PML-6 inhibits specifically the activity of thetetrameric form of PKM. The nuclear isoforms (isoform PML-1,isoform PML-2, isoform PML-3, isoform PML-4 and isoform PML-5) inconcert with SATB1 are involved in local chromatin-loop remodelingand gene expression regulation at the MHC-I locus. Isoform PML-2is required for efficient IFN-gamma induced MHC II genetranscription via regulation of CIITA. Cytoplasmic PML is involvedin the regulation of the TGF-beta signaling pathway. PML alsoregulates transcription activity of ELF4 and can act as animportant mediator for TNF-alpha- and IFN-alpha-mediatedinhibition of endothelial cell network formation and migration.Exhibits antiviral activity against both DNA and RNAviruses. The antiviral activity can involve one or severalisoform(s) and can be enhanced by the permanent PML-NB-associatedprotein DAXX or by the recruitment of p53/TP53 within thesestructures. Isoform PML-4 restricts varicella zoster virus (VZV)via sequestration of virion capsids in PML-NBs thereby preventingtheir nuclear egress and inhibiting formation of infectious virusparticles. The sumoylated isoform PML-4 restricts rabies virus byinhibiting viral mRNA and protein synthesis. The cytoplasmicisoform PML-14 can restrict herpes simplex virus-1 (HHV-1)replication by sequestering the viral E3 ubiquitin-protein ligaseICP0 in the cytoplasm. Isoform PML-6 shows restriction activitytowards human cytomegalovirus (HCMV) and influenza A virus strainsPR8(H1N1) and ST364(H3N2). Sumoylated isoform PML-4 and isoformPML-12 show antiviral activity against encephalomyocarditis virus(EMCV) by promoting nuclear sequestration of viral polymerase(P3D-POL) within PML NBs. Isoform PML-3 exhibits antiviralactivity against poliovirus by inducing apoptosis in infectedcells through the recruitment and the activation of p53/TP53 inthe PML-NBs. Isoform PML-3 represses human foamy virus (HFV)transcription by complexing the HFV transactivator, bel1/tas,preventing its binding to viral DNA. PML may positively regulateinfectious hepatitis C viral (HCV) production and isoform PML-2may enhance adenovirus transcription.",
            "isseed":false,
            "uniprot":"P29590",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":554.5,
            "y":555.5
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SUMO1",
            "altered":0,
            "rank":36,
            "cited":1235,
            "uniprotdesc":"Ubiquitin-like protein that can be covalently attachedto proteins as a monomer or a lysine-linked polymer. Covalentattachment via an isopeptide bond to its substrates requires prioractivation by the E1 complex SAE1-SAE2 and linkage to the E2enzyme UBE2I, and can be promoted by E3 ligases such as PIAS1-4,RANBP2 or CBX4. This post-translational modification on lysineresidues of proteins plays a crucial role in a number of cellularprocesses such as nuclear transport, DNA replication and repair,mitosis and signal transduction. Involved for instance intargeting RANGAP1 to the nuclear pore complex protein RANBP2.Polymeric SUMO1 chains are also susceptible to polyubiquitinationwhich functions as a signal for proteasomal degradation ofmodified proteins. May also regulate a network of genes involvedin palate development. ",
            "isseed":false,
            "uniprot":"P63165",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":554.5,
            "y":288.62556472004735
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PPP2CA",
            "altered":0,
            "rank":146,
            "cited":55,
            "uniprotdesc":"PP2A is the major phosphatase for microtubule-associatedproteins (MAPs). PP2A can modulate the activity of phosphorylase Bkinase casein kinase 2, mitogen-stimulated S6 kinase, and MAP-2kinase. Cooperates with SGOL2 to protect centromeric cohesin fromseparase-mediated cleavage in oocytes specifically during meiosisI (By similarity). Can dephosphorylate SV40 large T antigen andp53/TP53. Activates RAF1 by dephosphorylating it at 'Ser-259'.",
            "isseed":false,
            "uniprot":"P67775",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":727.4045308438311,
            "y":428.2362230943131
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PPP2CB",
            "altered":0,
            "rank":185,
            "cited":6,
            "uniprotdesc":"PP2A can modulate the activity of phosphorylase B kinasecasein kinase 2, mitogen-stimulated S6 kinase, and MAP-2 kinase.",
            "isseed":false,
            "uniprot":"P62714",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":584.3919246562758,
            "y":291.16974245431265
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAA1",
            "altered":0,
            "rank":78,
            "cited":330,
            "uniprotdesc":"Catalytic subunit of AMP-activated protein kinase(AMPK), an energy sensor protein kinase that plays a key role inregulating cellular energy metabolism. In response to reduction ofintracellular ATP levels, AMPK activates energy-producing pathwaysand inhibits energy-consuming processes: inhibits protein,carbohydrate and lipid biosynthesis, as well as cell growth andproliferation. AMPK acts via direct phosphorylation of metabolicenzymes, and by longer-term effects via phosphorylation oftranscription regulators. Also acts as a regulator of cellularpolarity by remodeling the actin cytoskeleton; probably byindirectly activating myosin. Regulates lipid synthesis byphosphorylating and inactivating lipid metabolic enzymes such asACACA, ACACB, GYS1, HMGCR and LIPE; regulates fatty acid andcholesterol synthesis by phosphorylating acetyl-CoA carboxylase(ACACA and ACACB) and hormone-sensitive lipase (LIPE) enzymes,respectively. Regulates insulin-signaling and glycolysis byphosphorylating IRS1, PFKFB2 and PFKFB3. AMPK stimulates glucoseuptake in muscle by increasing the translocation of the glucosetransporter SLC2A4/GLUT4 to the plasma membrane, possibly bymediating phosphorylation of TBC1D4/AS160. Regulates transcriptionand chromatin structure by phosphorylating transcriptionregulators involved in energy metabolism such as CRTC2/TORC2,FOXO3, histone H2B, HDAC5, MEF2C, MLXIPL/ChREBP, EP300, HNF4A,p53/TP53, SREBF1, SREBF2 and PPARGC1A. Acts as a key regulator ofglucose homeostasis in liver by phosphorylating CRTC2/TORC2,leading to CRTC2/TORC2 sequestration in the cytoplasm. In responseto stress, phosphorylates 'Ser-36' of histone H2B (H2BS36ph),leading to promote transcription. Acts as a key regulator of cellgrowth and proliferation by phosphorylating TSC2, RPTOR andATG1/ULK1: in response to nutrient limitation, negativelyregulates the mTORC1 complex by phosphorylating RPTOR component ofthe mTORC1 complex and by phosphorylating and activating TSC2. Inresponse to nutrient limitation, promotes autophagy byphosphorylating and activating ATG1/ULK1. AMPK also acts as aregulator of circadian rhythm by mediating phosphorylation ofCRY1, leading to destabilize it. May regulate the Wnt signalingpathway by phosphorylating CTNNB1, leading to stabilize it. Alsohas tau-protein kinase activity: in response to amyloid beta A4protein (APP) exposure, activated by CAMKK2, leading tophosphorylation of MAPT/TAU; however the relevance of such dataremains unclear in vivo. Also phosphorylates CFTR, EEF2K, KLC1,NOS3 and SLC12A1. ",
            "isseed":false,
            "uniprot":"Q13131",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":731.2150654362375,
            "y":457.9932361520795
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAA2",
            "altered":0,
            "rank":29,
            "cited":2074,
            "uniprotdesc":"Catalytic subunit of AMP-activated protein kinase(AMPK), an energy sensor protein kinase that plays a key role inregulating cellular energy metabolism. In response to reduction ofintracellular ATP levels, AMPK activates energy-producing pathwaysand inhibits energy-consuming processes: inhibits protein,carbohydrate and lipid biosynthesis, as well as cell growth andproliferation. AMPK acts via direct phosphorylation of metabolicenzymes, and by longer-term effects via phosphorylation oftranscription regulators. Also acts as a regulator of cellularpolarity by remodeling the actin cytoskeleton; probably byindirectly activating myosin. Regulates lipid synthesis byphosphorylating and inactivating lipid metabolic enzymes such asACACA, ACACB, GYS1, HMGCR and LIPE; regulates fatty acid andcholesterol synthesis by phosphorylating acetyl-CoA carboxylase(ACACA and ACACB) and hormone-sensitive lipase (LIPE) enzymes,respectively. Regulates insulin-signaling and glycolysis byphosphorylating IRS1, PFKFB2 and PFKFB3. AMPK stimulates glucoseuptake in muscle by increasing the translocation of the glucosetransporter SLC2A4/GLUT4 to the plasma membrane, possibly bymediating phosphorylation of TBC1D4/AS160. Regulates transcriptionand chromatin structure by phosphorylating transcriptionregulators involved in energy metabolism such as CRTC2/TORC2,FOXO3, histone H2B, HDAC5, MEF2C, MLXIPL/ChREBP, EP300, HNF4A,p53/TP53, SREBF1, SREBF2 and PPARGC1A. Acts as a key regulator ofglucose homeostasis in liver by phosphorylating CRTC2/TORC2,leading to CRTC2/TORC2 sequestration in the cytoplasm. In responseto stress, phosphorylates 'Ser-36' of histone H2B (H2BS36ph),leading to promote transcription. Acts as a key regulator of cellgrowth and proliferation by phosphorylating TSC2, RPTOR andATG1/ULK1: in response to nutrient limitation, negativelyregulates the mTORC1 complex by phosphorylating RPTOR component ofthe mTORC1 complex and by phosphorylating and activating TSC2. Inresponse to nutrient limitation, promotes autophagy byphosphorylating and activating ATG1/ULK1. AMPK also acts as aregulator of circadian rhythm by mediating phosphorylation ofCRY1, leading to destabilize it. May regulate the Wnt signalingpathway by phosphorylating CTNNB1, leading to stabilize it. Alsophosphorylates CFTR, EEF2K, KLC1, NOS3 and SLC12A1.",
            "isseed":false,
            "uniprot":"P54646",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":613.4239130642807,
            "y":298.7290842958664
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK11",
            "altered":0,
            "rank":127,
            "cited":81,
            "uniprotdesc":"Serine/threonine kinase which acts as an essentialcomponent of the MAP kinase signal transduction pathway. MAPK11 isone of the four p38 MAPKs which play an important role in thecascades of cellular responses evoked by extracellular stimulisuch as proinflammatory cytokines or physical stress leading todirect activation of transcription factors. Accordingly, p38 MAPKsphosphorylate a broad range of proteins and it has been estimatedthat they may have approximately 200 to 300 substrates each.MAPK11 functions are mostly redundant with those of MAPK14. Someof the targets are downstream kinases which are activated throughphosphorylation and further phosphorylate additional targets.RPS6KA5/MSK1 and RPS6KA4/MSK2 can directly phosphorylate andactivate transcription factors such as CREB1, ATF1, the NF-kappa-Bisoform RELA/NFKB3, STAT1 and STAT3, but can also phosphorylatehistone H3 and the nucleosomal protein HMGN1. RPS6KA5/MSK1 andRPS6KA4/MSK2 play important roles in the rapid induction ofimmediate-early genes in response to stress or mitogenic stimuli,either by inducing chromatin remodeling or by recruiting thetranscription machinery. On the other hand, two other kinasetargets, MAPKAPK2/MK2 and MAPKAPK3/MK3, participate in the controlof gene expression mostly at the post-transcriptional level, byphosphorylating ZFP36 (tristetraprolin) and ELAVL1, and byregulating EEF2K, which is important for the elongation of mRNAduring translation. MKNK1/MNK1 and MKNK2/MNK2, two other kinasesactivated by p38 MAPKs, regulate protein synthesis byphosphorylating the initiation factor EIF4E2. In the cytoplasm,the p38 MAPK pathway is an important regulator of proteinturnover. For example, CFLAR is an inhibitor of TNF-inducedapoptosis whose proteasome-mediated degradation is regulated byp38 MAPK phosphorylation. Ectodomain shedding of transmembraneproteins is regulated by p38 MAPKs as well. In response toinflammatory stimuli, p38 MAPKs phosphorylate the membrane-associated metalloprotease ADAM17. Such phosphorylation isrequired for ADAM17-mediated ectodomain shedding of TGF-alphafamily ligands, which results in the activation of EGFR signalingand cell proliferation. Additional examples of p38 MAPK substratesare the FGFR1. FGFR1 can be translocated from the extracellularspace into the cytosol and nucleus of target cells, and regulatesprocesses such as rRNA synthesis and cell growth. FGFR1translocation requires p38 MAPK activation. In the nucleus, manytranscription factors are phosphorylated and activated by p38MAPKs in response to different stimuli. Classical examples includeATF1, ATF2, ATF6, ELK1, PTPRH, DDIT3, TP53/p53 and MEF2C andMEF2A. The p38 MAPKs are emerging as important modulators of geneexpression by regulating chromatin modifiers and remodelers. Thepromoters of several genes involved in the inflammatory response,such as IL6, IL8 and IL12B, display a p38 MAPK-dependentenrichment of histone H3 phosphorylation on 'Ser-10' (H3S10ph) inLPS-stimulated myeloid cells. This phosphorylation enhances theaccessibility of the cryptic NF-kappa-B-binding sites markingpromoters for increased NF-kappa-B recruitment.",
            "isseed":false,
            "uniprot":"Q15759",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":640.7607677759248,
            "y":311.0861217431862
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP73",
            "altered":0,
            "rank":116,
            "cited":124,
            "uniprotdesc":"Participates in the apoptotic response to DNA damage.Isoforms containing the transactivation domain are pro-apoptotic,isoforms lacking the domain are anti-apoptotic and block thefunction of p53 and transactivating p73 isoforms. May be a tumorsuppressor protein. ",
            "isseed":false,
            "uniprot":"O15350",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":554.5,
            "y":435.5
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CCNB1",
            "altered":0,
            "rank":62,
            "cited":491,
            "uniprotdesc":"Essential for the control of the cell cycle at the G2/M(mitosis) transition. ",
            "isseed":false,
            "uniprot":"P14635",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":455.45477593090067,
            "y":612.0421763339963
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK12",
            "altered":0,
            "rank":131,
            "cited":79,
            "uniprotdesc":"Serine/threonine kinase which acts as an essentialcomponent of the MAP kinase signal transduction pathway. MAPK12 isone of the four p38 MAPKs which play an important role in thecascades of cellular responses evoked by extracellular stimulisuch as proinflammatory cytokines or physical stress leading todirect activation of transcription factors such as ELK1 and ATF2.Accordingly, p38 MAPKs phosphorylate a broad range of proteins andit has been estimated that they may have approximately 200 to 300substrates each. Some of the targets are downstream kinases suchas MAPKAPK2, which are activated through phosphorylation andfurther phosphorylate additional targets. Plays a role in myoblastdifferentiation and also in the down-regulation of cyclin D1 inresponse to hypoxia in adrenal cells suggesting MAPK12 may inhibitcell proliferation while promoting differentiation. PhosphorylatesDLG1. Following osmotic shock, MAPK12 in the cell nucleusincreases its association with nuclear DLG1, thereby causingdissociation of DLG1-SFPQ complexes. This function is independentof its catalytic activity and could affect mRNA processing and/orgene transcription to aid cell adaptation to osmolarity changes inthe environment. Regulates UV-induced checkpoint signaling andrepair of UV-induced DNA damage and G2 arrest after gamma-radiation exposure. MAPK12 is involved in the regulation of SLC2A1expression and basal glucose uptake in L6 myotubes; and negativelyregulates SLC2A4 expression and contraction-mediated glucoseuptake in adult skeletal muscle. C-Jun (JUN) phosphorylation isstimulated by MAPK14 and inhibited by MAPK12, leading to adistinct AP-1 regulation. MAPK12 is required for the normalkinetochore localization of PLK1, prevents chromosomal instabilityand supports mitotic cell viability. MAPK12-signaling is alsopositively regulating the expansion of transient amplifyingmyogenic precursor cells during muscle growth and regeneration.",
            "isseed":false,
            "uniprot":"P53778",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":569.5000000000001,
            "y":641.7372431003224
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK13",
            "altered":0,
            "rank":126,
            "cited":88,
            "uniprotdesc":"Serine/threonine kinase which acts as an essentialcomponent of the MAP kinase signal transduction pathway. MAPK13 isone of the four p38 MAPKs which play an important role in thecascades of cellular responses evoked by extracellular stimulisuch as proinflammatory cytokines or physical stress leading todirect activation of transcription factors such as ELK1 and ATF2.Accordingly, p38 MAPKs phosphorylate a broad range of proteins andit has been estimated that they may have approximately 200 to 300substrates each. MAPK13 is one of the less studied p38 MAPKisoforms. Some of the targets are downstream kinases such asMAPKAPK2, which are activated through phosphorylation and furtherphosphorylate additional targets. Plays a role in the regulationof protein translation by phosphorylating and inactivating EEF2K.Involved in cytoskeletal remodeling through phosphorylation ofMAPT and STMN1. Mediates UV irradiation induced up-regulation ofthe gene expression of CXCL14. Plays an important role in theregulation of epidermal keratinocyte differentiation, apoptosisand skin tumor development. Phosphorylates the transcriptionalactivator MYB in response to stress which leads to rapid MYBdegradation via a proteasome-dependent pathway. MAPK13 alsophosphorylates and down-regulates PRKD1 during regulation ofinsulin secretion in pancreatic beta cells.",
            "isseed":false,
            "uniprot":"O15264",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":687.2747400718758,
            "y":348.6435313966404
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK14",
            "altered":0,
            "rank":23,
            "cited":2528,
            "uniprotdesc":"Serine/threonine kinase which acts as an essentialcomponent of the MAP kinase signal transduction pathway. MAPK14 isone of the four p38 MAPKs which play an important role in thecascades of cellular responses evoked by extracellular stimulisuch as proinflammatory cytokines or physical stress leading todirect activation of transcription factors. Accordingly, p38 MAPKsphosphorylate a broad range of proteins and it has been estimatedthat they may have approximately 200 to 300 substrates each. Someof the targets are downstream kinases which are activated throughphosphorylation and further phosphorylate additional targets.RPS6KA5/MSK1 and RPS6KA4/MSK2 can directly phosphorylate andactivate transcription factors such as CREB1, ATF1, the NF-kappa-Bisoform RELA/NFKB3, STAT1 and STAT3, but can also phosphorylatehistone H3 and the nucleosomal protein HMGN1. RPS6KA5/MSK1 andRPS6KA4/MSK2 play important roles in the rapid induction ofimmediate-early genes in response to stress or mitogenic stimuli,either by inducing chromatin remodeling or by recruiting thetranscription machinery. On the other hand, two other kinasetargets, MAPKAPK2/MK2 and MAPKAPK3/MK3, participate in the controlof gene expression mostly at the post-transcriptional level, byphosphorylating ZFP36 (tristetraprolin) and ELAVL1, and byregulating EEF2K, which is important for the elongation of mRNAduring translation. MKNK1/MNK1 and MKNK2/MNK2, two other kinasesactivated by p38 MAPKs, regulate protein synthesis byphosphorylating the initiation factor EIF4E2. MAPK14 interactsalso with casein kinase II, leading to its activation throughautophosphorylation and further phosphorylation of TP53/p53. Inthe cytoplasm, the p38 MAPK pathway is an important regulator ofprotein turnover. For example, CFLAR is an inhibitor of TNF-induced apoptosis whose proteasome-mediated degradation isregulated by p38 MAPK phosphorylation. In a similar way, MAPK14phosphorylates the ubiquitin ligase SIAH2, regulating its activitytowards EGLN3. MAPK14 may also inhibit the lysosomal degradationpathway of autophagy by interfering with the intracellulartrafficking of the transmembrane protein ATG9. Another function ofMAPK14 is to regulate the endocytosis of membrane receptors bydifferent mechanisms that impinge on the small GTPase RAB5A. Inaddition, clathrin-mediated EGFR internalization induced byinflammatory cytokines and UV irradiation depends on MAPK14-mediated phosphorylation of EGFR itself as well as of RAB5Aeffectors. Ectodomain shedding of transmembrane proteins isregulated by p38 MAPKs as well. In response to inflammatorystimuli, p38 MAPKs phosphorylate the membrane-associatedmetalloprotease ADAM17. Such phosphorylation is required forADAM17-mediated ectodomain shedding of TGF-alpha family ligands,which results in the activation of EGFR signaling and cellproliferation. Another p38 MAPK substrate is FGFR1. FGFR1 can betranslocated from the extracellular space into the cytosol andnucleus of target cells, and regulates processes such as rRNAsynthesis and cell growth. FGFR1 translocation requires p38 MAPKactivation. In the nucleus, many transcription factors arephosphorylated and activated by p38 MAPKs in response to differentstimuli. Classical examples include ATF1, ATF2, ATF6, ELK1, PTPRH,DDIT3, TP53/p53 and MEF2C and MEF2A. The p38 MAPKs are emerging asimportant modulators of gene expression by regulating chromatinmodifiers and remodelers. The promoters of several genes involvedin the inflammatory response, such as IL6, IL8 and IL12B, displaya p38 MAPK-dependent enrichment of histone H3 phosphorylation on'Ser-10' (H3S10ph) in LPS-stimulated myeloid cells. Thisphosphorylation enhances the accessibility of the cryptic NF-kappa-B-binding sites marking promoters for increased NF-kappa-Brecruitment. Phosphorylates CDC25B and CDC25C which is requiredfor binding to 14-3-3 proteins and leads to initiation of a G2delay after ultraviolet radiation. Phosphorylates TIAR followingDNA damage, releasing TIAR from GADD45A mRNA and preventing mRNAdegradation. The p38 MAPKs may also have kinase-independent roles,which are thought to be due to the binding to targets in theabsence of phosphorylation. Protein O-Glc-N-acylation catalyzed bythe OGT is regulated by MAPK14, and, although OGT does not seem tobe phosphorylated by MAPK14, their interaction increases uponMAPK14 activation induced by glucose deprivation. This interactionmay regulate OGT activity by recruiting it to specific targetssuch as neurofilament H, stimulating its O-Glc-N-acylation.Required in mid-fetal development for the growth of embryo-derivedblood vessels in the labyrinth layer of the placenta. Also playsan essential role in developmental and stress-inducederythropoiesis, through regulation of EPO gene expression. IsoformMXI2 activation is stimulated by mitogens and oxidative stress andonly poorly phosphorylates ELK1 and ATF2. Isoform EXIP may play arole in the early onset of apoptosis. Phosphorylates S100A9 at'Thr-113'. ",
            "isseed":false,
            "uniprot":"Q16539",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":705.1137353610045,
            "y":372.7634453076487
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAB2",
            "altered":0,
            "rank":189,
            "cited":4,
            "uniprotdesc":"Non-catalytic subunit of AMP-activated protein kinase(AMPK), an energy sensor protein kinase that plays a key role inregulating cellular energy metabolism. In response to reduction ofintracellular ATP levels, AMPK activates energy-producing pathwaysand inhibits energy-consuming processes: inhibits protein,carbohydrate and lipid biosynthesis, as well as cell growth andproliferation. AMPK acts via direct phosphorylation of metabolicenzymes, and by longer-term effects via phosphorylation oftranscription regulators. Also acts as a regulator of cellularpolarity by remodeling the actin cytoskeleton; probably byindirectly activating myosin. Beta non-catalytic subunit acts as ascaffold on which the AMPK complex assembles, via its C-terminusthat bridges alpha (PRKAA1 or PRKAA2) and gamma subunits (PRKAG1,PRKAG2 or PRKAG3).",
            "isseed":false,
            "uniprot":"O43741",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":718.6198477055347,
            "y":399.5512210521357
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAB1",
            "altered":0,
            "rank":73,
            "cited":368,
            "uniprotdesc":"Non-catalytic subunit of AMP-activated protein kinase(AMPK), an energy sensor protein kinase that plays a key role inregulating cellular energy metabolism. In response to reduction ofintracellular ATP levels, AMPK activates energy-producing pathwaysand inhibits energy-consuming processes: inhibits protein,carbohydrate and lipid biosynthesis, as well as cell growth andproliferation. AMPK acts via direct phosphorylation of metabolicenzymes, and by longer-term effects via phosphorylation oftranscription regulators. Also acts as a regulator of cellularpolarity by remodeling the actin cytoskeleton; probably byindirectly activating myosin. Beta non-catalytic subunit acts as ascaffold on which the AMPK complex assembles, via its C-terminusthat bridges alpha (PRKAA1 or PRKAA2) and gamma subunits (PRKAG1,PRKAG2 or PRKAG3).",
            "isseed":false,
            "uniprot":"Q9Y478",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":476.5577136594006,
            "y":420.49999999999994
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"EP300",
            "altered":0,
            "rank":19,
            "cited":2835,
            "uniprotdesc":"Functions as histone acetyltransferase and regulatestranscription via chromatin remodeling. Acetylates all four corehistones in nucleosomes. Histone acetylation gives an epigenetictag for transcriptional activation. Mediates cAMP-gene regulationby binding specifically to phosphorylated CREB protein. Mediatesacetylation of histone H3 at 'Lys-122' (H3K122ac), a modificationthat localizes at the surface of the histone octamer andstimulates transcription, possibly by promoting nucleosomeinstability. Mediates acetylation of histone H3 at 'Lys-27'(H3K27ac). Also functions as acetyltransferase for nonhistonetargets. Acetylates 'Lys-131' of ALX1 and acts as its coactivatorin the presence of CREBBP. Acetylates SIRT2 and is proposed toindirectly increase the transcriptional activity of TP53 throughacetylation and subsequent attenuation of SIRT2 deacetylasefunction. Acetylates HDAC1 leading to its inactivation andmodulation of transcription. Acts as a TFAP2A-mediatedtranscriptional coactivator in presence of CITED2. Plays a role asa coactivator of NEUROD1-dependent transcription of the secretinand p21 genes and controls terminal differentiation of cells inthe intestinal epithelium. Promotes cardiac myocyte enlargement.Can also mediate transcriptional repression. Binds to and may beinvolved in the transforming capacity of the adenovirus E1Aprotein. In case of HIV-1 infection, it is recruited by the viralprotein Tat. Regulates Tat's transactivating activity and may helpinducing chromatin remodeling of proviral genes. Acetylates FOXO1and enhances its transcriptional activity. Acetylates BCL6 wichdisrupts its ability to recruit histone deacetylases and hindersits transcriptional repressor activity. Participates in CLOCK orNPAS2-regulated rhythmic gene transcription; exhibits a circadianassociation with CLOCK or NPAS2, correlating with increase inPER1/2 mRNA and histone H3 acetylation on the PER1/2 promoter.Acetylates MTA1 at 'Lys-626' which is essential for itstranscriptional coactivator activity (PubMed:10733570,PubMed:11430825, PubMed:11701890, PubMed:12402037,PubMed:12586840, PubMed:12929931, PubMed:14645221,PubMed:15186775, PubMed:15890677, PubMed:16617102,PubMed:16762839, PubMed:18722353, PubMed:18995842,PubMed:23415232, PubMed:23911289, PubMed:23934153,PubMed:8945521). Acetylates XBP1 isoform 2; acetylation increasesprotein stability of XBP1 isoform 2 and enhances itstranscriptional activity (By similarity).",
            "isseed":false,
            "uniprot":"Q09472",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":476.5577136594005,
            "y":510.5
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"IGBP1",
            "altered":0,
            "rank":125,
            "cited":89,
            "uniprotdesc":"Associated to surface IgM-receptor; may be involved insignal transduction. Involved in regulation of the catalyticactivity of the phosphatases PP2A, PP4 and PP6 by protecting theirpartially folded catalytic subunits from degradativepolyubiquitination until they associate with regulatory subunits.",
            "isseed":false,
            "uniprot":"P78318",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":729.941829340353,
            "y":487.96620513822
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"FBXO11",
            "altered":0,
            "rank":188,
            "cited":5,
            "uniprotdesc":"Substrate recognition component of a SCF (SKP1-CUL1-F-box protein) E3 ubiquitin-protein ligase complex which mediatesthe ubiquitination and subsequent proteasomal degradation oftarget proteins, such as DTL/CDT2, BCL6 and PRDM1/BLIMP1. TheSCF(FBXO11) complex mediates ubiquitination and degradation ofBCL6, thereby playing a role in the germinal center B-cellsterminal differentiation toward memory B-cells and plasma cells.The SCF(FBXO11) complex also mediates ubiquitination anddegradation of DTL, an important step for the regulation of TGF-beta signaling, cell migration and the timing of the cell-cycleprogression and exit. Binds to and neddylates phosphorylatedp53/TP53, inhibiting its transcriptional activity. SCF(FBXO11)does not seem to direct ubiquitination of p53/TP53.",
            "isseed":false,
            "uniprot":"Q86XK2",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":481.64519932573637,
            "y":626.6730246484053
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HMGB1",
            "altered":0,
            "rank":54,
            "cited":617,
            "uniprotdesc":"DNA binding proteins that associates with chromatin andhas the ability to bend DNA. Binds preferentially single-strandedDNA. Involved in V(D)J recombination by acting as a cofactor ofthe RAG complex. Acts by stimulating cleavage and RAG proteinbinding at the 23 bp spacer of conserved recombination signalsequences (RSS). Heparin-binding protein that has a role in theextension of neurite-type cytoplasmic processes in developingcells (By similarity). ",
            "isseed":false,
            "uniprot":"P09429",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":712.435756907072,
            "y":545.1295331255459
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"NEDD8",
            "altered":0,
            "rank":205,
            "cited":236,
            "uniprotdesc":"Ubiquitin-like protein which plays an important role incell cycle control and embryogenesis. Covalent attachment to itssubstrates requires prior activation by the E1 complex UBE1C-APPBP1 and linkage to the E2 enzyme UBE2M. Attachment of NEDD8 tocullins activates their associated E3 ubiquitin ligase activity,and thus promotes polyubiquitination and proteasomal degradationof cyclins and other regulatory proteins.",
            "isseed":false,
            "uniprot":"Q15843",
            "isvalid":true,
            "importance":2
         },
         "position":{
            "x":432.1137033013474,
            "y":593.195576415127
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"ASF1A",
            "altered":0,
            "rank":161,
            "cited":27,
            "uniprotdesc":"Histone chaperone that facilitates histone depositionand histone exchange and removal during nucleosome assembly anddisassembly. Cooperates with chromatin assembly factor 1 (CAF-1)to promote replication-dependent chromatin assembly and with HIRAto promote replication-independent chromatin assembly. Requiredfor the formation of senescence-associated heterochromatin foci(SAHF) and efficient senescence-associated cell cycle exit.",
            "isseed":false,
            "uniprot":"Q9Y294",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":412.29346159964496,
            "y":570.6754072575452
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"KAT8",
            "altered":0,
            "rank":199,
            "cited":0,
            "uniprotdesc":"Histone acetyltransferase which may be involved intranscriptional activation. May influence the function of ATM. Aspart of the MSL complex it is involved in acetylation ofnucleosomal histone H4 producing specifically H4K16ac. As part ofthe NSL complex it may be involved in acetylation of nucleosomalhistone H4 on several lysine residues. That activity is lessspecific than the one of the MSL complex. Can also acetylateTP53/p53 at 'Lys-120'. ",
            "isseed":false,
            "uniprot":"Q9H7Z6",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":452.21985952208775,
            "y":707.5238675129558
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HTT",
            "altered":0,
            "rank":43,
            "cited":858,
            "uniprotdesc":"May play a role in microtubule-mediated transport orvesicle function.",
            "isseed":false,
            "uniprot":"P42858",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":632.4422863405995,
            "y":510.5
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"WRN",
            "altered":0,
            "rank":53,
            "cited":621,
            "uniprotdesc":"Multifunctional enzyme that has both magnesium and ATP-dependent DNA-helicase activity and 3'->5' exonuclease activitytowards double-stranded DNA with a 5'-overhang. Has no nucleaseactivity towards single-stranded DNA or blunt-ended double-stranded DNA. Binds preferentially to DNA substrates containingalternate secondary structures, such as replication forks andHolliday junctions. May play an important role in the dissociationof joint DNA molecules that can arise as products of homologousrecombination, at stalled replication forks or during DNA repair.Alleviates stalling of DNA polymerases at the site of DNA lesions.Important for genomic integrity. Plays a role in the formation ofDNA replication focal centers; stably associates with focielements generating binding sites for RP-A (By similarity). Playsa role in double-strand break repair after gamma-irradiation.",
            "isseed":false,
            "uniprot":"Q14191",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":676.8862966986526,
            "y":593.1955764151269
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CDKN1A",
            "altered":0,
            "rank":9,
            "cited":4784,
            "uniprotdesc":"May be the important intermediate by which p53/TP53mediates its role as an inhibitor of cellular proliferation inresponse to DNA damage. Binds to and inhibits cyclin-dependentkinase activity, preventing phosphorylation of critical cyclin-dependent kinase substrates and blocking cell cycle progression.Functions in the nuclear localization and assembly of cyclin D-CDK4 complex and promotes its kinase activity towards RB1. Athigher stoichiometric ratios, inhibits the kinase activity of thecyclin D-CDK4 complex. ",
            "isseed":false,
            "uniprot":"P38936",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":653.5452240690994,
            "y":612.0421763339962
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PLK3",
            "altered":0,
            "rank":115,
            "cited":130,
            "uniprotdesc":"Serine/threonine-protein kinase involved in cell cycleregulation, response to stress and Golgi disassembly. Polo-likekinases act by binding and phosphorylating proteins are thatalready phosphorylated on a specific motif recognized by the POLObox domains. Phosphorylates ATF2, BCL2L1, CDC25A, CDC25C, CHEK2,HIF1A, JUN, p53/TP53, p73/TP73, PTEN, TOP2A and VRK1. Involved incell cycle regulation: required for entry into S phase andcytokinesis. Phosphorylates BCL2L1, leading to regulate the G2checkpoint and progression to cytokinesis during mitosis. Plays akey role in response to stress: rapidly activated upon stressstimulation, such as ionizing radiation, reactive oxygen species(ROS), hyperosmotic stress, UV irradiation and hypoxia. Involvedin DNA damage response and G1/S transition checkpoint byphosphorylating CDC25A, p53/TP53 and p73/TP73. Phosphorylatesp53/TP53 in response to reactive oxygen species (ROS), therebypromoting p53/TP53-mediated apoptosis. Phosphorylates CHEK2 inresponse to DNA damage, promoting the G2/M transition checkpoint.Phosphorylates the transcription factor p73/TP73 in response toDNA damage, leading to inhibit p73/TP73-mediated transcriptionalactivation and pro-apoptotic functions. Phosphorylates HIF1A andJUN is response to hypoxia. Phosphorylates ATF2 followinghyperosmotic stress in corneal epithelium. Also involved in Golgidisassembly during the cell cycle: part of a MEK1/MAP2K1-dependentpathway that induces Golgi fragmentation during mitosis bymediating phosphorylation of VRK1. May participate in endomitoticcell cycle, a form of mitosis in which both karyokinesis andcytokinesis are interrupted and is a hallmark of megakaryocytedifferentiation, via its interaction with CIB1.",
            "isseed":false,
            "uniprot":"Q9H4B4",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":377.7849345637625,
            "y":457.9932361520796
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"DYRK1A",
            "altered":0,
            "rank":128,
            "cited":81,
            "uniprotdesc":"May play a role in a signaling pathway regulatingnuclear functions of cell proliferation. Modulates alternativesplicing by phosphorylating the splice factor SRSF6 (Bysimilarity). Phosphorylates serine, threonine and tyrosineresidues in its sequence and in exogenous substrates such as CRY2,FOXO1, SRSF6 and SIRT1. Exhibits a sugstrate preference forproline at position P+1 and arginine at position P-3.",
            "isseed":false,
            "uniprot":"Q13627",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":584.451073117784,
            "y":204.46415418337006
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAG1",
            "altered":0,
            "rank":177,
            "cited":11,
            "uniprotdesc":"AMP/ATP-binding subunit of AMP-activated protein kinase(AMPK), an energy sensor protein kinase that plays a key role inregulating cellular energy metabolism. In response to reduction ofintracellular ATP levels, AMPK activates energy-producing pathwaysand inhibits energy-consuming processes: inhibits protein,carbohydrate and lipid biosynthesis, as well as cell growth andproliferation. AMPK acts via direct phosphorylation of metabolicenzymes, and by longer-term effects via phosphorylation oftranscription regulators. Also acts as a regulator of cellularpolarity by remodeling the actin cytoskeleton; probably byindirectly activating myosin. Gamma non-catalytic subunit mediatesbinding to AMP, ADP and ATP, leading to activate or inhibit AMPK:AMP-binding results in allosteric activation of alpha catalyticsubunit (PRKAA1 or PRKAA2) both by inducing phosphorylation andpreventing dephosphorylation of catalytic subunits. ADP alsostimulates phosphorylation, without stimulating alreadyphosphorylated catalytic subunit. ATP promotes dephosphorylationof catalytic subunit, rendering the AMPK enzyme inactive.",
            "isseed":false,
            "uniprot":"P54619",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":627.3548006742639,
            "y":626.6730246484052
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CASP6",
            "altered":0,
            "rank":104,
            "cited":161,
            "uniprotdesc":"Involved in the activation cascade of caspasesresponsible for apoptosis execution. Cleaves poly(ADP-ribose)polymerase in vitro, as well as lamins. Overexpression promotesprogrammed cell death.",
            "isseed":false,
            "uniprot":"P55212",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":642.7964774016498,
            "y":218.03179666759164
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAX",
            "altered":0,
            "rank":158,
            "cited":28,
            "uniprotdesc":"Transcription regulator. Forms a sequence-specific DNA-binding protein complex with MYC or MAD which recognizes the coresequence 5'-CAC[GA]TG-3'. The MYC:MAX complex is a transcriptionalactivator, whereas the MAD:MAX complex is a repressor. May represstranscription via the recruitment of a chromatin remodelingcomplex containing H3 'Lys-9' histone methyltransferase activity.",
            "isseed":false,
            "uniprot":"P61244",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":524.5489268822162,
            "y":204.46415418337
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"FOS",
            "altered":0,
            "rank":61,
            "cited":496,
            "uniprotdesc":"Nuclear phosphoprotein which forms a tight but non-covalently linked complex with the JUN/AP-1 transcription factor.In the heterodimer, FOS and JUN/AP-1 basic regions each seems tointeract with symmetrical DNA half sites. On TGF-beta activation,forms a multimeric SMAD3/SMAD4/JUN/FOS complex at the AP1/SMAD-binding site to regulate TGF-beta-mediated signaling. Has acritical function in regulating the development of cells destinedto form and maintain the skeleton. It is thought to have animportant role in signal transduction, cell proliferation anddifferentiation. In growing cells, activates phospholipidsynthesis, possibly by activating CDS1 and PI4K2A. This activityrequires Tyr-dephosphorylation and association with theendoplasmic reticulum. ",
            "isseed":false,
            "uniprot":"P01100",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":696.5525693086885,
            "y":244.46188759648143
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53I3",
            "altered":0,
            "rank":152,
            "cited":39,
            "uniprotdesc":"May be involved in the generation of reactive oxygenspecies (ROS). Has low NADPH-dependent beta-naphthoquinonereductase activity, with a preference for 1,2-beta-naphthoquinoneover 1,4-beta-naphthoquinone. Has low NADPH-dependent diaminereductase activity (in vitro). ",
            "isseed":false,
            "uniprot":"Q53FA7",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":720.823079139937,
            "y":262.0954451652557
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRMT1",
            "altered":0,
            "rank":206,
            "cited":189,
            "uniprotdesc":"Arginine methyltransferase that methylates (mono andasymmetric dimethylation) the guanidino nitrogens of arginylresidues present in proteins such as ESR1, histone H2, H3 and H4,PIAS1, HNRNPA1, HNRNPD, NFATC2IP, SUPT5H, TAF15 and EWS.Constitutes the main enzyme that mediates monomethylation andasymmetric dimethylation of histone H4 'Arg-4' (H4R3me1 andH4R3me2a, respectively), a specific tag for epigenetictranscriptional activation. Together with dimethylated PIAS1,represses STAT1 transcriptional activity, in the late phase ofinterferon gamma (IFN-gamma) signaling. May be involved in theregulation of TAF15 transcriptional activity, act as an activatorof estrogen receptor (ER)-mediated transactivation, play a keyrole in neurite outgrowth and act as a negative regulator ofmegakaryocytic differentiation, by modulating p38 MAPK pathway.Methylates FOXO1 and retains it in the nucleus increasing itstranscriptional activity. ",
            "isseed":false,
            "uniprot":"Q99873",
            "isvalid":true,
            "importance":2
         },
         "position":{
            "x":742.9253132762292,
            "y":282.3806921734763
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PCBP4",
            "altered":0,
            "rank":165,
            "cited":20,
            "uniprotdesc":"Single-stranded nucleic acid binding protein that bindspreferentially to oligo dC. ",
            "isseed":false,
            "uniprot":"P57723",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":762.5711352945876,
            "y":305.05317940410396
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRMT5",
            "altered":0,
            "rank":118,
            "cited":111,
            "uniprotdesc":"Arginine methyltransferase that can both catalyze theformation of omega-N monomethylarginine (MMA) and symmetricaldimethylarginine (sDMA), with a preference for the formation ofMMA. Specifically mediates the symmetrical dimethylation ofarginine residues in the small nuclear ribonucleoproteins Sm D1(SNRPD1) and Sm D3 (SNRPD3); such methylation being required forthe assembly and biogenesis of snRNP core particles. MethylatesSUPT5H. Mono- and dimethylates arginine residues of myelin basicprotein (MBP) in vitro. Plays a role in the assembly of snRNP coreparticles. May play a role in cytokine-activated transductionpathways. Negatively regulates cyclin E1 promoter activity andcellular proliferation. May regulate the SUPT5H transcriptionalelongation properties. May be part of a pathway that is connectedto a chloride current, possibly through cytoskeletalrearrangement. Methylates histone H2A and H4 'Arg-3' during germcell development. Methylates histone H3 'Arg-8', which may represstranscription. Methylates the Piwi proteins (PIWIL1, PIWIL2 andPIWIL4), methylation of Piwi proteins being required for theinteraction with Tudor domain-containing proteins and subsequentlocalization to the meiotic nuage. Methylates RPS10. AttenuatesEGF signaling through the MAPK1/MAPK3 pathway acting at 2 levels.First, monomethylates EGFR; this enhances EGFR 'Tyr-1197'phosphorylation and PTPN6 recruitment, eventually leading toreduced SOS1 phosphorylation. Second, methylates RAF1 and probablyBRAF, hence destabilizing these 2 signaling proteins and reducingtheir catalytic activity. Required for induction of E-selectin andVCAM-1, on the endothelial cells surface at sites of inflammation.Methylates HOXA9. Methylates and regulates SRGAP2 which isinvolved in cell migration and differentiation. Acts as atranscriptional corepressor in CRY1-mediated repression of thecore circadian component PER1 by regulating the H4R3 dimethylationat the PER1 promoter. ",
            "isseed":false,
            "uniprot":"O14744",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":779.5044318605908,
            "y":329.8173363129873
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"E4F1",
            "altered":0,
            "rank":150,
            "cited":47,
            "uniprotdesc":"May function as a transcriptional repressor. May alsofunction as a ubiquitin ligase mediating ubiquitination ofchromatin-associated TP53. Functions in cell survival andproliferation through control of the cell cycle. Functions in thep53 and pRB tumor suppressor pathways and regulates the cyclinCCNA2 transcription.Identified as a cellular target of the adenoviraloncoprotein E1A, it is required for both transcriptionalactivation and repression of viral genes.",
            "isseed":false,
            "uniprot":"Q66K89",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":793.504451557361,
            "y":356.35032424292166
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CASP1",
            "altered":0,
            "rank":64,
            "cited":473,
            "uniprotdesc":"Thiol protease that cleaves IL-1 beta between an Asp andan Ala, releasing the mature cytokine which is involved in avariety of inflammatory processes. Important for defense againstpathogens. Cleaves and activates sterol regulatory element bindingproteins (SREBPs). Can also promote apoptosis.",
            "isseed":false,
            "uniprot":"P29466",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":804.3886827178802,
            "y":384.3062451192652
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKCA",
            "altered":0,
            "rank":84,
            "cited":275,
            "uniprotdesc":"Calcium-activated, phospholipid- and diacylglycerol(DAG)-dependent serine/threonine-protein kinase that is involvedin positive and negative regulation of cell proliferation,apoptosis, differentiation, migration and adhesion, tumorigenesis,cardiac hypertrophy, angiogenesis, platelet function andinflammation, by directly phosphorylating targets such as RAF1,BCL2, CSPG4, TNNT2/CTNT, or activating signaling cascade involvingMAPK1/3 (ERK1/2) and RAP1GAP. Involved in cell proliferation andcell growth arrest by positive and negative regulation of the cellcycle. Can promote cell growth by phosphorylating and activatingRAF1, which mediates the activation of the MAPK/ERK signalingcascade, and/or by up-regulating CDKN1A, which facilitates activecyclin-dependent kinase (CDK) complex formation in glioma cells.In intestinal cells stimulated by the phorbol ester PMA, cantrigger a cell cycle arrest program which is associated with theaccumulation of the hyper-phosphorylated growth-suppressive formof RB1 and induction of the CDK inhibitors CDKN1A and CDKN1B.Exhibits anti-apoptotic function in glioma cells and protects themfrom apoptosis by suppressing the p53/TP53-mediated activation ofIGFBP3, and in leukemia cells mediates anti-apoptotic action byphosphorylating BCL2. During macrophage differentiation induced bymacrophage colony-stimulating factor (CSF1), is translocated tothe nucleus and is associated with macrophage development. Afterwounding, translocates from focal contacts to lamellipodia andparticipates in the modulation of desmosomal adhesion. Plays arole in cell motility by phosphorylating CSPG4, which inducesassociation of CSPG4 with extensive lamellipodia at the cellperiphery and polarization of the cell accompanied by increases incell motility. Is highly expressed in a number of cancer cellswhere it can act as a tumor promoter and is implicated inmalignant phenotypes of several tumors such as gliomas and breastcancers. Negatively regulates myocardial contractility andpositively regulates angiogenesis, platelet aggregation andthrombus formation in arteries. Mediates hypertrophic growth ofneonatal cardiomyocytes, in part through a MAPK1/3 (ERK1/2)-dependent signaling pathway, and upon PMA treatment, is requiredto induce cardiomyocyte hypertrophy up to heart failure and death,by increasing protein synthesis, protein-DNA ratio and cellsurface area. Regulates cardiomyocyte function by phosphorylatingcardiac troponin T (TNNT2/CTNT), which induces significantreduction in actomyosin ATPase activity, myofilament calciumsensitivity and myocardial contractility. In angiogenesis, isrequired for full endothelial cell migration, adhesion tovitronectin (VTN), and vascular endothelial growth factor A(VEGFA)-dependent regulation of kinase activation and vasculartube formation. Involved in the stabilization of VEGFA mRNA atpost-transcriptional level and mediates VEGFA-induced cellproliferation. In the regulation of calcium-induced plateletaggregation, mediates signals from the CD36/GP4 receptor forgranule release, and activates the integrin heterodimer ITGA2B-ITGB3 through the RAP1GAP pathway for adhesion. During response tolipopolysaccharides (LPS), may regulate selective LPS-inducedmacrophage functions involved in host defense and inflammation.But in some inflammatory responses, may negatively regulate NF-kappa-B-induced genes, through IL1A-dependent induction of NF-kappa-B inhibitor alpha (NFKBIA/IKBA). Upon stimulation with 12-O-tetradecanoylphorbol-13-acetate (TPA), phosphorylates EIF4G1,which modulates EIF4G1 binding to MKNK1 and may be involved in theregulation of EIF4E phosphorylation. Phosphorylates KIT, leadingto inhibition of KIT activity. Phosphorylates ATF2 which promotescooperation between ATF2 and JUN, activating transcription.",
            "isseed":false,
            "uniprot":"P17252",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":812.0152327436849,
            "y":413.32065076032404
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CDK1",
            "altered":0,
            "rank":27,
            "cited":2149,
            "uniprotdesc":"Plays a key role in the control of the eukaryotic cellcycle by modulating the centrosome cycle as well as mitotic onset;promotes G2-M transition, and regulates G1 progress and G1-Stransition via association with multiple interphase cyclins.Required in higher cells for entry into S-phase and mitosis.Phosphorylates PARVA/actopaxin, APC, AMPH, APC, BARD1, Bcl-xL/BCL2L1, BRCA2, CALD1, CASP8, CDC7, CDC20, CDC25A, CDC25C,CC2D1A, CSNK2 proteins/CKII, FZR1/CDH1, CDK7, CEBPB, CHAMP1,DMD/dystrophin, EEF1 proteins/EF-1, EZH2, KIF11/EG5, EGFR, FANCG,FOS, GFAP, GOLGA2/GM130, GRASP1, UBE2A/hHR6A, HIST1H1proteins/histone H1, HMGA1, HIVEP3/KRC, LMNA, LMNB, LMNC, LBR,LATS1, MAP1B, MAP4, MARCKS, MCM2, MCM4, MKLP1, MYB, NEFH, NFIC,NPC/nuclear pore complex, PITPNM1/NIR2, NPM1, NCL, NUCKS1,NPM1/numatrin, ORC1, PRKAR2A, EEF1E1/p18, EIF3F/p47, p53/TP53,NONO/p54NRB, PAPOLA, PLEC/plectin, RB1, UL40/R2, RAB4A, RAP1GAP,RCC1, RPS6KB1/S6K1, KHDRBS1/SAM68, ESPL1, SKI, BIRC5/survivin,STIP1, TEX14, beta-tubulins, MAPT/TAU, NEDD1, VIM/vimentin, TK1,FOXO1, RUNX1/AML1, SIRT2 and RUNX2. CDK1/CDC2-cyclin-B controlspronuclear union in interphase fertilized eggs. Essential forearly stages of embryonic development. During G2 and earlymitosis, CDC25A/B/C-mediated dephosphorylation activatesCDK1/cyclin complexes which phosphorylate several substrates thattrigger at least centrosome separation, Golgi dynamics, nuclearenvelope breakdown and chromosome condensation. Once chromosomesare condensed and aligned at the metaphase plate, CDK1 activity isswitched off by WEE1- and PKMYT1-mediated phosphorylation to allowsister chromatid separation, chromosome decondensation,reformation of the nuclear envelope and cytokinesis. Inactivatedby PKR/EIF2AK2- and WEE1-mediated phosphorylation upon DNA damageto stop cell cycle and genome replication at the G2 checkpointthus facilitating DNA repair. Reactivated after successful DNArepair through WIP1-dependent signaling leading to CDC25A/B/C-mediated dephosphorylation and restoring cell cycle progression.In proliferating cells, CDK1-mediated FOXO1 phosphorylation at theG2-M phase represses FOXO1 interaction with 14-3-3 proteins andthereby promotes FOXO1 nuclear accumulation and transcriptionfactor activity, leading to cell death of postmitotic neurons. Thephosphorylation of beta-tubulins regulates microtubule dynamicsduring mitosis. NEDD1 phosphorylation promotes PLK1-mediated NEDD1phosphorylation and subsequent targeting of the gamma-tubulin ringcomplex (gTuRC) to the centrosome, an important step for spindleformation. In addition, CC2D1A phosphorylation regulates CC2D1Aspindle pole localization and association with SCC1/RAD21 andcentriole cohesion during mitosis. The phosphorylation of Bcl-xL/BCL2L1 after prolongated G2 arrest upon DNA damage triggersapoptosis. In contrast, CASP8 phosphorylation during mitosisprevents its activation by proteolysis and subsequent apoptosis.This phosphorylation occurs in cancer cell lines, as well as inprimary breast tissues and lymphocytes. EZH2 phosphorylationpromotes H3K27me3 maintenance and epigenetic gene silencing. CALD1phosphorylation promotes Schwann cell migration during peripheralnerve regeneration. ",
            "isseed":false,
            "uniprot":"P06493",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":816.2846778918833,
            "y":443.01529401675197
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CDK9",
            "altered":0,
            "rank":66,
            "cited":448,
            "uniprotdesc":"Protein kinase involved in the regulation oftranscription. Member of the cyclin-dependent kinase pair(CDK9/cyclin-T) complex, also called positive transcriptionelongation factor b (P-TEFb), which facilitates the transitionfrom abortive to productive elongation by phosphorylating the CTD(C-terminal domain) of the large subunit of RNA polymerase II(RNAP II) POLR2A, SUPT5H and RDBP. This complex is inactive whenin the 7SK snRNP complex form. Phosphorylates EP300, MYOD1,RPB1/POLR2A and AR, and the negative elongation factors DSIF andNELF. Regulates cytokine inducible transcription networks byfacilitating promoter recognition of target transcription factors(e.g. TNF-inducible RELA/p65 activation and IL-6-inducible STAT3signaling). Promotes RNA synthesis in genetic programs for cellgrowth, differentiation and viral pathogenesis. P-TEFb is alsoinvolved in cotranscriptional histone modification, mRNAprocessing and mRNA export. Modulates a complex network ofchromatin modifications including histone H2B monoubiquitination(H2Bub1), H3 lysine 4 trimethylation (H3K4me3) and H3K36me3;integrates phosphorylation during transcription with chromatinmodifications to control co-transcriptional histone mRNAprocessing. The CDK9/cyclin-K complex has also a kinase activitytowards CTD of RNAP II and can substitute for CDK9/cyclin-T P-TEFbin vitro. Replication stress response protein; the CDK9/cyclin-Kcomplex is required for genome integrity maintenance, by promotingcell cycle recovery from replication arrest and limiting single-stranded DNA amount in response to replication stress, thusreducing the breakdown of stalled replication forks and avoidingDNA damage. In addition, probable function in DNA repair ofisoform 2 via interaction with KU70/XRCC6. Promotes cardiacmyocyte enlargement. RPB1/POLR2A phosphorylation on 'Ser-2' in CTDactivates transcription. AR phosphorylation modulates ARtranscription factor promoter selectivity and cell growth. DSIFand NELF phosphorylation promotes transcription by inhibitingtheir negative effect. The phosphorylation of MYOD1 enhances itstranscriptional activity and thus promotes muscle differentiation.",
            "isseed":false,
            "uniprot":"P50750",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":817.1413594156943,
            "y":473.0030598015976
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"RB1",
            "altered":0,
            "rank":15,
            "cited":3201,
            "uniprotdesc":"Key regulator of entry into cell division that acts as atumor suppressor. Promotes G0-G1 transition when phosphorylated byCDK3/cyclin-C. Acts as a transcription repressor of E2F1 targetgenes. The underphosphorylated, active form of RB1 interacts withE2F1 and represses its transcription activity, leading to cellcycle arrest. Directly involved in heterochromatin formation bymaintaining overall chromatin structure and, in particular, thatof constitutive heterochromatin by stabilizing histonemethylation. Recruits and targets histone methyltransferasesSUV39H1, SUV420H1 and SUV420H2, leading to epigenetictranscriptional repression. Controls histone H4 'Lys-20'trimethylation. Inhibits the intrinsic kinase activity of TAF1.Mediates transcriptional repression by SMARCA4/BRG1 by recruitinga histone deacetylase (HDAC) complex to the c-FOS promoter. Inresting neurons, transcription of the c-FOS promoter is inhibitedby BRG1-dependent recruitment of a phospho-RB1-HDAC1 repressorcomplex. Upon calcium influx, RB1 is dephosphorylated bycalcineurin, which leads to release of the repressor complex (Bysimilarity). In case of viral infections, interactions with SV40large T antigen, HPV E7 protein or adenovirus E1A protein inducethe disassembly of RB1-E2F1 complex thereby disrupting RB1'sactivity. ",
            "isseed":false,
            "uniprot":"P06400",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":814.574109161329,
            "y":502.89301172747827
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CDK7",
            "altered":0,
            "rank":72,
            "cited":372,
            "uniprotdesc":"Serine/threonine kinase involved in cell cycle controland in RNA polymerase II-mediated RNA transcription. Cyclin-dependent kinases (CDKs) are activated by the binding to a cyclinand mediate the progression through the cell cycle. Each differentcomplex controls a specific transition between 2 subsequent phasesin the cell cycle. Required for both activation and complexformation of CDK1/cyclin-B during G2-M transition, and foractivation of CDK2/cyclins during G1-S transition (but not complexformation). CDK7 is the catalytic subunit of the CDK-activatingkinase (CAK) complex. Phosphorylates SPT5/SUPT5H, SF1/NR5A1,POLR2A, p53/TP53, CDK1, CDK2, CDK4, CDK6 and CDK11B/CDK11. CAKactivates the cyclin-associated kinases CDK1, CDK2, CDK4 and CDK6by threonine phosphorylation, thus regulating cell cycleprogression. CAK complexed to the core-TFIIH basal transcriptionfactor activates RNA polymerase II by serine phosphorylation ofthe repetitive C-terminal domain (CTD) of its large subunit(POLR2A), allowing its escape from the promoter and elongation ofthe transcripts. Phosphorylation of POLR2A in complex with DNApromotes transcription initiation by triggering dissociation fromDNA. Its expression and activity are constant throughout the cellcycle. Upon DNA damage, triggers p53/TP53 activation byphosphorylation, but is inactivated in turn by p53/TP53; thisfeedback loop may lead to an arrest of the cell cycle and of thetranscription, helping in cell recovery, or to apoptosis. Requiredfor DNA-bound peptides-mediated transcription and cellular growthinhibition. ",
            "isseed":false,
            "uniprot":"P50613",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":808.6163951619568,
            "y":532.2954885602501
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HMGA2",
            "altered":0,
            "rank":82,
            "cited":280,
            "uniprotdesc":"Functions as a transcriptional regulator. Functions incell cycle regulation through CCNA2. Plays an important role inchromosome condensation during the meiotic G2/M transition ofspermatocytes. ",
            "isseed":false,
            "uniprot":"P52926",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":390.3801522944653,
            "y":399.55122105213593
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"DAPK3",
            "altered":0,
            "rank":112,
            "cited":146,
            "uniprotdesc":"Serine/threonine kinase which is involved in theregulation of apoptosis, autophagy, transcription, translation andactin cytoskeleton reorganization. Involved in the regulation ofsmooth muscle contraction. Regulates both type I (caspase-dependent) apoptotic and type II (caspase-independent) autophagiccell deaths signal, depending on the cellular setting. Involved inregulation of starvation-induced autophagy. Regulates myosinphosphorylation in both smooth muscle and non-muscle cells. Insmooth muscle, regulates myosin either directly by phosphorylatingMYL12B and MYL9 or through inhibition of smooth muscle myosinphosphatase (SMPP1M) via phosphorylation of PPP1R12A; theinhibition of SMPP1M functions to enhance muscle responsiveness toCa(2+) and promote a contractile state. Phosphorylates MYL12B innon-muscle cells leading to reorganization of actin cytoskeleton.Isoform 2 can phosphorylate myosin, PPP1R12A and MYL12B.Overexpression leads to condensation of actin stress fibers intothick bundles. Involved in actin filament focal adhesion dynamics.The function in both reorganization of actin cytoskeleton andfocal adhesion dissolution is modulated by RhoD. Positivelyregulates canonical Wnt/beta-catenin signaling through interactionwith NLK and TCF7L2. Phosphorylates RPL13A on 'Ser-77' uponinterferon-gamma activation which is causing RPL13A release fromthe ribosome, RPL13A association with the GAIT complex and itssubsequent involvement in transcript-selective translationinhibition. Enhances transcription from AR-responsive promoters ina hormone- and kinase-dependent manner. Involved in regulation ofcell cycle progression and cell proliferation. May be a tumorsuppressor. ",
            "isseed":false,
            "uniprot":"O43293",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":786.8834349406518,
            "y":588.1161439097402
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HMGA1",
            "altered":0,
            "rank":111,
            "cited":147,
            "uniprotdesc":"HMG-I/Y bind preferentially to the minor groove of A+Trich regions in double-stranded DNA. It is suggested that theseproteins could function in nucleosome phasing and in the 3'-endprocessing of mRNA transcripts. They are also involved in thetranscription regulation of genes containing, or in closeproximity to A+T-rich regions.",
            "isseed":false,
            "uniprot":"P17096",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":403.8862646389955,
            "y":372.7634453076488
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKCD",
            "altered":0,
            "rank":122,
            "cited":90,
            "uniprotdesc":"Calcium-independent, phospholipid- and diacylglycerol(DAG)-dependent serine/threonine-protein kinase that playscontrasting roles in cell death and cell survival by functioningas a pro-apoptotic protein during DNA damage-induced apoptosis,but acting as an anti-apoptotic protein during cytokine receptor-initiated cell death, is involved in tumor suppression as well assurvival of several cancers, is required for oxygen radicalproduction by NADPH oxidase and acts as positive or negativeregulator in platelet functional responses. Negatively regulates Bcell proliferation and also has an important function in self-antigen induced B cell tolerance induction. Upon DNA damage,activates the promoter of the death-promoting transcription factorBCLAF1/Btf to trigger BCLAF1-mediated p53/TP53 gene transcriptionand apoptosis. In response to oxidative stress, interact with andactivate CHUK/IKKA in the nucleus, causing the phosphorylation ofp53/TP53. In the case of ER stress or DNA damage-inducedapoptosis, can form a complex with the tyrosine-protein kinaseABL1 which trigger apoptosis independently of p53/TP53. In cytosolcan trigger apoptosis by activating MAPK11 or MAPK14, inhibitingAKT1 and decreasing the level of X-linked inhibitor of apoptosisprotein (XIAP), whereas in nucleus induces apoptosis via theactivation of MAPK8 or MAPK9. Upon ionizing radiation treatment,is required for the activation of the apoptosis regulators BAX andBAK, which trigger the mitochondrial cell death pathway. Canphosphorylate MCL1 and target it for degradation which issufficient to trigger for BAX activation and apoptosis. Isrequired for the control of cell cycle progression both at G1/Sand G2/M phases. Mediates phorbol 12-myristate 13-acetate (PMA)-induced inhibition of cell cycle progression at G1/S phase by up-regulating the CDK inhibitor CDKN1A/p21 and inhibiting the cyclinCCNA2 promoter activity. In response to UV irradiation canphosphorylate CDK1, which is important for the G2/M DNA damagecheckpoint activation. Can protect glioma cells from the apoptosisinduced by TNFSF10/TRAIL, probably by inducing increasedphosphorylation and subsequent activation of AKT1. Is highlyexpressed in a number of cancer cells and promotes cell survivaland resistance against chemotherapeutic drugs by inducing cyclinD1 (CCND1) and hyperphosphorylation of RB1, and via several pro-survival pathways, including NF-kappa-B, AKT1 and MAPK1/3(ERK1/2). Can also act as tumor suppressor upon mitogenicstimulation with PMA or TPA. In N-formyl-methionyl-leucyl-phenylalanine (fMLP)-treated cells, is required for NCF1 (p47-phox) phosphorylation and activation of NADPH oxidase activity,and regulates TNF-elicited superoxide anion production inneutrophils, by direct phosphorylation and activation of NCF1 orindirectly through MAPK1/3 (ERK1/2) signaling pathways. May alsoplay a role in the regulation of NADPH oxidase activity ineosinophil after stimulation with IL5, leukotriene B4 or PMA. Incollagen-induced platelet aggregation, acts a negative regulatorof filopodia formation and actin polymerization by interactingwith and negatively regulating VASP phosphorylation. Downstream ofPAR1, PAR4 and CD36/GP4 receptors, regulates differentiallyplatelet dense granule secretion; acts as a positive regulator inPAR-mediated granule secretion, whereas it negatively regulatesCD36/GP4-mediated granule release. Phosphorylates MUC1 in the C-terminal and regulates the interaction between MUC1 and beta-catenin. The catalytic subunit phosphorylates 14-3-3 proteins(YWHAB, YWHAZ and YWHAH) in a sphingosine-dependent fashion (Bysimilarity). ",
            "isseed":false,
            "uniprot":"Q05655",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":753.0720747024197,
            "y":637.5636821949563
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"UBN1",
            "altered":0,
            "rank":180,
            "cited":8,
            "uniprotdesc":"Acts as a novel regulator of senescence. Involved in theformation of senescence-associated heterochromatin foci (SAHF),which represses expression of proliferation-promoting genes. Bindsto proliferation-promoting genes. May be required for replication-independent chromatin assembly. ",
            "isseed":false,
            "uniprot":"Q9NPG3",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":421.7252599281241,
            "y":348.6435313966405
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CDK5",
            "altered":0,
            "rank":44,
            "cited":837,
            "uniprotdesc":"Proline-directed serine/threonine-protein kinaseessential for neuronal cell cycle arrest and differentiation andmay be involved in apoptotic cell death in neuronal diseases bytriggering abortive cell cycle re-entry. Interacts with D1 and D3-type G1 cyclins. Phosphorylates SRC, NOS3, VIM/vimentin,p35/CDK5R1, MEF2A, SIPA1L1, SH3GLB1, PXN, PAK1, MCAM/MUC18, SEPT5,SYN1, DNM1, AMPH, SYNJ1, CDK16, RAC1, RHOA, CDC42, TONEBP/NFAT5,MAPT/TAU, MAP1B, histone H1, p53/TP53, HDAC1, APEX1, PTK2/FAK1,huntingtin/HTT, ATM, MAP2, NEFH and NEFM. Regulates severalneuronal development and physiological processes includingneuronal survival, migration and differentiation, axonal andneurite growth, synaptogenesis, oligodendrocyte differentiation,synaptic plasticity and neurotransmission, by phosphorylating keyproteins. Activated by interaction with CDK5R1 (p35) and CDK5R2(p39), especially in post-mitotic neurons, and promotes CDK5R1(p35) expression in an autostimulation loop. Phosphorylates manydownstream substrates such as Rho and Ras family small GTPases(e.g. PAK1, RAC1, RHOA, CDC42) or microtubule-binding proteins(e.g. MAPT/TAU, MAP2, MAP1B), and modulates actin dynamics toregulate neurite growth and/or spine morphogenesis. Phosphorylatesalso exocytosis associated proteins such as MCAM/MUC18, SEPT5,SYN1, and CDK16/PCTAIRE1 as well as endocytosis associatedproteins such as DNM1, AMPH and SYNJ1 at synaptic terminals. Inthe mature central nervous system (CNS), regulatesneurotransmitter movements by phosphorylating substratesassociated with neurotransmitter release and synapse plasticity;synaptic vesicle exocytosis, vesicles fusion with the presynapticmembrane, and endocytosis. Promotes cell survival by activatinganti-apoptotic proteins BCL2 and STAT3, and negatively regulatingof JNK3/MAPK10 activity. Phosphorylation of p53/TP53 in responseto genotoxic and oxidative stresses enhances its stabilization bypreventing ubiquitin ligase-mediated proteasomal degradation, andinduces transactivation of p53/TP53 target genes, thus regulatingapoptosis. Phosphorylation of p35/CDK5R1 enhances itsstabilization by preventing calpain-mediated proteolysis producingp25/CDK5R1 and avoiding ubiquitin ligase-mediated proteasomaldegradation. During aberrant cell-cycle activity and DNA damage,p25/CDK5 activity elicits cell-cycle activity and double-strandDNA breaks that precedes neuronal death by deregulating HDAC1. DNAdamage triggered phosphorylation of huntingtin/HTT in nuclei ofneurons protects neurons against polyglutamine expansion as wellas DNA damage mediated toxicity. Phosphorylation of PXN reducesits interaction with PTK2/FAK1 in matrix-cell focal adhesions(MCFA) during oligodendrocytes (OLs) differentiation. Negativeregulator of Wnt/beta-catenin signaling pathway. Activator of theGAIT (IFN-gamma-activated inhibitor of translation) pathway, whichsuppresses expression of a post-transcriptional regulon ofproinflammatory genes in myeloid cells; phosphorylates the linkerdomain of glutamyl-prolyl tRNA synthetase (EPRS) in a IFN-gamma-dependent manner, the initial event in assembly of the GAITcomplex. Phosphorylation of SH3GLB1 is required for autophagyinduction in starved neurons. Phosphorylation of TONEBP/NFAT5 inresponse to osmotic stress mediates its rapid nuclearlocalization. MEF2 is inactivated by phosphorylation in nucleus inresponse to neurotoxin, thus leading to neuronal apoptosis. APEX1AP-endodeoxyribonuclease is repressed by phosphorylation,resulting in accumulation of DNA damage and contributing toneuronal death. NOS3 phosphorylation down regulates NOS3-derivednitrite (NO) levels. SRC phosphorylation mediates its ubiquitin-dependent degradation and thus leads to cytoskeletalreorganization. May regulate endothelial cell migration andangiogenesis via the modulation of lamellipodia formation.Involved in dendritic spine morphogenesis by mediating the EFNA1-EPHA4 signaling. The complex p35/CDK5 participates in theregulation of the circadian clock by modulating the function ofCLOCK protein: phosphorylates CLOCK at 'Thr-451' and 'Thr-461' andregulates the transcriptional activity of the CLOCK-ARNTL/BMAL1heterodimer in association with altered stability and subcellulardistribution. ",
            "isseed":false,
            "uniprot":"Q00535",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":708.9396993235885,
            "y":678.0680099519911
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CDK2",
            "altered":0,
            "rank":34,
            "cited":1469,
            "uniprotdesc":"Serine/threonine-protein kinase involved in the controlof the cell cycle; essential for meiosis, but dispensable formitosis. Phosphorylates CTNNB1, USP37, p53/TP53, NPM1, CDK7, RB1,BRCA2, MYC, NPAT, EZH2. Interacts with cyclins A, B1, B3, D, or E.Triggers duplication of centrosomes and DNA. Acts at the G1-Stransition to promote the E2F transcriptional program and theinitiation of DNA synthesis, and modulates G2 progression;controls the timing of entry into mitosis/meiosis by controllingthe subsequent activation of cyclin B/CDK1 by phosphorylation, andcoordinates the activation of cyclin B/CDK1 at the centrosome andin the nucleus. Crucial role in orchestrating a fine balancebetween cellular proliferation, cell death, and DNA repair inhuman embryonic stem cells (hESCs). Activity of CDK2 is maximalduring S phase and G2; activated by interaction with cyclin Eduring the early stages of DNA synthesis to permit G1-Stransition, and subsequently activated by cyclin A2 (cyclin A1 ingerm cells) during the late stages of DNA replication to drive thetransition from S phase to mitosis, the G2 phase. EZH2phosphorylation promotes H3K27me3 maintenance and epigenetic genesilencing. Phosphorylates CABLES1 (By similarity). Cyclin E/CDK2prevents oxidative stress-mediated Ras-induced senescence byphosphorylating MYC. Involved in G1-S phase DNA damage checkpointthat prevents cells with damaged DNA from initiating mitosis;regulates homologous recombination-dependent repair byphosphorylating BRCA2, this phosphorylation is low in S phase whenrecombination is active, but increases as cells progress towardsmitosis. In response to DNA damage, double-strand break repair byhomologous recombination a reduction of CDK2-mediated BRCA2phosphorylation. Phosphorylation of RB1 disturbs its interactionwith E2F1. NPM1 phosphorylation by cyclin E/CDK2 promotes itsdissociates from unduplicated centrosomes, thus initiatingcentrosome duplication. Cyclin E/CDK2-mediated phosphorylation ofNPAT at G1-S transition and until prophase stimulates the NPAT-mediated activation of histone gene transcription during S phase.Required for vitamin D-mediated growth inhibition by being itselfinactivated. Involved in the nitric oxide- (NO) mediated signalingin a nitrosylation/activation-dependent manner. USP37 is activatedby phosphorylation and thus triggers G1-S transition. CTNNB1phosphorylation regulates insulin internalization. ",
            "isseed":false,
            "uniprot":"P24941",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":683.702093338653,
            "y":694.2872344756593
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"DAPK1",
            "altered":0,
            "rank":49,
            "cited":691,
            "uniprotdesc":"Calcium/calmodulin-dependent serine/threonine kinaseinvolved in multiple cellular signaling pathways that trigger cellsurvival, apoptosis, and autophagy. Regulates both type Iapoptotic and type II autophagic cell deaths signal, depending onthe cellular setting. The former is caspase-dependent, while thelatter is caspase-independent and is characterized by theaccumulation of autophagic vesicles. Phosphorylates PIN1 resultingin inhibition of its catalytic activity, nuclear localization, andcellular function. Phosphorylates TPM1, enhancing stress fiberformation in endothelial cells. Phosphorylates STX1A andsignificantly decreases its binding to STXBP1. PhosphorylatesPRKD1 and regulates JNK signaling by binding and activating PRKD1under oxidative stress. Phosphorylates BECN1, reducing itsinteraction with BCL2 and BCL2L1 and promoting the induction ofautophagy. Phosphorylates TSC2, disrupting the TSC1-TSC2 complexand stimulating mTORC1 activity in a growth factor-dependentpathway. Phosphorylates RPS6, MYL9 and DAPK3. Acts as a signalingamplifier of NMDA receptors at extrasynaptic sites for mediatingbrain damage in stroke. Cerebral ischemia recruits DAPK1 into theNMDA receptor complex and it phosphorylates GRINB at Ser-1303inducing injurious Ca(2+) influx through NMDA receptor channels,resulting in an irreversible neuronal death. Required togetherwith DAPK3 for phosphorylation of RPL13A upon interferon-gammaactivation which is causing RPL13A involvement in transcript-selective translation inhibition.Isoform 2 cannot induce apoptosis but can inducemembrane blebbing.",
            "isseed":false,
            "uniprot":"P53355",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":656.7801404779123,
            "y":707.5238675129558
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"RFWD2",
            "altered":0,
            "rank":143,
            "cited":60,
            "uniprotdesc":"E3 ubiquitin-protein ligase that mediates ubiquitinationand subsequent proteasomal degradation of target proteins. E3ubiquitin ligases accept ubiquitin from an E2 ubiquitin-conjugating enzyme in the form of a thioester and then directlytransfers the ubiquitin to targeted substrates. Involved in JUNubiquitination and degradation. Directly involved in p53 (TP53)ubiquitination and degradation, thereby abolishing p53-dependenttranscription and apoptosis. Ubiquitinates p53 independently ofMDM2 or RCHY1. Probably mediates E3 ubiquitin ligase activity byfunctioning as the essential RING domain subunit of larger E3complexes. In contrast, it does not constitute the catalytic RINGsubunit in the DCX DET1-COP1 complex that negatively regulatesJUN, the ubiquitin ligase activity being mediated by RBX1.Involved in 14-3-3 protein sigma/SFN ubiquitination andproteasomal degradation, leading to AKT activation and promotionof cell survival. Ubiquitinates MTA1 leading to its proteasomaldegradation. ",
            "isseed":false,
            "uniprot":"Q8NHY2",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":628.5248095687266,
            "y":717.6053493094187
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PPM1J",
            "altered":0,
            "rank":191,
            "cited":3,
            "uniprotdesc":"",
            "isseed":false,
            "uniprot":"Q5JR12",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":599.3044520604564,
            "y":724.4002523321917
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"BTG2",
            "altered":0,
            "rank":97,
            "cited":205,
            "uniprotdesc":"Anti-proliferative protein; the function is mediated byassociation with deadenylase subunits of the CCR4-NOT complex.Activates mRNA deadenylation in a CNOT6 and CNOT7-dependentmanner. In vitro can inhibit deadenylase activity of CNOT7 andCNOT8. Involved in cell cycle regulation. Could be involved in thegrowth arrest and differentiation of the neuronal precursors (Bysimilarity). Modulates transcription regulation mediated by ESR1.Involved in mitochondrial depolarization and neurite outgrowth.",
            "isseed":false,
            "uniprot":"P78543",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":569.5000000000001,
            "y":727.8199946289078
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CD82",
            "altered":0,
            "rank":69,
            "cited":417,
            "uniprotdesc":"Associates with CD4 or CD8 and delivers costimulatorysignals for the TCR/CD3 pathway.",
            "isseed":false,
            "uniprot":"P27701",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":539.5,
            "y":727.8199946289078
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HIPK2",
            "altered":0,
            "rank":70,
            "cited":405,
            "uniprotdesc":"Serine/threonine-protein kinase involved intranscription regulation, p53/TP53-mediated cellular apoptosis andregulation of the cell cycle. Acts as a corepressor of severaltranscription factors, including SMAD1 and POU4F1/Brn3a andprobably NK homeodomain transcription factors. PhosphorylatesPDX1, ATF1, PML, p53/TP53, CREB1, CTBP1, CBX4, RUNX1, EP300,CTNNB1, HMGA1 and ZBTB4. Inhibits cell growth and promotesapoptosis through the activation of p53/TP53 both at thetranscription level and at the protein level (by phosphorylationand indirect acetylation). The phosphorylation of p53/TP53 may bemediated by a p53/TP53-HIPK2-AXIN1 complex. Involved in theresponse to hypoxia by acting as a transcriptional co-suppressorof HIF1A. Mediates transcriptional activation of TP73. In responseto TGFB, cooperates with DAXX to activate JNK. Negative regulatorthrough phosphorylation and subsequent proteasomal degradation ofCTNNB1 and the antiapoptotic factor CTBP1. In the Wnt/beta-cateninsignaling pathway acts as an intermediate kinase betweenMAP3K7/TAK1 and NLK to promote the proteasomal degradation of MYB.Phosphorylates CBX4 upon DNA damage and promotes its E3 SUMO-protein ligase activity. Activates CREB1 and ATF1 transcriptionfactors by phosphorylation in response to genotoxic stress. Inresponse to DNA damage, stabilizes PML by phosphorylation. PML,HIPK2 and FBXO3 may act synergically to activate p53/TP53-dependent transactivation. Promotes angiogenesis, and is involvedin erythroid differentiation, especially during fetal livererythropoiesis. Phosphorylation of RUNX1 and EP300 stimulatesEP300 transcription regulation activity. Triggers ZBTB4 proteindegradation in response to DNA damage. Modulates HMGA1 DNA-bindingaffinity. In response to high glucose, triggers phosphorylation-mediated subnuclear localization shifting of PDX1. Involved in theregulation of eye size, lens formation and retinal laminationduring late embryogenesis. ",
            "isseed":false,
            "uniprot":"Q9H2X6",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":509.69554793954376,
            "y":724.4002523321917
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"DDB2",
            "altered":0,
            "rank":102,
            "cited":168,
            "uniprotdesc":"Required for DNA repair. Binds to DDB1 to form the UV-damaged DNA-binding protein complex (the UV-DDB complex). The UV-DDB complex may recognize UV-induced DNA damage and recruitproteins of the nucleotide excision repair pathway (the NERpathway) to initiate DNA repair. The UV-DDB complex preferentiallybinds to cyclobutane pyrimidine dimers (CPD), 6-4 photoproducts(6-4 PP), apurinic sites and short mismatches. Also appears tofunction as the substrate recognition module for the DCX (DDB1-CUL4-X-box) E3 ubiquitin-protein ligase complex DDB1-CUL4-ROC1(also known as CUL4-DDB-ROC1 and CUL4-DDB-RBX1). The DDB1-CUL4-ROC1 complex may ubiquitinate histone H2A, histone H3 and histoneH4 at sites of UV-induced DNA damage. The ubiquitination ofhistones may facilitate their removal from the nucleosome andpromote subsequent DNA repair. The DDB1-CUL4-ROC1 complex alsoubiquitinates XPC, which may enhance DNA-binding by XPC andpromote NER. Isoform D1 and isoform D2 inhibit UV-damaged DNArepair. ",
            "isseed":false,
            "uniprot":"Q92466",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":480.47519043127363,
            "y":717.6053493094188
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MDM2",
            "altered":0,
            "rank":7,
            "cited":5024,
            "uniprotdesc":"E3 ubiquitin-protein ligase that mediates ubiquitinationof p53/TP53, leading to its degradation by the proteasome.Inhibits p53/TP53- and p73/TP73-mediated cell cycle arrest andapoptosis by binding its transcriptional activation domain. Alsoacts as a ubiquitin ligase E3 toward itself and ARRB1. Permits thenuclear export of p53/TP53. Promotes proteasome-dependentubiquitin-independent degradation of retinoblastoma RB1 protein.Inhibits DAXX-mediated apoptosis by inducing its ubiquitinationand degradation. Component of the TRIM28/KAP1-MDM2-p53/TP53complex involved in stabilizing p53/TP53. Also component of theTRIM28/KAP1-ERBB4-MDM2 complex which links growth factor and DNAdamage response pathways. Mediates ubiquitination and subsequentproteasome degradation of DYRK2 in nucleus. Ubiquitinates IGF1Rand SNAI1 and promotes them to proteasomal degradation.",
            "isseed":false,
            "uniprot":"Q00987",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":599.0684773104314,
            "y":636.667218490667
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CTSD",
            "altered":0,
            "rank":85,
            "cited":273,
            "uniprotdesc":"Acid protease active in intracellular protein breakdown.Involved in the pathogenesis of several diseases such as breastcancer and possibly Alzheimer disease.",
            "isseed":false,
            "uniprot":"P07339",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":425.2979066613468,
            "y":694.287234475659
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"NGFR",
            "altered":0,
            "rank":80,
            "cited":313,
            "uniprotdesc":"Plays a role in the regulation of the translocation ofGLUT4 to the cell surface in adipocytes and skeletal muscle cellsin response to insulin, probably by regulating RAB31 activity, andthereby contributes to the regulation of insulin-dependent glucoseuptake (By similarity). Low affinity receptor which can bind toNGF, BDNF, NT-3, and NT-4. Can mediate cell survival as well ascell death of neural cells. Necessary for the circadianoscillation of the clock genes ARNTL/BMAL1, PER1, PER2 and NR1D1in the suprachiasmatic nucleus (SCN) of the brain and in liver andof the genes involved in glucose and lipid metabolism in theliver. ",
            "isseed":false,
            "uniprot":"P08138",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":400.06030067641166,
            "y":678.0680099519913
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CARM1",
            "altered":0,
            "rank":207,
            "cited":152,
            "uniprotdesc":"Methylates (mono- and asymmetric dimethylation) theguanidino nitrogens of arginyl residues in several proteinsinvolved in DNA packaging, transcription regulation, pre-mRNAsplicing, and mRNA stability. Recruited to promoters upon geneactivation together with histone acetyltransferases fromEP300/P300 and p160 families, methylates histone H3 at 'Arg-17'(H3R17me), forming mainly asymmetric dimethylarginine (H3R17me2a),leading to activate transcription via chromatin remodeling. Duringnuclear hormone receptor activation and TCF7L2/TCF4 activation,acts synergically with EP300/P300 and either one of the p160histone acetyltransferases NCOA1/SRC1, NCOA2/GRIP1 and NCOA3/ACTRor CTNNB1/beta-catenin to activate transcription. During myogenictranscriptional activation, acts together with NCOA3/ACTR as acoactivator for MEF2C. During monocyte inflammatory stimulation,acts together with EP300/P300 as a coactivator for NF-kappa-B.Acts as coactivator for PPARG, promotes adipocyte differentiationand the accumulation of brown fat tissue. Plays a role in theregulation of pre-mRNA alternative splicing by methylation ofsplicing factors. Also seems to be involved in p53/TP53transcriptional activation. Methylates EP300/P300, both at 'Arg-2142', which may loosen its interaction with NCOA2/GRIP1, and at'Arg-580' and 'Arg-604' in the KIX domain, which impairs itsinteraction with CREB and inhibits CREB-dependent transcriptionalactivation. Also methylates arginine residues in RNA-bindingproteins PABPC1, ELAVL1 and ELAV4, which may affect their mRNA-stabilizing properties and the half-life of their target mRNAs.",
            "isseed":false,
            "uniprot":"Q86X55",
            "isvalid":true,
            "importance":2
         },
         "position":{
            "x":376.83605235723905,
            "y":659.0776363377252
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TYRP1",
            "altered":0,
            "rank":87,
            "cited":265,
            "uniprotdesc":"Oxidation of 5,6-dihydroxyindole-2-carboxylic acid(DHICA) into indole-5,6-quinone-2-carboxylic acid. May regulate orinfluence the type of melanin synthesized.",
            "isseed":false,
            "uniprot":"P17643",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":355.92792529758043,
            "y":637.5636821949564
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKDC",
            "altered":0,
            "rank":74,
            "cited":363,
            "uniprotdesc":"Serine/threonine-protein kinase that acts as a molecularsensor for DNA damage. Involved in DNA non-homologous end joining(NHEJ) required for double-strand break (DSB) repair and V(D)Jrecombination. Must be bound to DNA to express its catalyticproperties. Promotes processing of hairpin DNA structures in V(D)Jrecombination by activation of the hairpin endonuclease artemis(DCLRE1C). The assembly of the DNA-PK complex at DNA ends is alsorequired for the NHEJ ligation step. Required to protect and alignbroken ends of DNA. May also act as a scaffold protein to aid thelocalization of DNA repair proteins to the site of damage. Foundat the ends of chromosomes, suggesting a further role in themaintenance of telomeric stability and the prevention ofchromosomal end fusion. Also involved in modulation oftranscription. Recognizes the substrate consensus sequence [ST]-Q.Phosphorylates 'Ser-139' of histone variant H2AX/H2AFX, therebyregulating DNA damage response mechanism. Phosphorylates DCLRE1C,c-Abl/ABL1, histone H1, HSPCA, c-jun/JUN, p53/TP53, PARP1, POU2F1,DHX9, SRF, XRCC1, XRCC1, XRCC4, XRCC5, XRCC6, WRN, MYC and RFA2.Can phosphorylate C1D not only in the presence of linear DNA butalso in the presence of supercoiled DNA. Ability to phosphorylatep53/TP53 in the presence of supercoiled DNA is dependent on C1D.Contributes to the determination of the circadian period length byantagonizing phosphorylation of CRY1 'Ser-588' and increasing CRY1protein stability, most likely through an indirect machanism.Interacts with CRY1 and CRY2; negatively regulates CRY1phosphorylation. ",
            "isseed":false,
            "uniprot":"P78527",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":337.6084889085172,
            "y":613.806614817288
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HIC1",
            "altered":0,
            "rank":99,
            "cited":185,
            "uniprotdesc":"Transcriptional repressor. Recognizes and binds to theconsensus sequence '5-[CG]NG[CG]GGGCA[CA]CC-3'. May act as a tumorsuppressor. May be involved in development of head, face, limbsand ventral body wall. Involved in down-regulation of SIRT1 andthereby is involved in regulation of p53/TP53-dependent apoptoticDNA-damage responses. The specific target gene promoterassociation seems to be depend on corepressors, such as CTBP1 orCTBP2 and MTA1. The regulation of SIRT1 transcription in responseto nutrient deprivation seems to involve CTBP1. In cooperationwith MTA1 (indicative for an association with the NuRD complex)represses transcription from CCND1/cyclin-D1 and CDKN1C/p57Kip2specifically in quiescent cells. Involved in regulation of the Wntsignaling pathway probably by association with TCF7L2 andpreventing TCF7L2 and CTNNB1 association with promoters of TCF-responsive genes. Seems to repress transcription from E2F1 andATOH1 which involves ARID1A, indicative for the participation of adistinct SWI/SNF-type chromatin-remodeling complex. Probablyrepresses transcription from ACKR3, FGFBP1 and EFNA1.",
            "isseed":false,
            "uniprot":"Q14526",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":322.1165650593482,
            "y":588.1161439097402
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TAP1",
            "altered":0,
            "rank":63,
            "cited":474,
            "uniprotdesc":"Involved in the transport of antigens from the cytoplasmto the endoplasmic reticulum for association with MHC class Imolecules. Also acts as a molecular scaffold for the final stageof MHC class I folding, namely the binding of peptide. Nascent MHCclass I molecules associate with TAP via tapasin. Inhibited by thecovalent attachment of herpes simplex virus ICP47 protein, whichblocks the peptide-binding site of TAP. Inhibited by humancytomegalovirus US6 glycoprotein, which binds to the lumenal sideof the TAP complex and inhibits peptide translocation byspecifically blocking ATP-binding to TAP1 and prevents theconformational rearrangement of TAP induced by peptide binding.Inhibited by human adenovirus E3-19K glycoprotein, which binds theTAP complex and acts as a tapasin inhibitor, preventing MHC classI/TAP association. Expression of TAP1 is down-regulated by humanEpstein-Barr virus vIL-10 protein, thereby affecting the transportof peptides into the endoplasmic reticulum and subsequent peptideloading by MHC class I molecules.",
            "isseed":false,
            "uniprot":"Q03518",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":309.6541146692916,
            "y":560.8271840491045
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PYCARD",
            "altered":0,
            "rank":89,
            "cited":244,
            "uniprotdesc":"Functions as key mediator in apoptosis and inflammation.Promotes caspase-mediated apoptosis involving predominantlycaspase-8 and also caspase-9 in a probable cell type-specificmanner. Involved in activation of the mitochondrial apoptoticpathway, promotes caspase-8-dependent proteolytic maturation ofBID independently of FADD in certain cell types and also mediatesmitochondrial translocation of BAX and activates BAX-dependentapoptosis coupled to activation of caspase-9, -2 and -3. Involvedin macrophage pyroptosis, a caspase-1-dependent inflammatory formof cell death and is the major constituent of the ASC pyroptosomewhich forms upon potassium depletion and rapidly recruits andactivates caspase-1. In innate immune response believed to act asan integral adapter in the assembly of the inflammasome whichactivates caspase-1 leading to processing and secretion ofproinflammatory cytokines. The function as activating adapter indifferent types of inflammasomes is mediated by the DAPIN and CARDdomains and their homotypic interactions. Required for recruitmentof caspase-1 to inflammasomes containing certain patternrecognition receptors, such as NLRP2, NLRP3, AIM2 and probablyIFI16. In the NLRP1 and NLRC4 inflammasomes seems not be requiredbut facilitates the processing of procaspase-1. In cooperationwith NOD2 involved in an inflammasome activated by bacterialmuramyl dipeptide leading to caspase-1 activation. May be involvedin DDX58-triggered proinflammatory responses and inflammasomeactivation. Isoform 2 may have a regulating effect on the functionas inflammasome adapter. Isoform 3 seems to inhibit inflammasome-mediated maturation of interleukin-1 beta. In collaboration withAIM2 which detects cytosolic double-stranded DNA may also beinvolved in a caspase-1-independent cell death that involvescaspase-8. In adaptive immunity may be involved in maturation ofdendritic cells to stimulate T-cell immunity and in cytoskeletalrearrangements coupled to chemotaxis and antigen uptake may beinvolved in post-transcriptional regulation of the guaninenucleotide exchange factor DOCK2; the latter function is proposedto involve the nuclear form. Also involved in transcriptionalactivation of cytokines and chemokines independent of theinflammasome; this function may involve AP-1, NF-kappa-B, MAPK andcaspase-8 signaling pathways. For regulation of NF-kappa-Bactivating and inhibiting functions have been reported. ModulatesNF-kappa-B induction at the level of the IKK complex by inhibitingkinase activity of CHUK and IKBK. Proposed to compete with RIPK2for association with CASP1 thereby down-regulating CASP1-mediatedRIPK2-dependent NF-kappa-B activation and activating interleukin-1beta processing. ",
            "isseed":false,
            "uniprot":"Q9ULZ3",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":300.38360483804325,
            "y":532.2954885602502
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"APC",
            "altered":0,
            "rank":22,
            "cited":2683,
            "uniprotdesc":"Tumor suppressor. Promotes rapid degradation of CTNNB1and participates in Wnt signaling as a negative regulator. APCactivity is correlated with its phosphorylation state. Activatesthe GEF activity of SPATA13 and ARHGEF4. Plays a role inhepatocyte growth factor (HGF)-induced cell migration. Requiredfor MMP9 up-regulation via the JNK signaling pathway in colorectaltumor cells. Acts as a mediator of ERBB2-dependent stabilizationof microtubules at the cell cortex. It is required for thelocalization of MACF1 to the cell membrane and this localizationof MACF1 is critical for its function in microtubulestabilization. ",
            "isseed":false,
            "uniprot":"P25054",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":294.42589083867097,
            "y":502.89301172747867
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"RNF144B",
            "altered":0,
            "rank":190,
            "cited":4,
            "uniprotdesc":"E3 ubiquitin-protein ligase which accepts ubiquitin fromE2 ubiquitin-conjugating enzymes UBE2L3 and UBE2L6 in the form ofa thioester and then directly transfers the ubiquitin to targetedsubstrates such as LCMT2, thereby promoting their degradation.Induces apoptosis via a p53/TP53-dependent but caspase-independentmechanism. However, its overexpression also produces a decrease ofthe ubiquitin-dependent stability of BAX, a pro-apoptotic protein,ultimately leading to protection of cell death; But, it is not ananti-apoptotic protein per se. ",
            "isseed":false,
            "uniprot":"Q7Z419",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":291.8586405843057,
            "y":473.0030598015978
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"KAT2B",
            "altered":0,
            "rank":55,
            "cited":609,
            "uniprotdesc":null,
            "isseed":false,
            "uniprot":null,
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":292.7153221081166,
            "y":443.0152940167519
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MSH2",
            "altered":0,
            "rank":21,
            "cited":2696,
            "uniprotdesc":"Component of the post-replicative DNA mismatch repairsystem (MMR). Forms two different heterodimers: MutS alpha (MSH2-MSH6 heterodimer) and MutS beta (MSH2-MSH3 heterodimer) whichbinds to DNA mismatches thereby initiating DNA repair. When bound,heterodimers bend the DNA helix and shields approximately 20 basepairs. MutS alpha recognizes single base mismatches anddinucleotide insertion-deletion loops (IDL) in the DNA. MutS betarecognizes larger insertion-deletion loops up to 13 nucleotideslong. After mismatch binding, MutS alpha or beta forms a ternarycomplex with the MutL alpha heterodimer, which is thought to beresponsible for directing the downstream MMR events, includingstrand discrimination, excision, and resynthesis. ATP binding andhydrolysis play a pivotal role in mismatch repair functions. TheATPase activity associated with MutS alpha regulates bindingsimilar to a molecular switch: mismatched DNA provokes ADP-->ATPexchange, resulting in a discernible conformational transitionthat converts MutS alpha into a sliding clamp capable ofhydrolysis-independent diffusion along the DNA backbone. Thistransition is crucial for mismatch repair. MutS alpha may alsoplay a role in DNA homologous recombination repair. In melanocytesmay modulate both UV-B-induced cell cycle regulation andapoptosis. ",
            "isseed":false,
            "uniprot":"P43246",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":296.9847672563152,
            "y":413.32065076032376
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PPP1R13L",
            "altered":0,
            "rank":140,
            "cited":63,
            "uniprotdesc":"Regulator that plays a central role in regulation ofapoptosis and transcription via its interaction with NF-kappa-Band p53/TP53 proteins. Blocks transcription of HIV-1 virus byinhibiting the action of both NF-kappa-B and SP1. Also inhibitsp53/TP53 function, possibly by preventing the association betweenp53/TP53 and ASPP1 or ASPP2, and therefore suppressing thesubsequent activation of apoptosis. ",
            "isseed":false,
            "uniprot":"Q8WUF5",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":304.6113172821198,
            "y":384.3062451192653
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SIRT1",
            "altered":0,
            "rank":46,
            "cited":799,
            "uniprotdesc":"NAD-dependent protein deacetylase that linkstranscriptional regulation directly to intracellular energeticsand participates in the coordination of several separated cellularfunctions such as cell cycle, response to DNA damage, metobolism,apoptosis and autophagy. Can modulate chromatin function throughdeacetylation of histones and can promote alterations in themethylation of histones and DNA, leading to transcriptionalrepression. Deacetylates a broad range of transcription factorsand coregulators, thereby regulating target gene expressionpositively and negatively. Serves as a sensor of the cytosolicratio of NAD(+)/NADH which is altered by glucose deprivation andmetabolic changes associated with caloric restriction. Isessential in skeletal muscle cell differentiation and in responseto low nutrients mediates the inhibitory effect on skeletalmyoblast differentiation which also involves 5'-AMP-activatedprotein kinase (AMPK) and nicotinamide phosphoribosyltransferase(NAMPT). Component of the eNoSC (energy-dependent nucleolarsilencing) complex, a complex that mediates silencing of rDNA inresponse to intracellular energy status and acts by recruitinghistone-modifying enzymes. The eNoSC complex is able to sense theenergy status of cell: upon glucose starvation, elevation ofNAD(+)/NADP(+) ratio activates SIRT1, leading to histone H3deacetylation followed by dimethylation of H3 at 'Lys-9' (H3K9me2)by SUV39H1 and the formation of silent chromatin in the rDNAlocus. Deacetylates 'Lys-266' of SUV39H1, leading to itsactivation. Inhibits skeletal muscle differentiation bydeacetylating PCAF and MYOD1. Deacetylates H2A and 'Lys-26' ofHIST1H1E. Deacetylates 'Lys-16' of histone H4 (in vitro). Involvedin NR0B2/SHP corepression function through chromatin remodeling:Recruited to LRH1 target gene promoters by NR0B2/SHP therebystimulating histone H3 and H4 deacetylation leading totranscriptional repression. Proposed to contribute to genomicintegrity via positive regulation of telomere length; however,reports on localization to pericentromeric heterochromatin areconflicting. Proposed to play a role in constitutiveheterochromatin (CH) formation and/or maintenance throughregulation of the available pool of nuclear SUV39H1. Uponoxidative/metabolic stress decreases SUV39H1 degradation byinhibiting SUV39H1 polyubiquitination by MDM2. This increase inSUV39H1 levels enhances SUV39H1 turnover in CH, which in turnseems to accelerate renewal of the heterochromatin whichcorrelates with greater genomic integrity during stress response.Deacetylates 'Lys-382' of p53/TP53 and impairs its ability toinduce transcription-dependent proapoptotic program and modulatecell senescence. Deacetylates TAF1B and thereby represses rDNAtranscription by the RNA polymerase I. Deacetylates MYC, promotesthe association of MYC with MAX and decreases MYC stabilityleading to compromised transformational capability. DeacetylatesFOXO3 in response to oxidative stress thereby increasing itsability to induce cell cycle arrest and resistance to oxidativestress but inhibiting FOXO3-mediated induction of apoptosistranscriptional activity; also leading to FOXO3 ubiquitination andprotesomal degradation. Appears to have a similar effect onMLLT7/FOXO4 in regulation of transcriptional activity andapoptosis. Deacetylates DNMT1; thereby impairs DNMT1methyltransferase-independent transcription repressor activity,modulates DNMT1 cell cycle regulatory function and DNMT1-mediatedgene silencing. Deacetylates RELA/NF-kappa-B p65 therebyinhibiting its transactivating potential and augments apoptosis inresponse to TNF-alpha. Deacetylates HIF1A, KAT5/TIP60, RB1 andHIC1. Deacetylates FOXO1 resulting in its nuclear retention andenhancement of its transcriptional activity leading to increasedgluconeogenesis in liver. Inhibits E2F1 transcriptional activityand apoptotic function, possibly by deacetylation. Involved inHES1- and HEY2-mediated transcriptional repression. In cooperationwith MYCN seems to be involved in transcriptional repression ofDUSP6/MAPK3 leading to MYCN stabilization by phosphorylation at'Ser-62'. Deacetylates MEF2D. Required for antagonist-mediatedtranscription suppression of AR-dependent genes which may belinked to local deacetylation of histone H3. Represses HNF1A-mediated transcription. Required for the repression of ESRRG byCREBZF. Modulates AP-1 transcription factor activity. DeacetylatesNR1H3 AND NR1H2 and deacetylation of NR1H3 at 'Lys-434' positivelyregulates transcription of NR1H3:RXR target genes, promotes NR1H3proteosomal degradation and results in cholesterol efflux; apromoter clearing mechanism after reach round of transcription isproposed. Involved in lipid metabolism. Implicated in regulationof adipogenesis and fat mobilization in white adipocytes byrepression of PPARG which probably involves association with NCOR1and SMRT/NCOR2. Deacetylates ACSS2 leading to its activation, andHMGCS1. Involved in liver and muscle metabolism. Throughdeacteylation and activation of PPARGC1A is required to activatefatty acid oxidation in skeletel muscle under low-glucoseconditions and is involved in glucose homeostasis. Involved inregulation of PPARA and fatty acid beta-oxidation in liver.Involved in positive regulation of insulin secretion in pancreaticbeta cells in response to glucose; the function seems to implytranscriptional repression of UCP2. Proposed to deacetylate IRS2thereby facilitating its insulin-induced tyrosine phosphorylation.Deacetylates SREBF1 isoform SREBP-1C thereby decreasing itsstability and transactivation in lipogenic gene expression.Involved in DNA damage response by repressing genes which areinvolved in DNA repair, such as XPC and TP73, deacetylatingXRCC6/Ku70, and faciliting recruitment of additional factors tosites of damaged DNA, such as SIRT1-deacetylated NBN can recruitATM to initiate DNA repair and SIRT1-deacetylated XPA interactswith RPA2. Also involved in DNA repair of DNA double-strand breaksby homologous recombination and specifically single-strandannealing independently of XRCC6/Ku70 and NBN. Transcriptionalsuppression of XPC probably involves an E2F4:RBL2 suppressorcomplex and protein kinase B (AKT) signaling. Transcriptionalsuppression of TP73 probably involves E2F4 and PCAF. DeacetylatesWRN thereby regulating its helicase and exonuclease activities andregulates WRN nuclear translocation in response to DNA damage.Deacetylates APEX1 at 'Lys-6' and 'Lys-7' and stimulates cellularAP endonuclease activity by promoting the association of APEX1 toXRCC1. Increases p53/TP53-mediated transcription-independentapoptosis by blocking nuclear translocation of cytoplasmicp53/TP53 and probably redirecting it to mitochondria. DeacetylatesXRCC6/Ku70 at 'Lys-539' and 'Lys-542' causing it to sequester BAXaway from mitochondria thereby inhibiting stress-inducedapoptosis. Is involved in autophagy, presumably by deacetylatingATG5, ATG7 and MAP1LC3B/ATG8. Deacetylates AKT1 which leads toenhanced binding of AKT1 and PDK1 to PIP3 and promotes theiractivation. Proposed to play role in regulation of STK11/LBK1-dependent AMPK signaling pathways implicated in cellularsenescence which seems to involve the regulation of theacetylation status of STK11/LBK1. Can deacetylate STK11/LBK1 andthereby increase its activity, cytoplasmic localization andassociation with STRAD; however, the relevance of such activity innormal cells is unclear. In endothelial cells is shown to inhibitSTK11/LBK1 activity and to promote its degradation. DeacetylatesSMAD7 at 'Lys-64' and 'Lys-70' thereby promoting its degradation.Deacetylates CIITA and augments its MHC class II transactivationand contributes to its stability. Deacteylates MECOM/EVI1. Isoform2 is shown to deacetylate 'Lys-382' of p53/TP53, however withlower activity than isoform 1. In combination, the two isoformsexert an additive effect. Isoform 2 regulates p53/TP53 expressionand cellular stress response and is in turn repressed by p53/TP53presenting a SIRT1 isoform-dependent auto-regulatory loop. In caseof HIV-1 infection, interacts with and deacetylates the viral Tatprotein. The viral Tat protein inhibits SIRT1 deacetylationactivity toward RELA/NF-kappa-B p65, thereby potentiates itstranscriptional activity and SIRT1 is proposed to contribute to T-cell hyperactivation during infection. Deacetylates PML at 'Lys-487' and this deacetylation promotes PML control of PER2 nuclearlocalization. During the neurogenic transition, repress selectiveNOTCH1-target genes through histone deacetylation in a BCL6-dependent manner and leading to neuronal differentiation.Regulates the circadian expression of several core clock genes,including ARNTL/BMAL1, RORC, PER2 and CRY1 and plays a criticalrole in maintaining a controlled rhythmicity in histoneacetylation, thereby contributing to circadian chromatinremodeling. Deacetylates ARNTL/BMAL1 and histones at the circadiangene promoters in order to facilitate repression by inhibitorycomponents of the circadian oscillator. Deacetylates PER2,facilitating its ubiquitination and degradation by the proteosome.Protects cardiomyocytes against palmitate-induced apoptosis(PubMed:11672523, PubMed:12006491, PubMed:14976264,PubMed:14980222, PubMed:15126506, PubMed:15152190,PubMed:15205477, PubMed:15469825, PubMed:15692560,PubMed:16079181, PubMed:16166628, PubMed:16892051,PubMed:16998810, PubMed:17283066, PubMed:17334224,PubMed:17505061, PubMed:17612497, PubMed:17620057,PubMed:17936707, PubMed:18203716, PubMed:18296641,PubMed:18662546, PubMed:18687677, PubMed:19188449,PubMed:19220062, PubMed:19364925, PubMed:19690166,PubMed:19934257, PubMed:20097625, PubMed:20100829,PubMed:20203304, PubMed:20375098, PubMed:20620956,PubMed:20670893, PubMed:20817729, PubMed:20975832,PubMed:21149730, PubMed:21245319, PubMed:21471201,PubMed:21504832, PubMed:21555002, PubMed:21698133,PubMed:21701047, PubMed:21775285, PubMed:21807113,PubMed:21841822, PubMed:21890893, PubMed:21909281,PubMed:21947282, PubMed:22274616). Deacetylates XBP1 isoform 2;deacetylation decreases protein stability of XBP1 isoform 2 andinhibits its transcriptional activity (By similarity).",
            "isseed":false,
            "uniprot":"Q96EB6",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":632.4422863405995,
            "y":420.5
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CASP10",
            "altered":0,
            "rank":100,
            "cited":181,
            "uniprotdesc":"Involved in the activation cascade of caspasesresponsible for apoptosis execution. Recruited to both Fas- andTNFR-1 receptors in a FADD dependent manner. May participate inthe granzyme B apoptotic pathways. Cleaves and activates caspase-3, -4, -6, -7, -8, and -9. Hydrolyzes the small- moleculesubstrates, Tyr-Val-Ala-Asp-|-AMC and Asp-Glu-Val-Asp-|-AMC.",
            "isseed":false,
            "uniprot":"Q92851",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":329.4955681394091,
            "y":329.81733631298744
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SP1",
            "altered":0,
            "rank":204,
            "cited":351,
            "uniprotdesc":"Transcription factor that can activate or represstranscription in response to physiological and pathologicalstimuli. Binds with high affinity to GC-rich motifs and regulatesthe expression of a large number of genes involved in a variety ofprocesses such as cell growth, apoptosis, differentiation andimmune responses. Highly regulated by post-translationalmodifications (phosphorylations, sumoylation, proteolyticcleavage, glycosylation and acetylation). Binds also the PDGFR-alpha G-box promoter. May have a role in modulating the cellularresponse to DNA damage. Implicated in chromatin remodeling. Playsa role in the recruitment of SMARCA4/BRG1 on the c-FOS promoter.Plays an essential role in the regulation of FE65 gene expression.In complex with ATF7IP, maintains telomerase activity in cancercells by inducing TERT and TERC gene expression. Isoform 3 is astronger activator of transcription than isoform 1. Positivelyregulates the transcription of the core clock componentARNTL/BMAL1. ",
            "isseed":false,
            "uniprot":"P08047",
            "isvalid":true,
            "importance":2
         },
         "position":{
            "x":346.42886470541225,
            "y":305.05317940410407
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"IRF5",
            "altered":0,
            "rank":101,
            "cited":180,
            "uniprotdesc":"Transcription factor involved in the induction ofinterferons IFNA and INFB and inflammatory cytokines upon virusinfection. Activated by TLR7 or TLR8 signaling.",
            "isseed":false,
            "uniprot":"Q13568",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":366.0746867237709,
            "y":282.3806921734762
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"BAX",
            "altered":0,
            "rank":51,
            "cited":635,
            "uniprotdesc":"Accelerates programmed cell death by binding to, andantagonizing the apoptosis repressor BCL2 or its adenovirushomolog E1B 19k protein. Under stress conditions, undergoes aconformation change that causes translocation to the mitochondrionmembrane, leading to the release of cytochrome c that thentriggers apoptosis. Promotes activation of CASP3, and therebyapoptosis. ",
            "isseed":false,
            "uniprot":"Q07812",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":495.57608693571933,
            "y":298.7290842958664
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CABIN1",
            "altered":0,
            "rank":132,
            "cited":77,
            "uniprotdesc":"May be required for replication-independent chromatinassembly. May serve as a negative regulator of T-cell receptor(TCR) signaling via inhibition of calcineurin. Inhibition ofactivated calcineurin is dependent on both PKC and calciumsignals. Acts as a negative regulator of p53/TP53 by keeping p53in an inactive state on chromatin at promoters of a subset of it'starget genes. ",
            "isseed":false,
            "uniprot":"Q9Y6J0",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":524.6080753437241,
            "y":291.16974245431265
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SETD7",
            "altered":0,
            "rank":156,
            "cited":32,
            "uniprotdesc":"Histone methyltransferase that specificallymonomethylates 'Lys-4' of histone H3. H3 'Lys-4' methylationrepresents a specific tag for epigenetic transcriptionalactivation. Plays a central role in the transcriptional activationof genes such as collagenase or insulin. Recruited by IPF1/PDX-1to the insulin promoter, leading to activate transcription. Hasalso methyltransferase activity toward non-histone proteins suchas p53/TP53, TAF10, and possibly TAF7 by recognizing and bindingthe [KR]-[STA]-K in substrate proteins. Monomethylates 'Lys-189'of TAF10, leading to increase the affinity of TAF10 for RNApolymerase II. Monomethylates 'Lys-372' of p53/TP53, stabilizingp53/TP53 and increasing p53/TP53-mediated transcriptionalactivation. ",
            "isseed":false,
            "uniprot":"Q8WTS6",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":438.5698130049044,
            "y":229.70989985711353
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SETD8",
            "altered":0,
            "rank":164,
            "cited":22,
            "uniprotdesc":"Protein-lysine N-methyltransferase that monomethylatesboth histones and non-histone proteins. Specificallymonomethylates 'Lys-20' of histone H4 (H4K20me1). H4K20me1 isenriched during mitosis and represents a specific tag forepigenetic transcriptional repression. Mainly functions ineuchromatin regions, thereby playing a central role in thesilencing of euchromatic genes. Required for cell proliferation,probably by contributing to the maintenance of proper higher-orderstructure of DNA during mitosis. Involved in chromosomecondensation and proper cytokinesis. Nucleosomes are preferred assubstrate compared to free histones. Mediates monomethylation ofp53/TP53 at 'Lys-382', leading to repress p53/TP53-target genes.Plays a negative role in TGF-beta response regulation and apositive role in cell migration. ",
            "isseed":false,
            "uniprot":"Q9NQR1",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":466.2035225983501,
            "y":218.0317966675917
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"APAF1",
            "altered":0,
            "rank":83,
            "cited":278,
            "uniprotdesc":"Oligomeric Apaf-1 mediates the cytochrome c-dependentautocatalytic activation of pro-caspase-9 (Apaf-3), leading to theactivation of caspase-3 and apoptosis. This activation requiresATP. Isoform 6 is less effective in inducing apoptosis.",
            "isseed":false,
            "uniprot":"O14727",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":494.98831180678513,
            "y":209.57981996234875
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"GDF15",
            "altered":0,
            "rank":91,
            "cited":232,
            "uniprotdesc":"",
            "isseed":false,
            "uniprot":"Q99988",
            "isvalid":true,
            "importance":3
         },
         "position":{
            "x":388.1769208600632,
            "y":262.09544516525557
         },
         "group":"nodes",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-H1F0",
            "source":"TP53",
            "cited":0,
            "target":"H1F0",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         }
      }
   ],
   "edges":[
      {
         "data":{
            "id":"TP53-controls-expression-of-IRF5",
            "source":"TP53",
            "cited":0,
            "target":"IRF5",
            "pubmed":[
               "11973653"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"KAT8-controls-state-change-of-TP53",
            "source":"KAT8",
            "cited":0,
            "target":"TP53",
            "pubmed":[
               "17189187",
               "17534149"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"FBXO11-controls-state-change-of-TP53",
            "source":"FBXO11",
            "cited":4,
            "target":"TP53",
            "pubmed":[
               "17098746"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"ATR-controls-state-change-of-TP53",
            "source":"ATR",
            "cited":24,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10581258",
               "10606744",
               "10608806",
               "10611322",
               "10656682",
               "10656795",
               "10673501",
               "10706102",
               "10710310",
               "10713094",
               "10723129",
               "10733583",
               "10734067",
               "10744722",
               "10958792",
               "11030628",
               "11042698",
               "11096068",
               "11121242",
               "11244509",
               "11258706",
               "11300786",
               "11313957",
               "11314044",
               "11397945",
               "11423984",
               "11447225",
               "11495913",
               "11526498",
               "11546806",
               "11583595",
               "11706017",
               "11707453",
               "11709713",
               "11711532",
               "11850826",
               "11861384",
               "11865061",
               "11875057",
               "11896572",
               "11896587",
               "12021785",
               "12049739",
               "12071847",
               "12080066",
               "12082529",
               "12091386",
               "12151394",
               "12324477",
               "12384533",
               "12393500",
               "12397362",
               "12427754",
               "12511424",
               "12518062",
               "12519769",
               "12519780",
               "12531896",
               "12588868",
               "12606585",
               "12637545",
               "12676607",
               "12702572",
               "12756247",
               "12771937",
               "12776195",
               "12860987",
               "12890678",
               "12902982",
               "12939400",
               "12955074",
               "12959929",
               "12969974",
               "1406679",
               "14517211",
               "14527925",
               "14612532",
               "14665630",
               "14712210",
               "14871840",
               "14871926",
               "14968111",
               "15048074",
               "15064747",
               "15077171",
               "15078887",
               "15082766",
               "15140942",
               "15155458",
               "15159397",
               "15178764",
               "15181149",
               "15226429",
               "15254178",
               "15258567",
               "15269203",
               "15310764",
               "15322239",
               "15355354",
               "15361830",
               "15381073",
               "15456784",
               "15471885",
               "15489221",
               "15489892",
               "15533933",
               "15542844",
               "15580310",
               "15619621",
               "15665826",
               "15750624",
               "15758953",
               "15775976",
               "15782130",
               "15792956",
               "15794754",
               "15843377",
               "15866171",
               "15870257",
               "15897882",
               "16223874",
               "16247456",
               "16288207",
               "16293623",
               "16377624",
               "16397295",
               "16436515",
               "16478990",
               "16489034",
               "16501611",
               "16632641",
               "16636671",
               "16648554",
               "16651424",
               "16714289",
               "16717128",
               "16731759",
               "16774943",
               "16783362",
               "16790523",
               "16818505",
               "16818520",
               "16891474",
               "16916644",
               "16949371",
               "17045821",
               "17105820",
               "17108107",
               "17234789",
               "17245430",
               "17254968",
               "17283137",
               "17297446",
               "17297454",
               "17332358",
               "17339337",
               "17353187",
               "17363488",
               "17371838",
               "17380123",
               "17456577",
               "17486112",
               "17535811",
               "17553757",
               "17591690",
               "17596534",
               "17609585",
               "17616578",
               "17668048",
               "17684018",
               "17698850",
               "17712528",
               "17855337",
               "17891183",
               "17936559",
               "17967874",
               "17975552",
               "17976513",
               "17977830",
               "18032786",
               "18045533",
               "18056442",
               "18097051",
               "18159951",
               "18216278",
               "18246126",
               "18272544",
               "18289945",
               "18332866",
               "18339864",
               "18345031",
               "18381438",
               "18406507",
               "18431490",
               "18449195",
               "18490454",
               "18560558",
               "18604166",
               "18614045",
               "18715874",
               "18718914",
               "18765533",
               "18769132",
               "18778462",
               "18785202",
               "18794113",
               "18818083",
               "18847491",
               "18985806",
               "18995830",
               "19085961",
               "19203586",
               "19251701",
               "19266268",
               "19303885",
               "19477925",
               "19819244",
               "19828454",
               "19861417",
               "19882354",
               "19907922",
               "19934315",
               "19962312",
               "20005840",
               "20024960",
               "20026654",
               "20080565",
               "20123963",
               "20353948",
               "20372057",
               "20562916",
               "20599567",
               "20610713",
               "20637859",
               "20661218",
               "20663147",
               "20696760",
               "20711232",
               "20931131",
               "20935676",
               "20962272",
               "21317932",
               "21383696",
               "21394211",
               "21423215",
               "21441950",
               "21465263",
               "21532626",
               "21642861",
               "21660965",
               "21757780",
               "21765463",
               "21765464",
               "21832239",
               "22002314",
               "22011578",
               "22030623",
               "22037398",
               "22055193",
               "22099307",
               "22112863",
               "22179839",
               "22285752",
               "22728651",
               "22797063",
               "22815859",
               "22975381",
               "23148227",
               "23150668",
               "23184057",
               "23678107",
               "23798621",
               "23871434",
               "23890999",
               "23982736",
               "24038750",
               "24657168",
               "24711418",
               "24820418",
               "24958101",
               "25202122",
               "8327466",
               "9363941",
               "9733515",
               "9744860",
               "9843217",
               "9925639"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite",
               "pid",
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"FBXO11-controls-state-change-of-NEDD8",
            "source":"FBXO11",
            "cited":1,
            "target":"NEDD8",
            "pubmed":[
               "17098746"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"ATM-controls-state-change-of-TP53",
            "source":"ATM",
            "cited":237,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10570149",
               "10581258",
               "10606744",
               "10608806",
               "10611322",
               "10656682",
               "10656795",
               "10673500",
               "10673501",
               "10706102",
               "10710310",
               "10713094",
               "10723129",
               "10733583",
               "10734067",
               "10744722",
               "10958792",
               "11030628",
               "11096068",
               "11101524",
               "11121242",
               "11175334",
               "11244509",
               "11258706",
               "11300786",
               "11313957",
               "11314044",
               "11397945",
               "11423984",
               "11447225",
               "11495913",
               "11526498",
               "11546806",
               "11551930",
               "11583595",
               "11706017",
               "11707453",
               "11709713",
               "11711532",
               "11740489",
               "11780126",
               "11850826",
               "11861384",
               "11865061",
               "11875057",
               "11896572",
               "11896587",
               "12021785",
               "12049739",
               "12071847",
               "12080066",
               "12082529",
               "12091386",
               "12093737",
               "12151394",
               "12324477",
               "12384533",
               "12393500",
               "12397361",
               "12397362",
               "12427754",
               "12511424",
               "12518062",
               "12519769",
               "12519780",
               "12531896",
               "12588868",
               "12606585",
               "12637545",
               "12676607",
               "12702572",
               "12756247",
               "12771937",
               "12776195",
               "12810724",
               "12860987",
               "12861053",
               "12890678",
               "12902982",
               "12907596",
               "12939400",
               "12955074",
               "12959929",
               "12969974",
               "1406679",
               "14517211",
               "14527925",
               "14612532",
               "14665630",
               "14712210",
               "14749479",
               "14871840",
               "14871926",
               "14968111",
               "15048074",
               "15064747",
               "15077171",
               "15078887",
               "15082766",
               "15140942",
               "15155458",
               "15159397",
               "15178764",
               "15181149",
               "15226429",
               "15254178",
               "15258567",
               "15269203",
               "15310764",
               "15322239",
               "15355354",
               "15361830",
               "15381073",
               "15456784",
               "15471885",
               "15489221",
               "15489892",
               "15526030",
               "15533933",
               "15542844",
               "15580310",
               "15619621",
               "15629715",
               "15657359",
               "15665826",
               "15671037",
               "15706352",
               "15750624",
               "15758953",
               "15775976",
               "15782130",
               "15792956",
               "15794754",
               "15843377",
               "15866171",
               "15870257",
               "15897882",
               "15963507",
               "16219768",
               "16223874",
               "16247456",
               "16288207",
               "16293623",
               "16377624",
               "16397295",
               "16436515",
               "16478990",
               "16489034",
               "16501611",
               "16601678",
               "16632641",
               "16636671",
               "16648554",
               "16651424",
               "16714289",
               "16717128",
               "16731759",
               "16738336",
               "16774943",
               "16783362",
               "16790523",
               "16793543",
               "16818505",
               "16818520",
               "16891474",
               "16916644",
               "16949371",
               "17045821",
               "17105820",
               "17107963",
               "17108107",
               "17121812",
               "17135248",
               "17210684",
               "17234789",
               "17245430",
               "17283137",
               "17297446",
               "17297454",
               "17332358",
               "17339337",
               "17349958",
               "17349959",
               "17353187",
               "17363488",
               "17371838",
               "17380123",
               "17456577",
               "17486112",
               "17535811",
               "17553757",
               "17567906",
               "17591690",
               "17596534",
               "17609585",
               "17616578",
               "17668048",
               "17684018",
               "17698850",
               "17712528",
               "17855337",
               "17891183",
               "17906639",
               "17936559",
               "17967874",
               "17975552",
               "17976513",
               "17977830",
               "18032786",
               "18045533",
               "18056442",
               "18056705",
               "18097051",
               "18159951",
               "18160537",
               "18216278",
               "18246126",
               "18272544",
               "18289945",
               "18332866",
               "18339864",
               "18345031",
               "18381438",
               "18406507",
               "18431490",
               "18449195",
               "18490454",
               "18536714",
               "18560558",
               "18593910",
               "18604166",
               "18614045",
               "18669630",
               "18715874",
               "18718914",
               "18765533",
               "18769132",
               "18778462",
               "18785202",
               "18794113",
               "18818083",
               "18847491",
               "18985806",
               "18995830",
               "19085961",
               "19203586",
               "19251701",
               "19266268",
               "19303885",
               "19465479",
               "19477925",
               "19819244",
               "19828042",
               "19828454",
               "19857493",
               "19861417",
               "19882354",
               "19907922",
               "19934315",
               "19962312",
               "19965871",
               "20005840",
               "20009884",
               "20018442",
               "20024960",
               "20026654",
               "20041275",
               "20080565",
               "20123963",
               "20171273",
               "20353948",
               "20363803",
               "20372057",
               "20514025",
               "20562916",
               "20599567",
               "20610713",
               "20637859",
               "20661218",
               "20663147",
               "20673369",
               "20696760",
               "20711232",
               "20811699",
               "20931131",
               "20935676",
               "20962272",
               "21057547",
               "21148320",
               "21317932",
               "21383696",
               "21394211",
               "21423215",
               "21441950",
               "21454683",
               "21465263",
               "21532626",
               "21642861",
               "21660965",
               "21757780",
               "21765463",
               "21765464",
               "21832239",
               "22002314",
               "22011578",
               "22030623",
               "22037398",
               "22055193",
               "22099307",
               "22112863",
               "22179839",
               "22285752",
               "22728651",
               "22797063",
               "22815859",
               "22878263",
               "22975381",
               "23148227",
               "23149944",
               "23150668",
               "23184057",
               "23678107",
               "23798621",
               "23871434",
               "23890999",
               "23982736",
               "24038750",
               "24145406",
               "24196445",
               "24379358",
               "24657168",
               "24711418",
               "24820418",
               "24899407",
               "24958101",
               "25202122",
               "8327466",
               "9363941",
               "9733514",
               "9733515",
               "9843217",
               "9925639"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome",
               "PhosphoSite",
               "pid",
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SMYD2-controls-state-change-of-TP53",
            "source":"SMYD2",
            "cited":6,
            "target":"TP53",
            "pubmed":[
               "17108971"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"NGFR-controls-state-change-of-TP53",
            "source":"NGFR",
            "cited":0,
            "target":"TP53",
            "pubmed":[
               "15668238"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAX-controls-expression-of-TP53",
            "source":"MAX",
            "cited":0,
            "target":"TP53",
            "pubmed":[
               "8494784"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-PRKAB1",
            "source":"TP53",
            "cited":5,
            "target":"PRKAB1",
            "pubmed":[
               "16140933",
               "17409411"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MDM2-controls-state-change-of-TP53",
            "source":"MDM2",
            "cited":2748,
            "target":"TP53",
            "pubmed":[
               "10734067",
               "11127820",
               "11340074",
               "11713287",
               "14654783",
               "14671306",
               "14702041",
               "15210108",
               "15242646",
               "16107876",
               "9153395",
               "9153396",
               "9450543"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SETD8-controls-state-change-of-TP53",
            "source":"SETD8",
            "cited":8,
            "target":"TP53",
            "pubmed":[
               "17707234"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SETD7-controls-state-change-of-TP53",
            "source":"SETD7",
            "cited":8,
            "target":"TP53",
            "pubmed":[
               "15525938",
               "17646389"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-TYRP1",
            "source":"TP53",
            "cited":0,
            "target":"TYRP1",
            "pubmed":[
               "10640990"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"ATR-controls-state-change-of-TP63",
            "source":"ATR",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKCA-controls-state-change-of-TP53",
            "source":"PRKCA",
            "cited":12,
            "target":"TP53",
            "pubmed":[
               "10706102",
               "10733583",
               "11007451",
               "11423984",
               "11585729",
               "12080066",
               "14640983",
               "14744935",
               "15659650",
               "16227590",
               "17898864",
               "18812399",
               "19819244",
               "19933256",
               "9254608",
               "9315650",
               "9571186",
               "9620776",
               "9739174"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"ATM-controls-state-change-of-TP63",
            "source":"ATM",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CHEK2-controls-state-change-of-TP73",
            "source":"CHEK2",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-DDB2",
            "source":"TP53",
            "cited":23,
            "target":"DDB2",
            "pubmed":[
               "11971958"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKCD-controls-state-change-of-TP53",
            "source":"PRKCD",
            "cited":0,
            "target":"TP53",
            "pubmed":[
               "10581258",
               "11030628",
               "11101524",
               "11300786",
               "11495913",
               "11740489",
               "11780126",
               "11875057",
               "12021785",
               "12093737",
               "12397361",
               "12427754",
               "12676607",
               "12860987",
               "12902982",
               "12907596",
               "14749479",
               "15322239",
               "15381073",
               "15471885",
               "15489221",
               "15526030",
               "15619621",
               "15657359",
               "15706352",
               "15750624",
               "15843377",
               "15870257",
               "15897882",
               "16219768",
               "16247456",
               "16377624",
               "16601678",
               "16717128",
               "16738336",
               "16793543",
               "17105820",
               "17107963",
               "17210684",
               "17332358",
               "17349958",
               "17349959",
               "17380123",
               "17567906",
               "17591690",
               "17891183",
               "17906639",
               "18056705",
               "18160537",
               "18216278",
               "18536714",
               "18560558",
               "18769132",
               "19465479",
               "19819244",
               "19828042",
               "19857493",
               "19861417",
               "19882354",
               "19965871",
               "20018442",
               "20024960",
               "20123963",
               "20171273",
               "20363803",
               "20514025",
               "20610713",
               "20673369",
               "20696760",
               "20811699",
               "20935676",
               "20962272",
               "21057547",
               "21383696",
               "21394211",
               "21642861",
               "21660965",
               "21765463",
               "21765464",
               "22011578",
               "22797063",
               "22878263",
               "23149944",
               "24145406",
               "24196445",
               "24379358",
               "24899407"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite",
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-TP63",
            "source":"TP53",
            "cited":257,
            "target":"TP63",
            "pubmed":[
               "14576823"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-PML",
            "source":"TP53",
            "cited":102,
            "target":"PML",
            "pubmed":[
               "14992722"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"ATM-controls-state-change-of-TP73",
            "source":"ATM",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"WRN-controls-state-change-of-TP63",
            "source":"WRN",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"ATR-controls-state-change-of-TP73",
            "source":"ATR",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-PCBP4",
            "source":"TP53",
            "cited":0,
            "target":"PCBP4",
            "pubmed":[
               "10891498"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-GADD45A",
            "source":"TP53",
            "cited":45,
            "target":"GADD45A",
            "pubmed":[
               "15186775",
               "18250150"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKDC-controls-state-change-of-TP53",
            "source":"PRKDC",
            "cited":10,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10570149",
               "10581258",
               "10606744",
               "10608806",
               "10611322",
               "10656682",
               "10656795",
               "10673500",
               "10673501",
               "10706102",
               "10710310",
               "10713094",
               "10723129",
               "10733583",
               "10734067",
               "10744722",
               "10930428",
               "10958792",
               "11030628",
               "11042698",
               "11096068",
               "11101524",
               "11121242",
               "11175334",
               "11244509",
               "11258706",
               "11300786",
               "11313957",
               "11314044",
               "11397945",
               "11423984",
               "11447225",
               "11495913",
               "11526498",
               "11546806",
               "11551930",
               "11583595",
               "11706017",
               "11707453",
               "11709713",
               "11711532",
               "11740489",
               "11780126",
               "11850826",
               "11861384",
               "11865061",
               "11875057",
               "11883897",
               "11896572",
               "11896587",
               "12021785",
               "12049739",
               "12071847",
               "12080066",
               "12082529",
               "12091386",
               "12093737",
               "12151394",
               "12324477",
               "12384533",
               "12393500",
               "12397361",
               "12397362",
               "12427754",
               "12511424",
               "12518062",
               "12519769",
               "12519780",
               "12531896",
               "12588868",
               "12606585",
               "12637545",
               "12676607",
               "12702572",
               "12756247",
               "12771937",
               "12776195",
               "12810724",
               "12860987",
               "12861053",
               "12890678",
               "12902982",
               "12907596",
               "12939400",
               "12955074",
               "12959929",
               "12969974",
               "1406679",
               "14517211",
               "14527925",
               "14612532",
               "14665630",
               "14712210",
               "14749479",
               "14871840",
               "14871926",
               "14968111",
               "15048074",
               "15064747",
               "15077171",
               "15078887",
               "15082766",
               "15140942",
               "15155458",
               "15159397",
               "15178764",
               "15181149",
               "15226429",
               "15254178",
               "15258567",
               "15269203",
               "15310764",
               "15322239",
               "15355354",
               "15361830",
               "15381073",
               "15456784",
               "15471885",
               "15489221",
               "15489892",
               "15526030",
               "15533933",
               "15542844",
               "15580310",
               "15619621",
               "15629715",
               "15657359",
               "15665826",
               "15671037",
               "15706352",
               "15750624",
               "15758953",
               "15775976",
               "15782130",
               "15792956",
               "15794754",
               "15843377",
               "15866171",
               "15870257",
               "15897882",
               "15963507",
               "16219768",
               "16223874",
               "16247456",
               "16288207",
               "16293623",
               "16377624",
               "16397295",
               "16436515",
               "16478990",
               "16489034",
               "16501611",
               "16601678",
               "16632641",
               "16636671",
               "16648554",
               "16651424",
               "16714289",
               "16717128",
               "16731759",
               "16738336",
               "16774943",
               "16783362",
               "16790523",
               "16793543",
               "16818505",
               "16818520",
               "16891474",
               "16916644",
               "16949371",
               "17045821",
               "17105820",
               "17107963",
               "17108107",
               "17121812",
               "17135248",
               "17210684",
               "17234789",
               "17245430",
               "17254968",
               "17283137",
               "17297446",
               "17297454",
               "17332358",
               "17339337",
               "17349958",
               "17349959",
               "17353187",
               "17363488",
               "17371838",
               "17380123",
               "17456577",
               "17486112",
               "17535811",
               "17553757",
               "17567906",
               "17591690",
               "17596534",
               "17609585",
               "17616578",
               "17668048",
               "17684018",
               "17698850",
               "17712528",
               "17855337",
               "17891183",
               "17906639",
               "17936559",
               "17967874",
               "17975552",
               "17976513",
               "17977830",
               "18032786",
               "18045533",
               "18056442",
               "18056705",
               "18097051",
               "18159951",
               "18160537",
               "18216278",
               "18246126",
               "18272544",
               "18289945",
               "18332866",
               "18339864",
               "18345031",
               "18381438",
               "18406507",
               "18431490",
               "18449195",
               "18490454",
               "18536714",
               "18560558",
               "18593910",
               "18604166",
               "18614045",
               "18669630",
               "18715874",
               "18718914",
               "18765533",
               "18769132",
               "18778462",
               "18785202",
               "18794113",
               "18818083",
               "18847491",
               "18985806",
               "18995830",
               "19085961",
               "19203586",
               "19251701",
               "19266268",
               "19303885",
               "19345189",
               "19465479",
               "19477925",
               "19819244",
               "19828042",
               "19828454",
               "19857493",
               "19861417",
               "19882354",
               "19907922",
               "19934315",
               "19962312",
               "19965871",
               "20005840",
               "20009884",
               "20018442",
               "20024960",
               "20026654",
               "20041275",
               "20080565",
               "20123963",
               "20171273",
               "20353948",
               "20363803",
               "20372057",
               "20514025",
               "20562916",
               "20599567",
               "20610713",
               "20637859",
               "20661218",
               "20663147",
               "20673369",
               "20696760",
               "20711232",
               "20811699",
               "20931131",
               "20935676",
               "20962272",
               "21057547",
               "21148320",
               "21317932",
               "21383696",
               "21386980",
               "21394211",
               "21423215",
               "21441950",
               "21454683",
               "21465263",
               "21532626",
               "21642861",
               "21660965",
               "21757780",
               "21765463",
               "21765464",
               "21832239",
               "22002314",
               "22011578",
               "22030623",
               "22037398",
               "22055193",
               "22099307",
               "22112863",
               "22179839",
               "22285752",
               "22496350",
               "22728651",
               "22797063",
               "22815859",
               "22878263",
               "22975381",
               "23148227",
               "23149944",
               "23150668",
               "23184057",
               "23678107",
               "23798621",
               "23871434",
               "23890999",
               "23982736",
               "24038750",
               "24145406",
               "24196445",
               "24379358",
               "24657168",
               "24711418",
               "24820418",
               "24899407",
               "24958101",
               "25202122",
               "8327466",
               "9363941",
               "9733515",
               "9744860",
               "9843217",
               "9925639"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-GDF15",
            "source":"TP53",
            "cited":18,
            "target":"GDF15",
            "pubmed":[
               "10618379"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"WRN-controls-state-change-of-TP73",
            "source":"WRN",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-SERPINE1",
            "source":"TP53",
            "cited":35,
            "target":"SERPINE1",
            "pubmed":[
               "7479001"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-PPM1J",
            "source":"TP53",
            "cited":0,
            "target":"PPM1J",
            "pubmed":[
               "16260627"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HTT-controls-state-change-of-TP63",
            "source":"HTT",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-CASP1",
            "source":"TP53",
            "cited":6,
            "target":"CASP1",
            "pubmed":[
               "11278253"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK14-controls-state-change-of-TP63",
            "source":"MAPK14",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HMGB1-controls-state-change-of-TP73",
            "source":"HMGB1",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-CASP6",
            "source":"TP53",
            "cited":13,
            "target":"CASP6",
            "pubmed":[
               "12089322"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK13-controls-state-change-of-TP63",
            "source":"MAPK13",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HTT-controls-state-change-of-TP53",
            "source":"HTT",
            "cited":9,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK12-controls-state-change-of-TP63",
            "source":"MAPK12",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK11-controls-state-change-of-TP63",
            "source":"MAPK11",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SIRT1-controls-expression-of-CDKN1A",
            "source":"SIRT1",
            "cited":0,
            "target":"CDKN1A",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PML-controls-state-change-of-TP53",
            "source":"PML",
            "cited":101,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CREBBP-controls-state-change-of-TP53",
            "source":"CREBBP",
            "cited":72,
            "target":"TP53",
            "pubmed":[
               "18485870"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-TAP1",
            "source":"TP53",
            "cited":6,
            "target":"TAP1",
            "pubmed":[
               "10618714",
               "12857899"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK13-controls-state-change-of-TP53",
            "source":"MAPK13",
            "cited":0,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK12-controls-state-change-of-TP53",
            "source":"MAPK12",
            "cited":0,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK14-controls-state-change-of-TP53",
            "source":"MAPK14",
            "cited":65,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10212189",
               "10581258",
               "10606744",
               "10608806",
               "10611322",
               "10656682",
               "10656795",
               "10673501",
               "10706102",
               "10710310",
               "10713094",
               "10723129",
               "10733583",
               "10734067",
               "10744722",
               "10747897",
               "10958792",
               "11030628",
               "11078726",
               "11096068",
               "11101524",
               "11121242",
               "11244509",
               "11258706",
               "11283254",
               "11300786",
               "11313957",
               "11314044",
               "11397945",
               "11423984",
               "11447225",
               "11495913",
               "11526498",
               "11546806",
               "11583595",
               "11706017",
               "11707453",
               "11709713",
               "11711532",
               "11740489",
               "11780126",
               "11850826",
               "11861384",
               "11865061",
               "11875057",
               "11896572",
               "11896587",
               "12021785",
               "12049739",
               "12071847",
               "12080066",
               "12082529",
               "12091386",
               "12093737",
               "12151394",
               "12324477",
               "12384533",
               "12393500",
               "12397361",
               "12397362",
               "12427754",
               "12511424",
               "12518062",
               "12519769",
               "12519780",
               "12531896",
               "12588868",
               "12606585",
               "12637545",
               "12676607",
               "12702572",
               "12756247",
               "12771937",
               "12776195",
               "12860987",
               "12890678",
               "12902982",
               "12907596",
               "12939400",
               "12955074",
               "12959929",
               "12969974",
               "1406679",
               "14517211",
               "14527925",
               "14612532",
               "14665630",
               "14712210",
               "14749479",
               "14871840",
               "14871926",
               "14968111",
               "15048074",
               "15064747",
               "15077171",
               "15078887",
               "15082766",
               "15140942",
               "15155458",
               "15159397",
               "15178764",
               "15181149",
               "15226429",
               "15254178",
               "15258567",
               "15269203",
               "15310764",
               "15322239",
               "15355354",
               "15361830",
               "15381073",
               "15456784",
               "15471885",
               "15489221",
               "15489892",
               "15526030",
               "15533933",
               "15542844",
               "15580310",
               "15619621",
               "15657359",
               "15665826",
               "15706352",
               "15750624",
               "15758953",
               "15775976",
               "15782130",
               "15792956",
               "15794754",
               "15843377",
               "15866171",
               "15870257",
               "15897882",
               "16219768",
               "16223874",
               "16247456",
               "16288207",
               "16293623",
               "16377624",
               "16397295",
               "16436515",
               "16478990",
               "16489034",
               "16501611",
               "16552184",
               "16601678",
               "16632641",
               "16636671",
               "16648554",
               "16651424",
               "16714289",
               "16717128",
               "16731759",
               "16738336",
               "16774943",
               "16783362",
               "16790523",
               "16793543",
               "16818505",
               "16818520",
               "16891474",
               "16916644",
               "16949371",
               "17045821",
               "17105820",
               "17107963",
               "17108107",
               "17210684",
               "17234789",
               "17245430",
               "17254968",
               "17283137",
               "17297446",
               "17297454",
               "17332358",
               "17339337",
               "17349958",
               "17349959",
               "17353187",
               "17363488",
               "17371838",
               "17380123",
               "17456577",
               "17486112",
               "17535811",
               "17553757",
               "17567906",
               "17591690",
               "17596534",
               "17609585",
               "17616578",
               "17668048",
               "17684018",
               "17698850",
               "17712528",
               "17855337",
               "17891183",
               "17906639",
               "17936559",
               "17942552",
               "17967874",
               "17975552",
               "17976513",
               "17977830",
               "18032786",
               "18045533",
               "18056442",
               "18056705",
               "18097051",
               "18159951",
               "18160537",
               "18216278",
               "18246126",
               "18272544",
               "18289945",
               "18332866",
               "18339864",
               "18345031",
               "18381438",
               "18406507",
               "18431490",
               "18449195",
               "18490454",
               "18536714",
               "18560558",
               "18604166",
               "18614045",
               "18715874",
               "18718914",
               "18765533",
               "18769132",
               "18778462",
               "18785202",
               "18794113",
               "18818083",
               "18847491",
               "18985806",
               "18995830",
               "19085961",
               "19203586",
               "19251701",
               "19266268",
               "19303885",
               "19465479",
               "19477925",
               "19819244",
               "19828042",
               "19828454",
               "19857493",
               "19861417",
               "19882354",
               "19907922",
               "19934315",
               "19962312",
               "19965871",
               "20005840",
               "20018442",
               "20024960",
               "20026654",
               "20080565",
               "20123963",
               "20171273",
               "20353948",
               "20363803",
               "20372057",
               "20514025",
               "20562916",
               "20599567",
               "20610713",
               "20637859",
               "20661218",
               "20663147",
               "20673369",
               "20696760",
               "20711232",
               "20811699",
               "20931131",
               "20935676",
               "20962272",
               "21057547",
               "21317932",
               "21383696",
               "21394211",
               "21423215",
               "21441950",
               "21465263",
               "21532626",
               "21642861",
               "21660965",
               "21757780",
               "21765463",
               "21765464",
               "21832239",
               "22002314",
               "22011578",
               "22030623",
               "22037398",
               "22055193",
               "22099307",
               "22112863",
               "22179839",
               "22285752",
               "22728651",
               "22797063",
               "22815859",
               "22878263",
               "22975381",
               "23148227",
               "23149944",
               "23150668",
               "23184057",
               "23678107",
               "23798621",
               "23871434",
               "23890999",
               "23982736",
               "24038750",
               "24145406",
               "24196445",
               "24379358",
               "24657168",
               "24711418",
               "24820418",
               "24899407",
               "24958101",
               "25202122",
               "8327466",
               "9363941",
               "9372954",
               "9733515",
               "9744860",
               "9843217",
               "9925639"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome",
               "PhosphoSite",
               "pid",
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK11-controls-state-change-of-TP53",
            "source":"MAPK11",
            "cited":0,
            "target":"TP53",
            "pubmed":[
               "10212189",
               "10747897",
               "17254968"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome",
               "pid",
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-BAX",
            "source":"TP53",
            "cited":95,
            "target":"BAX",
            "pubmed":[
               "14963330"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HMGB1-controls-state-change-of-TP63",
            "source":"HMGB1",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-TP73",
            "source":"TP53",
            "cited":42,
            "target":"TP73",
            "pubmed":[
               "11314010"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PML-controls-state-change-of-TP63",
            "source":"PML",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PPP2CA-controls-state-change-of-TP73",
            "source":"PPP2CA",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PPP2CB-controls-state-change-of-TP73",
            "source":"PPP2CB",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"FOS-controls-expression-of-TP53",
            "source":"FOS",
            "cited":12,
            "target":"TP53",
            "pubmed":[
               "10072388",
               "11136975"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CREBBP-controls-expression-of-GADD45A",
            "source":"CREBBP",
            "cited":0,
            "target":"GADD45A",
            "pubmed":[
               "15186775",
               "18250150"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-CDKN1A",
            "source":"TP53",
            "cited":1480,
            "target":"CDKN1A",
            "pubmed":[
               "11080152",
               "15655109",
               "17110336",
               "17409421",
               "17707234",
               "17719541",
               "17938176",
               "18250150",
               "18485870",
               "19011621",
               "7796420",
               "8242752"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "Reactome",
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PML-controls-state-change-of-TP73",
            "source":"PML",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HMGB1-controls-state-change-of-TP53",
            "source":"HMGB1",
            "cited":21,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"IGBP1-controls-state-change-of-TP73",
            "source":"IGBP1",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HIPK2-controls-state-change-of-TP53",
            "source":"HIPK2",
            "cited":98,
            "target":"TP53",
            "pubmed":[
               "10581258",
               "11030628",
               "11101524",
               "11300786",
               "11495913",
               "11740489",
               "11780126",
               "11875057",
               "12021785",
               "12093737",
               "12397361",
               "12427754",
               "12676607",
               "12860987",
               "12902982",
               "12907596",
               "14749479",
               "15322239",
               "15381073",
               "15471885",
               "15489221",
               "15526030",
               "15619621",
               "15657359",
               "15706352",
               "15750624",
               "15843377",
               "15870257",
               "15897882",
               "16219768",
               "16247456",
               "16377624",
               "16601678",
               "16717128",
               "16738336",
               "16793543",
               "17105820",
               "17107963",
               "17210684",
               "17332358",
               "17349958",
               "17349959",
               "17380123",
               "17567906",
               "17591690",
               "17891183",
               "17906639",
               "18056705",
               "18160537",
               "18216278",
               "18536714",
               "18560558",
               "18769132",
               "19465479",
               "19819244",
               "19828042",
               "19857493",
               "19861417",
               "19882354",
               "19965871",
               "20018442",
               "20024960",
               "20123963",
               "20171273",
               "20363803",
               "20514025",
               "20610713",
               "20673369",
               "20696760",
               "20811699",
               "20935676",
               "20962272",
               "21057547",
               "21383696",
               "21394211",
               "21642861",
               "21660965",
               "21765463",
               "21765464",
               "22011578",
               "22797063",
               "22878263",
               "23149944",
               "24145406",
               "24196445",
               "24379358",
               "24899407"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite",
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-RNF144B",
            "source":"TP53",
            "cited":0,
            "target":"RNF144B",
            "pubmed":[
               "12853982"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-HIC1",
            "source":"TP53",
            "cited":22,
            "target":"HIC1",
            "pubmed":[
               "16301995"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-HIRA",
            "source":"TP53",
            "cited":1,
            "target":"HIRA",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK14-controls-state-change-of-TP73",
            "source":"MAPK14",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-UBN1",
            "source":"TP53",
            "cited":0,
            "target":"UBN1",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-RB1",
            "source":"TP53",
            "cited":262,
            "target":"RB1",
            "pubmed":[
               "8119988"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK11-controls-state-change-of-TP73",
            "source":"MAPK11",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-HMGA2",
            "source":"TP53",
            "cited":5,
            "target":"HMGA2",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-HMGA1",
            "source":"TP53",
            "cited":5,
            "target":"HMGA1",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK13-controls-state-change-of-TP73",
            "source":"MAPK13",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MAPK12-controls-state-change-of-TP73",
            "source":"MAPK12",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"IGBP1-controls-state-change-of-TP63",
            "source":"IGBP1",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-TP53I3",
            "source":"TP53",
            "cited":28,
            "target":"TP53I3",
            "pubmed":[
               "11684014",
               "11919562",
               "17719542",
               "18485870"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"DYRK1A-controls-state-change-of-TP53",
            "source":"DYRK1A",
            "cited":1,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10581258",
               "10606744",
               "10608806",
               "10611322",
               "10656682",
               "10656795",
               "10673501",
               "10706102",
               "10710310",
               "10713094",
               "10723129",
               "10733583",
               "10734067",
               "10744722",
               "10958792",
               "11030628",
               "11096068",
               "11121242",
               "11244509",
               "11258706",
               "11313957",
               "11314044",
               "11397945",
               "11423984",
               "11447225",
               "11495913",
               "11526498",
               "11546806",
               "11583595",
               "11706017",
               "11707453",
               "11709713",
               "11711532",
               "11850826",
               "11861384",
               "11865061",
               "11875057",
               "11896572",
               "11896587",
               "12021785",
               "12049739",
               "12071847",
               "12080066",
               "12082529",
               "12091386",
               "12151394",
               "12324477",
               "12384533",
               "12393500",
               "12397362",
               "12427754",
               "12511424",
               "12518062",
               "12519769",
               "12519780",
               "12531896",
               "12588868",
               "12606585",
               "12637545",
               "12676607",
               "12702572",
               "12756247",
               "12771937",
               "12776195",
               "12860987",
               "12890678",
               "12902982",
               "12939400",
               "12955074",
               "12959929",
               "12969974",
               "1406679",
               "14517211",
               "14527925",
               "14612532",
               "14665630",
               "14712210",
               "14871840",
               "14871926",
               "14968111",
               "15048074",
               "15064747",
               "15077171",
               "15078887",
               "15082766",
               "15140942",
               "15155458",
               "15159397",
               "15178764",
               "15181149",
               "15226429",
               "15254178",
               "15258567",
               "15269203",
               "15310764",
               "15322239",
               "15355354",
               "15361830",
               "15381073",
               "15456784",
               "15471885",
               "15489221",
               "15489892",
               "15533933",
               "15542844",
               "15580310",
               "15619621",
               "15665826",
               "15750624",
               "15758953",
               "15775976",
               "15782130",
               "15792956",
               "15794754",
               "15843377",
               "15866171",
               "15870257",
               "15897882",
               "16223874",
               "16247456",
               "16288207",
               "16293623",
               "16377624",
               "16397295",
               "16436515",
               "16478990",
               "16489034",
               "16501611",
               "16632641",
               "16636671",
               "16648554",
               "16651424",
               "16714289",
               "16717128",
               "16731759",
               "16774943",
               "16783362",
               "16790523",
               "16818505",
               "16818520",
               "16891474",
               "16916644",
               "16949371",
               "17045821",
               "17105820",
               "17108107",
               "17234789",
               "17245430",
               "17283137",
               "17297446",
               "17297454",
               "17332358",
               "17339337",
               "17353187",
               "17363488",
               "17371838",
               "17380123",
               "17456577",
               "17486112",
               "17535811",
               "17553757",
               "17591690",
               "17596534",
               "17609585",
               "17616578",
               "17668048",
               "17684018",
               "17698850",
               "17712528",
               "17855337",
               "17891183",
               "17936559",
               "17967874",
               "17975552",
               "17976513",
               "17977830",
               "18032786",
               "18045533",
               "18056442",
               "18097051",
               "18159951",
               "18216278",
               "18246126",
               "18272544",
               "18289945",
               "18332866",
               "18339864",
               "18345031",
               "18381438",
               "18406507",
               "18431490",
               "18449195",
               "18490454",
               "18560558",
               "18604166",
               "18614045",
               "18715874",
               "18718914",
               "18765533",
               "18769132",
               "18778462",
               "18785202",
               "18794113",
               "18818083",
               "18847491",
               "18985806",
               "18995830",
               "19085961",
               "19203586",
               "19251701",
               "19266268",
               "19303885",
               "19477925",
               "19819244",
               "19828454",
               "19861417",
               "19882354",
               "19907922",
               "19934315",
               "19962312",
               "20005840",
               "20024960",
               "20026654",
               "20080565",
               "20123963",
               "20353948",
               "20372057",
               "20562916",
               "20599567",
               "20610713",
               "20637859",
               "20661218",
               "20663147",
               "20696760",
               "20711232",
               "20931131",
               "20935676",
               "20962272",
               "21317932",
               "21383696",
               "21394211",
               "21423215",
               "21441950",
               "21465263",
               "21532626",
               "21642861",
               "21660965",
               "21757780",
               "21765463",
               "21765464",
               "21832239",
               "22002314",
               "22011578",
               "22030623",
               "22037398",
               "22055193",
               "22099307",
               "22112863",
               "22179839",
               "22285752",
               "22728651",
               "22797063",
               "22815859",
               "22975381",
               "23148227",
               "23150668",
               "23184057",
               "23678107",
               "23798621",
               "23871434",
               "23890999",
               "23982736",
               "24038750",
               "24657168",
               "24711418",
               "24820418",
               "24958101",
               "25202122",
               "8327466",
               "9363941",
               "9733515",
               "9843217",
               "9925639"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SUMO1-controls-state-change-of-TP73",
            "source":"SUMO1",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAG1-controls-state-change-of-TP63",
            "source":"PRKAG1",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SIRT1-controls-state-change-of-TP73",
            "source":"SIRT1",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PPP2CB-controls-state-change-of-TP63",
            "source":"PPP2CB",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CHEK2-controls-state-change-of-TP53",
            "source":"CHEK2",
            "cited":73,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10570149",
               "10581258",
               "10606744",
               "10608806",
               "10611322",
               "10656682",
               "10656795",
               "10673500",
               "10673501",
               "10706102",
               "10710310",
               "10713094",
               "10723129",
               "10733583",
               "10734067",
               "10744722",
               "10958792",
               "11030628",
               "11042698",
               "11096068",
               "11121242",
               "11175334",
               "11244509",
               "11258706",
               "11300786",
               "11313957",
               "11314044",
               "11397945",
               "11423984",
               "11447225",
               "11495913",
               "11526498",
               "11546806",
               "11551930",
               "11583595",
               "11585729",
               "11706017",
               "11707453",
               "11709713",
               "11711532",
               "11850826",
               "11861384",
               "11865061",
               "11875057",
               "11883897",
               "11896572",
               "11896587",
               "12021785",
               "12049739",
               "12071847",
               "12080066",
               "12082529",
               "12091386",
               "12151394",
               "12324477",
               "12384533",
               "12393500",
               "12397362",
               "12427754",
               "12511424",
               "12518062",
               "12519769",
               "12519780",
               "12531896",
               "12588868",
               "12606585",
               "12637545",
               "12676607",
               "12702572",
               "12756247",
               "12771937",
               "12776195",
               "12810724",
               "12860987",
               "12861053",
               "12890678",
               "12902982",
               "12939400",
               "12955074",
               "12959929",
               "12969974",
               "1406679",
               "14517211",
               "14527925",
               "14612532",
               "14640983",
               "14665630",
               "14712210",
               "14871840",
               "14871926",
               "14968111",
               "15048074",
               "15064747",
               "15077171",
               "15078887",
               "15082766",
               "15140942",
               "15155458",
               "15159397",
               "15178764",
               "15181149",
               "15226429",
               "15254178",
               "15258567",
               "15269203",
               "15310764",
               "15322239",
               "15355354",
               "15361830",
               "15381073",
               "15456784",
               "15471885",
               "15489221",
               "15489892",
               "15533933",
               "15542844",
               "15580310",
               "15619621",
               "15629715",
               "15659650",
               "15665826",
               "15671037",
               "15750624",
               "15758953",
               "15775976",
               "15782130",
               "15792956",
               "15794754",
               "15843377",
               "15866171",
               "15870257",
               "15897882",
               "15963507",
               "16223874",
               "16247456",
               "16288207",
               "16293623",
               "16377624",
               "16397295",
               "16436515",
               "16478990",
               "16489034",
               "16501611",
               "16632641",
               "16636671",
               "16648554",
               "16651424",
               "16714289",
               "16717128",
               "16731759",
               "16774943",
               "16783362",
               "16790523",
               "16818505",
               "16818520",
               "16891474",
               "16916644",
               "16949371",
               "17045821",
               "17105820",
               "17108107",
               "17121812",
               "17135248",
               "17234789",
               "17245430",
               "17254968",
               "17283137",
               "17297446",
               "17297454",
               "17332358",
               "17339337",
               "17353187",
               "17363488",
               "17371838",
               "17380123",
               "17456577",
               "17486112",
               "17535811",
               "17553757",
               "17591690",
               "17596534",
               "17609585",
               "17616578",
               "17668048",
               "17684018",
               "17698850",
               "17712528",
               "17855337",
               "17891183",
               "17898864",
               "17936559",
               "17967874",
               "17975552",
               "17976513",
               "17977830",
               "18032786",
               "18045533",
               "18056442",
               "18097051",
               "18159951",
               "18216278",
               "18246126",
               "18272544",
               "18289945",
               "18332866",
               "18339864",
               "18345031",
               "18381438",
               "18406507",
               "18431490",
               "18449195",
               "18490454",
               "18560558",
               "18593910",
               "18604166",
               "18614045",
               "18669630",
               "18715874",
               "18718914",
               "18765533",
               "18769132",
               "18778462",
               "18785202",
               "18794113",
               "18812399",
               "18818083",
               "18847491",
               "18985806",
               "18995830",
               "19085961",
               "19203586",
               "19251701",
               "19266268",
               "19303885",
               "19477925",
               "19819244",
               "19828454",
               "19861417",
               "19882354",
               "19907922",
               "19933256",
               "19934315",
               "19962312",
               "20005840",
               "20009884",
               "20024960",
               "20026654",
               "20041275",
               "20080565",
               "20123963",
               "20353948",
               "20372057",
               "20562916",
               "20599567",
               "20610713",
               "20637859",
               "20661218",
               "20663147",
               "20673369",
               "20696760",
               "20711232",
               "20931131",
               "20935676",
               "20962272",
               "21148320",
               "21317932",
               "21383696",
               "21386980",
               "21394211",
               "21423215",
               "21441950",
               "21454683",
               "21465263",
               "21532626",
               "21642861",
               "21660965",
               "21757780",
               "21765463",
               "21765464",
               "21832239",
               "22002314",
               "22011578",
               "22030623",
               "22037398",
               "22055193",
               "22099307",
               "22112863",
               "22179839",
               "22285752",
               "22728651",
               "22797063",
               "22815859",
               "22975381",
               "23148227",
               "23150668",
               "23184057",
               "23678107",
               "23798621",
               "23871434",
               "23890999",
               "23982736",
               "24038750",
               "24657168",
               "24711418",
               "24820418",
               "24958101",
               "25202122",
               "8327466",
               "9254608",
               "9315650",
               "9363941",
               "9571186",
               "9620776",
               "9733515",
               "9739174",
               "9744860",
               "9843217",
               "9925639"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite",
               "pid",
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PPP2CA-controls-state-change-of-TP63",
            "source":"PPP2CA",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CDK7-controls-state-change-of-TP53",
            "source":"CDK7",
            "cited":7,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10348343",
               "10581258",
               "10656682",
               "10656795",
               "10706102",
               "10733583",
               "10747897",
               "11007451",
               "11101524",
               "11239457",
               "11300786",
               "11423984",
               "11495913",
               "11526498",
               "11583595",
               "11585729",
               "11707453",
               "11709713",
               "11850826",
               "11875057",
               "12021785",
               "12080066",
               "12091386",
               "12393879",
               "12397361",
               "12397362",
               "12427754",
               "12519780",
               "12676607",
               "12776195",
               "12860987",
               "12959929",
               "1406679",
               "14640983",
               "14665630",
               "14744935",
               "14968111",
               "15064747",
               "15155458",
               "15322239",
               "15471885",
               "15489221",
               "15619621",
               "15659650",
               "15792956",
               "15843377",
               "16083285",
               "16227590",
               "16319070",
               "16436515",
               "16552184",
               "16632641",
               "16651424",
               "16717128",
               "16818520",
               "17105820",
               "17108107",
               "17237827",
               "17591690",
               "17891183",
               "17898864",
               "17906639",
               "17942552",
               "17975552",
               "18490454",
               "18523266",
               "18718914",
               "18765533",
               "18769132",
               "18778462",
               "18812399",
               "19251701",
               "19369195",
               "19413330",
               "19819244",
               "19861417",
               "19882354",
               "19933256",
               "20009884",
               "20024960",
               "20123963",
               "20562916",
               "20663147",
               "20686112",
               "20962272",
               "21317932",
               "21383696",
               "21423215",
               "21455220",
               "21460857",
               "21832239",
               "22011578",
               "22112863",
               "22975381",
               "23312004",
               "23603988",
               "23871434",
               "24194938",
               "24657168",
               "8327466",
               "9254608",
               "9315650",
               "9372954",
               "9467949",
               "9571186",
               "9620776",
               "9739174",
               "9744860"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CDK9-controls-state-change-of-TP53",
            "source":"CDK9",
            "cited":19,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10348343",
               "10581258",
               "10644693",
               "10656682",
               "10656795",
               "10706102",
               "10733583",
               "10747897",
               "11078726",
               "11101524",
               "11239457",
               "11300786",
               "11423984",
               "11495913",
               "11526498",
               "11583595",
               "11687964",
               "11707453",
               "11709713",
               "11850826",
               "11875057",
               "12021785",
               "12091386",
               "12393879",
               "12397361",
               "12397362",
               "12427754",
               "12519780",
               "12676607",
               "12776195",
               "12860987",
               "12959929",
               "1406679",
               "14640983",
               "14665630",
               "14702041",
               "14744935",
               "14968111",
               "15064747",
               "15155458",
               "15302935",
               "15322239",
               "15471885",
               "15489221",
               "15619621",
               "15792956",
               "15843377",
               "16037820",
               "16083285",
               "16227590",
               "16319070",
               "16436515",
               "16552184",
               "16632641",
               "16651424",
               "16674116",
               "16717128",
               "16784539",
               "16818520",
               "17105820",
               "17108107",
               "17237827",
               "17287340",
               "17591690",
               "17891183",
               "17906639",
               "17942552",
               "17975552",
               "18490454",
               "18521083",
               "18523266",
               "18718914",
               "18765533",
               "18769132",
               "18778462",
               "18847512",
               "19251701",
               "19369195",
               "19413330",
               "19574224",
               "19664995",
               "19819244",
               "19861417",
               "19882354",
               "20009884",
               "20024960",
               "20123963",
               "20363803",
               "20562916",
               "20661218",
               "20663147",
               "20686112",
               "20808790",
               "20815410",
               "20962272",
               "21317932",
               "21383696",
               "21423215",
               "21455220",
               "21460857",
               "21659604",
               "21832239",
               "21983960",
               "22011578",
               "22112863",
               "22496350",
               "22547687",
               "22975381",
               "23201157",
               "23312004",
               "23603988",
               "23871434",
               "24194938",
               "24657168",
               "25159151",
               "7596441",
               "8327466",
               "9254608",
               "9315650",
               "9372954",
               "9467949",
               "9739174",
               "9744860"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CDK2-controls-state-change-of-TP53",
            "source":"CDK2",
            "cited":26,
            "target":"TP53",
            "pubmed":[
               "10581258",
               "10644693",
               "10656795",
               "11078726",
               "11283254",
               "11300786",
               "11687964",
               "11875057",
               "12021785",
               "12397361",
               "12397362",
               "12860987",
               "1406679",
               "14640983",
               "14702041",
               "14744935",
               "15302935",
               "15619621",
               "16037820",
               "16227590",
               "16552184",
               "16674116",
               "16784539",
               "17287340",
               "17906639",
               "17942552",
               "18521083",
               "18769132",
               "18847512",
               "19413330",
               "19574224",
               "19664995",
               "20363803",
               "20562916",
               "20661218",
               "20686112",
               "20808790",
               "20815410",
               "21659604",
               "21983960",
               "22011578",
               "22496350",
               "22547687",
               "23201157",
               "23312004",
               "25159151",
               "7596441",
               "8327466",
               "9254608"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite",
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CDK5-controls-state-change-of-TP53",
            "source":"CDK5",
            "cited":23,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10570149",
               "10581258",
               "10606744",
               "10608806",
               "10611322",
               "10656682",
               "10656795",
               "10673500",
               "10673501",
               "10706102",
               "10710310",
               "10713094",
               "10723129",
               "10733583",
               "10734067",
               "10744722",
               "10958792",
               "11030628",
               "11096068",
               "11101524",
               "11121242",
               "11175334",
               "11244509",
               "11258706",
               "11300786",
               "11313957",
               "11314044",
               "11397945",
               "11423984",
               "11447225",
               "11495913",
               "11526498",
               "11546806",
               "11551930",
               "11583595",
               "11706017",
               "11707453",
               "11709713",
               "11711532",
               "11740489",
               "11780126",
               "11850826",
               "11861384",
               "11865061",
               "11875057",
               "11896572",
               "11896587",
               "12021785",
               "12049739",
               "12071847",
               "12080066",
               "12082529",
               "12091386",
               "12093737",
               "12151394",
               "12324477",
               "12384533",
               "12393500",
               "12397361",
               "12397362",
               "12427754",
               "12511424",
               "12518062",
               "12519769",
               "12519780",
               "12531896",
               "12588868",
               "12606585",
               "12637545",
               "12676607",
               "12702572",
               "12756247",
               "12771937",
               "12776195",
               "12810724",
               "12860987",
               "12861053",
               "12890678",
               "12902982",
               "12907596",
               "12939400",
               "12955074",
               "12959929",
               "12969974",
               "1406679",
               "14517211",
               "14527925",
               "14612532",
               "14665630",
               "14712210",
               "14749479",
               "14871840",
               "14871926",
               "14968111",
               "15048074",
               "15064747",
               "15077171",
               "15078887",
               "15082766",
               "15140942",
               "15155458",
               "15159397",
               "15178764",
               "15181149",
               "15226429",
               "15254178",
               "15258567",
               "15269203",
               "15310764",
               "15322239",
               "15355354",
               "15361830",
               "15381073",
               "15456784",
               "15471885",
               "15489221",
               "15489892",
               "15526030",
               "15533933",
               "15542844",
               "15580310",
               "15619621",
               "15629715",
               "15657359",
               "15665826",
               "15671037",
               "15706352",
               "15750624",
               "15758953",
               "15775976",
               "15782130",
               "15792956",
               "15794754",
               "15843377",
               "15866171",
               "15870257",
               "15897882",
               "15963507",
               "16219768",
               "16223874",
               "16247456",
               "16288207",
               "16293623",
               "16377624",
               "16397295",
               "16436515",
               "16478990",
               "16489034",
               "16501611",
               "16552184",
               "16601678",
               "16632641",
               "16636671",
               "16648554",
               "16651424",
               "16714289",
               "16717128",
               "16731759",
               "16738336",
               "16774943",
               "16783362",
               "16790523",
               "16793543",
               "16818505",
               "16818520",
               "16891474",
               "16916644",
               "16949371",
               "17045821",
               "17105820",
               "17107963",
               "17108107",
               "17121812",
               "17135248",
               "17210684",
               "17234789",
               "17245430",
               "17283137",
               "17297446",
               "17297454",
               "17332358",
               "17339337",
               "17349958",
               "17349959",
               "17353187",
               "17363488",
               "17371838",
               "17380123",
               "17456577",
               "17486112",
               "17535811",
               "17553757",
               "17567906",
               "17591690",
               "17596534",
               "17609585",
               "17616578",
               "17668048",
               "17684018",
               "17698850",
               "17712528",
               "17855337",
               "17891183",
               "17906639",
               "17936559",
               "17942552",
               "17967874",
               "17975552",
               "17976513",
               "17977830",
               "18032786",
               "18045533",
               "18056442",
               "18056705",
               "18097051",
               "18159951",
               "18160537",
               "18216278",
               "18246126",
               "18272544",
               "18289945",
               "18332866",
               "18339864",
               "18345031",
               "18381438",
               "18406507",
               "18431490",
               "18449195",
               "18490454",
               "18536714",
               "18560558",
               "18593910",
               "18604166",
               "18614045",
               "18669630",
               "18715874",
               "18718914",
               "18765533",
               "18769132",
               "18778462",
               "18785202",
               "18794113",
               "18818083",
               "18847491",
               "18985806",
               "18995830",
               "19085961",
               "19203586",
               "19251701",
               "19266268",
               "19303885",
               "19465479",
               "19477925",
               "19819244",
               "19828042",
               "19828454",
               "19857493",
               "19861417",
               "19882354",
               "19907922",
               "19934315",
               "19962312",
               "19965871",
               "20005840",
               "20009884",
               "20018442",
               "20024960",
               "20026654",
               "20041275",
               "20080565",
               "20123963",
               "20171273",
               "20353948",
               "20363803",
               "20372057",
               "20514025",
               "20562916",
               "20599567",
               "20610713",
               "20637859",
               "20661218",
               "20663147",
               "20673369",
               "20696760",
               "20711232",
               "20811699",
               "20931131",
               "20935676",
               "20962272",
               "21057547",
               "21148320",
               "21317932",
               "21383696",
               "21394211",
               "21423215",
               "21441950",
               "21454683",
               "21465263",
               "21532626",
               "21642861",
               "21660965",
               "21757780",
               "21765463",
               "21765464",
               "21832239",
               "22002314",
               "22011578",
               "22030623",
               "22037398",
               "22055193",
               "22099307",
               "22112863",
               "22179839",
               "22285752",
               "22728651",
               "22797063",
               "22815859",
               "22878263",
               "22975381",
               "23148227",
               "23149944",
               "23150668",
               "23184057",
               "23678107",
               "23798621",
               "23871434",
               "23890999",
               "23982736",
               "24038750",
               "24145406",
               "24196445",
               "24379358",
               "24657168",
               "24711418",
               "24820418",
               "24899407",
               "24958101",
               "25202122",
               "8327466",
               "9363941",
               "9372954",
               "9733515",
               "9744860",
               "9843217",
               "9925639"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"IGBP1-controls-state-change-of-TP53",
            "source":"IGBP1",
            "cited":0,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-BAX",
            "source":"TP53",
            "cited":95,
            "target":"BAX",
            "pubmed":[
               "10518217",
               "11080152",
               "11278953",
               "11684014",
               "11850816",
               "15598651",
               "17145718",
               "17189187"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-CCNB1",
            "source":"TP53",
            "cited":23,
            "target":"CCNB1",
            "pubmed":[
               "15710382"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SIRT1-controls-state-change-of-TP63",
            "source":"SIRT1",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAG1-controls-state-change-of-TP73",
            "source":"PRKAG1",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-MDM2",
            "source":"TP53",
            "cited":2784,
            "target":"MDM2",
            "pubmed":[
               "12138177",
               "17409421",
               "7651818",
               "8319905"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PPP2CB-controls-state-change-of-TP53",
            "source":"PPP2CB",
            "cited":0,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PPP2CA-controls-state-change-of-TP53",
            "source":"PPP2CA",
            "cited":5,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CHEK2-controls-state-change-of-TP63",
            "source":"CHEK2",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-CD82",
            "source":"TP53",
            "cited":44,
            "target":"CD82",
            "pubmed":[
               "9736732"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-FDXR",
            "source":"TP53",
            "cited":0,
            "target":"FDXR",
            "pubmed":[
               "12370809",
               "17409421"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-HTT",
            "source":"TP53",
            "cited":9,
            "target":"HTT",
            "pubmed":[
               "16278683"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-CTSD",
            "source":"TP53",
            "cited":0,
            "target":"CTSD",
            "pubmed":[
               "9619826"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SIRT1-controls-state-change-of-TP53",
            "source":"SIRT1",
            "cited":50,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-CASP10",
            "source":"TP53",
            "cited":0,
            "target":"CASP10",
            "pubmed":[
               "14688482"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-APAF1",
            "source":"TP53",
            "cited":26,
            "target":"APAF1",
            "pubmed":[
               "11559530"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-HGF",
            "source":"TP53",
            "cited":1,
            "target":"HGF",
            "pubmed":[
               "9023107"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRMT5-controls-state-change-of-TP53",
            "source":"PRMT5",
            "cited":0,
            "target":"TP53",
            "pubmed":[
               "19011621"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-APC",
            "source":"TP53",
            "cited":166,
            "target":"APC",
            "pubmed":[
               "11279192"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-BTG2",
            "source":"TP53",
            "cited":14,
            "target":"BTG2",
            "pubmed":[
               "11814693"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRMT1-controls-expression-of-GADD45A",
            "source":"PRMT1",
            "cited":0,
            "target":"GADD45A",
            "pubmed":[
               "15186775",
               "18250150"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-ASF1A",
            "source":"TP53",
            "cited":1,
            "target":"ASF1A",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-FAS",
            "source":"TP53",
            "cited":114,
            "target":"FAS",
            "pubmed":[
               "9841917"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"HTT-controls-state-change-of-TP73",
            "source":"HTT",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAG1-controls-state-change-of-TP53",
            "source":"PRKAG1",
            "cited":0,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-TRIAP1",
            "source":"TP53",
            "cited":2,
            "target":"TRIAP1",
            "pubmed":[
               "15735003"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-RFWD2",
            "source":"TP53",
            "cited":11,
            "target":"RFWD2",
            "pubmed":[
               "15103385"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"EP300-controls-state-change-of-TP63",
            "source":"EP300",
            "cited":2,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-COL18A1",
            "source":"TP53",
            "cited":1,
            "target":"COL18A1",
            "pubmed":[
               "15958553"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"EP300-controls-expression-of-GADD45A",
            "source":"EP300",
            "cited":0,
            "target":"GADD45A",
            "pubmed":[
               "15186775",
               "18250150"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAB2-controls-state-change-of-TP73",
            "source":"PRKAB2",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"EP300-controls-state-change-of-TP53",
            "source":"EP300",
            "cited":117,
            "target":"TP53",
            "pubmed":[
               "18485870"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid",
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAB1-controls-state-change-of-TP73",
            "source":"PRKAB1",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-HIST1H1B",
            "source":"TP53",
            "cited":2,
            "target":"HIST1H1B",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"EP300-controls-state-change-of-TP73",
            "source":"EP300",
            "cited":1,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-HIST1H1C",
            "source":"TP53",
            "cited":4,
            "target":"HIST1H1C",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-HIST1H1A",
            "source":"TP53",
            "cited":0,
            "target":"HIST1H1A",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CARM1-controls-expression-of-GADD45A",
            "source":"CARM1",
            "cited":0,
            "target":"GADD45A",
            "pubmed":[
               "15186775",
               "18250150"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-HIST1H1D",
            "source":"TP53",
            "cited":0,
            "target":"HIST1H1D",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-HIST1H1E",
            "source":"TP53",
            "cited":0,
            "target":"HIST1H1E",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"WRN-controls-state-change-of-TP53",
            "source":"WRN",
            "cited":46,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-MSH2",
            "source":"TP53",
            "cited":109,
            "target":"MSH2",
            "pubmed":[
               "10984493",
               "11350971"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"KAT2B-controls-state-change-of-TP53",
            "source":"KAT2B",
            "cited":34,
            "target":"TP53",
            "pubmed":[
               "17110336",
               "9744860",
               "9891054"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-PYCARD",
            "source":"TP53",
            "cited":13,
            "target":"PYCARD",
            "pubmed":[
               "14730312"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAA2-controls-state-change-of-TP53",
            "source":"PRKAA2",
            "cited":5,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SP1-controls-expression-of-CCNB1",
            "source":"SP1",
            "cited":1,
            "target":"CCNB1",
            "pubmed":[
               "15710382"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAA1-controls-state-change-of-TP53",
            "source":"PRKAA1",
            "cited":1,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10570149",
               "10606744",
               "10656795",
               "10673500",
               "10673501",
               "10706102",
               "10710310",
               "10733583",
               "10734067",
               "11030628",
               "11175334",
               "11244509",
               "11258706",
               "11397945",
               "11423984",
               "11447225",
               "11495913",
               "11526498",
               "11551930",
               "11583595",
               "11707453",
               "11850826",
               "11861384",
               "11875057",
               "11883897",
               "11896572",
               "11896587",
               "12091386",
               "12384533",
               "12427754",
               "12531896",
               "12676607",
               "12756247",
               "12810724",
               "12860987",
               "12861053",
               "12902982",
               "12955074",
               "12959929",
               "12969974",
               "14517211",
               "14527925",
               "14612532",
               "14665630",
               "15064747",
               "15140942",
               "15254178",
               "15269203",
               "15322239",
               "15355354",
               "15471885",
               "15489221",
               "15542844",
               "15629715",
               "15671037",
               "15750624",
               "15843377",
               "15963507",
               "16377624",
               "16651424",
               "16731759",
               "16774943",
               "16818520",
               "17105820",
               "17121812",
               "17135248",
               "17245430",
               "17339337",
               "17353187",
               "17363488",
               "17698850",
               "17967874",
               "17977830",
               "18159951",
               "18246126",
               "18593910",
               "18669630",
               "18718914",
               "18778462",
               "18794113",
               "19819244",
               "19861417",
               "19882354",
               "20009884",
               "20024960",
               "20041275",
               "20562916",
               "20673369",
               "20696760",
               "20962272",
               "21148320",
               "21383696",
               "21386980",
               "21454683",
               "21832239",
               "22030623",
               "22112863",
               "22285752",
               "22728651",
               "22797063",
               "22815859",
               "23798621",
               "23871434",
               "24657168",
               "8327466"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite",
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-GPX1",
            "source":"TP53",
            "cited":10,
            "target":"GPX1",
            "pubmed":[
               "16140933"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-state-change-of-CABIN1",
            "source":"TP53",
            "cited":6,
            "target":"CABIN1",
            "pubmed":[
               "15621527",
               "15655109",
               "16901784",
               "17158953",
               "17242198",
               "17242207",
               "23964094"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "Reactome"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"MDM2-controls-state-change-of-NEDD8",
            "source":"MDM2",
            "cited":10,
            "target":"NEDD8",
            "pubmed":[
               "15242646"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PLK3-controls-state-change-of-TP53",
            "source":"PLK3",
            "cited":16,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10570149",
               "10656795",
               "10673500",
               "10673501",
               "10706102",
               "10710310",
               "10734067",
               "11030628",
               "11175334",
               "11244509",
               "11258706",
               "11397945",
               "11423984",
               "11447225",
               "11495913",
               "11526498",
               "11551930",
               "11583595",
               "11707453",
               "11850826",
               "11861384",
               "11875057",
               "11896572",
               "11896587",
               "12091386",
               "12384533",
               "12427754",
               "12531896",
               "12676607",
               "12756247",
               "12810724",
               "12860987",
               "12861053",
               "12902982",
               "12955074",
               "12959929",
               "12969974",
               "14517211",
               "14527925",
               "14612532",
               "14665630",
               "15064747",
               "15140942",
               "15254178",
               "15269203",
               "15322239",
               "15471885",
               "15489221",
               "15629715",
               "15671037",
               "15750624",
               "15843377",
               "15963507",
               "16377624",
               "16651424",
               "16731759",
               "16774943",
               "16818520",
               "17105820",
               "17121812",
               "17135248",
               "17245430",
               "17339337",
               "17353187",
               "17363488",
               "17698850",
               "17967874",
               "17977830",
               "18159951",
               "18593910",
               "18669630",
               "18718914",
               "18778462",
               "19819244",
               "19861417",
               "19882354",
               "20009884",
               "20024960",
               "20041275",
               "20562916",
               "20673369",
               "20696760",
               "20962272",
               "21148320",
               "21383696",
               "21454683",
               "21832239",
               "22030623",
               "22112863",
               "22285752",
               "22728651",
               "22797063",
               "22815859",
               "23798621",
               "23871434",
               "24657168",
               "8327466"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite",
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAB1-controls-state-change-of-TP53",
            "source":"PRKAB1",
            "cited":5,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SUMO1-controls-state-change-of-TP53",
            "source":"SUMO1",
            "cited":39,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAA1-controls-state-change-of-TP63",
            "source":"PRKAA1",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PPP1R13L-controls-state-change-of-TP53",
            "source":"PPP1R13L",
            "cited":13,
            "target":"TP53",
            "pubmed":[
               "10581258",
               "11740489",
               "11780126",
               "16377624",
               "17349958",
               "17906639"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"TP53-controls-expression-of-PLK3",
            "source":"TP53",
            "cited":16,
            "target":"PLK3",
            "pubmed":[
               "16140933"
            ],
            "isdirected":true,
            "type":"controls-expression-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAA2-controls-state-change-of-TP63",
            "source":"PRKAA2",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAB2-controls-state-change-of-TP53",
            "source":"PRKAB2",
            "cited":0,
            "target":"TP53",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"DAPK1-controls-state-change-of-TP53",
            "source":"DAPK1",
            "cited":6,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10570149",
               "10606744",
               "10656795",
               "10673500",
               "10673501",
               "10706102",
               "10710310",
               "10733583",
               "10734067",
               "11030628",
               "11175334",
               "11244509",
               "11258706",
               "11397945",
               "11423984",
               "11447225",
               "11495913",
               "11526498",
               "11551930",
               "11583595",
               "11707453",
               "11850826",
               "11861384",
               "11875057",
               "11883897",
               "11896572",
               "11896587",
               "12091386",
               "12384533",
               "12427754",
               "12531896",
               "12676607",
               "12756247",
               "12810724",
               "12860987",
               "12861053",
               "12902982",
               "12955074",
               "12959929",
               "12969974",
               "14517211",
               "14527925",
               "14612532",
               "14665630",
               "15064747",
               "15140942",
               "15254178",
               "15269203",
               "15322239",
               "15355354",
               "15471885",
               "15489221",
               "15542844",
               "15629715",
               "15671037",
               "15750624",
               "15843377",
               "15963507",
               "16377624",
               "16651424",
               "16731759",
               "16774943",
               "16818520",
               "17105820",
               "17121812",
               "17135248",
               "17245430",
               "17339337",
               "17353187",
               "17363488",
               "17698850",
               "17967874",
               "17977830",
               "18159951",
               "18246126",
               "18593910",
               "18669630",
               "18718914",
               "18778462",
               "18794113",
               "19819244",
               "19861417",
               "19882354",
               "20009884",
               "20024960",
               "20041275",
               "20562916",
               "20673369",
               "20696760",
               "20847049",
               "20851891",
               "20959462",
               "20962272",
               "21148320",
               "21383696",
               "21386980",
               "21454683",
               "21832239",
               "22030623",
               "22112863",
               "22285752",
               "22728651",
               "22797063",
               "22815859",
               "23798621",
               "23871434",
               "24657168",
               "8327466"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"CDK1-controls-state-change-of-TP53",
            "source":"CDK1",
            "cited":47,
            "target":"TP53",
            "pubmed":[
               "10644693",
               "10656795",
               "11078726",
               "11300786",
               "11687964",
               "11875057",
               "12021785",
               "12397361",
               "12397362",
               "12860987",
               "1406679",
               "14640983",
               "14702041",
               "14744935",
               "15302935",
               "15619621",
               "16037820",
               "16227590",
               "16552184",
               "16674116",
               "16784539",
               "17287340",
               "17906639",
               "17942552",
               "18521083",
               "18769132",
               "18847512",
               "19413330",
               "19574224",
               "19664995",
               "20363803",
               "20562916",
               "20661218",
               "20686112",
               "20808790",
               "20815410",
               "21659604",
               "21983960",
               "22011578",
               "22496350",
               "22547687",
               "23201157",
               "23312004",
               "25159151",
               "7596441",
               "8327466",
               "9254608"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"DAPK3-controls-state-change-of-TP53",
            "source":"DAPK3",
            "cited":0,
            "target":"TP53",
            "pubmed":[
               "10202145",
               "10570149",
               "10656795",
               "10673500",
               "10673501",
               "10706102",
               "10710310",
               "10734067",
               "11030628",
               "11175334",
               "11244509",
               "11258706",
               "11397945",
               "11423984",
               "11447225",
               "11495913",
               "11526498",
               "11551930",
               "11583595",
               "11707453",
               "11850826",
               "11861384",
               "11875057",
               "11896572",
               "11896587",
               "12091386",
               "12384533",
               "12427754",
               "12531896",
               "12676607",
               "12756247",
               "12810724",
               "12860987",
               "12861053",
               "12902982",
               "12955074",
               "12959929",
               "12969974",
               "14517211",
               "14527925",
               "14612532",
               "14665630",
               "15064747",
               "15140942",
               "15254178",
               "15269203",
               "15322239",
               "15471885",
               "15489221",
               "15629715",
               "15671037",
               "15750624",
               "15843377",
               "15963507",
               "16377624",
               "16651424",
               "16731759",
               "16774943",
               "16818520",
               "17105820",
               "17121812",
               "17135248",
               "17245430",
               "17339337",
               "17353187",
               "17363488",
               "17698850",
               "17967874",
               "17977830",
               "18159951",
               "18593910",
               "18669630",
               "18718914",
               "18778462",
               "19819244",
               "19861417",
               "19882354",
               "20009884",
               "20024960",
               "20041275",
               "20562916",
               "20673369",
               "20696760",
               "20962272",
               "21148320",
               "21383696",
               "21454683",
               "21832239",
               "22030623",
               "22112863",
               "22285752",
               "22728651",
               "22797063",
               "22815859",
               "23798621",
               "23871434",
               "24657168",
               "8327466"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PhosphoSite"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAB1-controls-state-change-of-TP63",
            "source":"PRKAB1",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"SUMO1-controls-state-change-of-TP63",
            "source":"SUMO1",
            "cited":1,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAA1-controls-state-change-of-TP73",
            "source":"PRKAA1",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"E4F1-controls-state-change-of-TP53",
            "source":"E4F1",
            "cited":12,
            "target":"TP53",
            "pubmed":[
               "17110336",
               "9288740"
            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "pid"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAA2-controls-state-change-of-TP73",
            "source":"PRKAA2",
            "cited":0,
            "target":"TP73",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      },
      {
         "data":{
            "id":"PRKAB2-controls-state-change-of-TP63",
            "source":"PRKAB2",
            "cited":0,
            "target":"TP63",
            "pubmed":[

            ],
            "isdirected":true,
            "type":"controls-state-change-of",
            "datasource":[
               "PANTHER"
            ]
         },
         "position":{

         },
         "group":"edges",
         "removed":false,
         "selected":false,
         "selectable":true,
         "locked":false,
         "grabbable":true,
         "classes":""
      }
   ]
}""")
len(tmp_dict)

376139